# Final Goal
## Make a database that contains all the apperance for houses that involved in races on 20150101 and onward
- As we will have time-lag features
- That means I will crawl all the historical record of houses that have appeared in the interested period (20150101 - Present)
- So form record will be larger and a race result complement will be crawled for form record normailzation

- Besides, i found that some races or even race day will be skipped when crawling. Not sure if because my computer do not have enough ram or whatever other reasons.
- So some steps i took below may worth learning as well.

### It is a project built for profit. Therefore, leakage and overfitting are the first thing to avoid which makes this project a good project for portfolio demonstration as well.

In [1]:
#import modin.pandas as pd #pip install modin
import pandas as pd
form_record = pd.read_csv('../scarping/done/form_record.csv', encoding= 'unicode_escape')
#prize_money = pd.read_csv('C:/Users/chunl/Desktop/code_data/scarping/done/prize_money.csv', dtype = {"match" : "int"})
race_result = pd.read_csv('..//scarping/done/race_result.csv', encoding= 'unicode_escape')
racecard = pd.read_csv('..//scarping/done/racecard.csv')
section_time = pd.read_csv('..//scarping/done/section_time.csv')

C:\Users\carlos.lau\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3418: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
# Racecard will not include horses that withdrawn from races for any reasons (but also with some exception) while race result will include all horses in a race. 
# Therefore, race result should not have any missing compared to racecard. We can make sure race_result is completed first by comparing it to racecard

# create brand_no for checking purposes
race_result["brand_no"] = race_result['horseid'].str[-4:]

race_result['check'] = race_result['date'].astype(str) + "_" + race_result['match'].astype(str) + "_" + race_result['brand_no'].astype(str)
racecard['check'] = racecard['date'].astype(str) + "_" +  racecard['match'].astype(str) + "_" + racecard['Brand No.'].astype(str)

# print the first 5 row for preview
racecard.head()

,Horse No.,Colour,Horse,Brand No.,Wt.,Jockey,Over Wt.,Draw,Trainer,Rtg.,...,Season Stakes,Priority,Gear,Owner,Sire,Dam,Import Cat.,date,match,check
0,1,NaN,NAMJONG TURBO,N250,131,K C Leung(-2),NaN,11,P O'Sullivan,38,...,293250,1,SR,Ivis Sze,Nadeem,Fine Return,PPG,20150101,1,20150101_1_N250
1,2,NaN,FOUR FOUR FIVE,P445,130,U Rispoli,NaN,4,R Gibson,37,...,126500,2,B/TT,Super Diamond Syndicate,Al Maher,Top Tips,PPG,20150101,1,20150101_1_P445
2,3,NaN,MY SURPLUS,S016,130,D Whyte,NaN,12,C H Yip,37,...,0,*1,B1/TT,Andy Wu Bark To,Dane Shadow,Anaheart,PPG,20150101,1,20150101_1_S016
3,4,NaN,HAPPY KAISER,N130,128,N Callan,NaN,14,K L Man,35,...,66125,1,H,Patrick Luk Wing Tong,Commands,Clean,PPG,20150101,1,20150101_1_N130
4,5,NaN,CASA JUNIOR,M366,127,K Teetan,NaN,9,T K Ng,34,...,34500,1,CP,Wu Wei Lam & Kenneth Wu Kai Chung,Danehill Dancer,Mystifying,PP,20150101,1,20150101_1_M366


In [3]:
print('racecard is missing:', list(set(race_result['check'].unique()) - set(racecard['check'].unique())))
print("\n")

# race_result has no missing which means the race result is completed compared to racecard
# racecard has missing because of (WV/WX) matches or 20190323

print('race_result is missing:', list(set(racecard['check'].unique()) - set(race_result['check'].unique())))

racecard is missing: ['20160605_3_T426', '20160921_5_P344', '20151219_6_P387', '20160918_2_V010', '20161023_8_V194', '20191027_3_B367', '20190529_1_A327', '20150520_3_S074', '20190130_4_V014', '20150701_10_P029', '20190925_4_B150', '20180624_10_S142', '20150503_3_S419', '20160701_7_T114', '20170402_6_A049', '20190323_2_V202', '20160206_7_S247', '20200405_4_D053', '20160622_7_S420', '20150125_10_P254', '20190106_2_B063', '20170604_3_T133', '20161008_2_S254', '20190317_4_B368', '20191001_5_A086', '20160113_4_N220', '20190511_10_B049', '20190202_7_B332', '20160619_11_T176', '20170125_2_N166', '20190109_3_B096', '20160701_5_S231', '20160925_10_V166', '20181010_2_P190', '20190217_4_S212', '20190323_5_A249', '20170226_6_V088', '20190422_10_B433', '20170101_9_N436', '20160403_2_M152', '20191226_5_V362', '20160410_3_S424', '20200101_9_V142', '20180225_5_T171', '20191023_9_A379', '20150708_6_S004', '20160124_7_T403', '20190323_2_V164', '20180411_6_A362', '20150426_5_P093', '20151114_4_L401', '2

In [4]:
# random check the corresponding missing value in racecard

print('racecard is missing:', len(list(set(race_result['check'].unique()) - set(racecard['check'].unique()))))

l = list(set(race_result['check'].unique()) - set(racecard['check'].unique()))
print(race_result.loc[race_result['check'].isin(l)]['Plc.'].value_counts(dropna=False))


# matches with place is the match in 20190323 which is the only one match held in Conghua
# https://racing.hkjc.com/racing/information/english/Racing/LocalResults.aspx?RaceDate=2019/03/23&Racecourse=CH&RaceNo=1
race_result.loc[(race_result['check'].isin(l)) & (race_result['Plc.'] != 'WV') & (race_result['Plc.'] != 'WX'), 'date']

racecard is missing: 680
WV    618
WX     18
1       5
2       5
3       5
4       5
5       5
6       5
7       4
8       4
9       4
10      2
Name: Plc., dtype: int64


42979    20190323
42980    20190323
42981    20190323
42982    20190323
42983    20190323
42984    20190323
42985    20190323
42986    20190323
42987    20190323
42988    20190323
42989    20190323
42990    20190323
42991    20190323
42992    20190323
42993    20190323
42994    20190323
42995    20190323
42996    20190323
42997    20190323
42998    20190323
42999    20190323
43000    20190323
43001    20190323
43002    20190323
43003    20190323
43005    20190323
43006    20190323
43007    20190323
43008    20190323
43009    20190323
43010    20190323
43011    20190323
43012    20190323
43013    20190323
43015    20190323
43016    20190323
43017    20190323
43018    20190323
43019    20190323
43020    20190323
43021    20190323
43022    20190323
43023    20190323
43024    20190323
Name: date, dtype: int64

In [5]:
# cross-check race_result and racecard by dropping (WV/WX) matches
# after dropping, racecard should only have 20190323 missing

race_result_tmp = race_result[~race_result['Plc.'].isin(['WV', 'WX'])]

print('racecard is missing(after dropping WV/WX):', list(set(race_result_tmp['check'].unique()) - set(racecard['check'].unique())))
print('\n')
print('race_result is missing:', list(set(racecard['check'].unique()) - set(race_result_tmp['check'].unique())))

racecard is missing(after dropping WV/WX): ['20190323_4_V123', '20190323_2_A055', '20190323_2_V164', '20190323_3_B073', '20190323_3_V126', '20190323_3_S253', '20190323_4_B375', '20190323_4_V398', '20190323_1_A308', '20190323_1_B371', '20190323_3_A064', '20190323_4_V412', '20190323_2_V202', '20190323_5_B137', '20190323_4_A296', '20190323_4_A336', '20190323_5_V369', '20190323_5_V039', '20190323_2_P324', '20190323_3_B057', '20190323_1_A314', '20190323_1_B445', '20190323_2_B233', '20190323_4_A305', '20190323_2_A285', '20190323_2_S417', '20190323_2_B297', '20190323_5_S147', '20190323_5_B345', '20190323_5_A333', '20190323_3_B013', '20190323_5_S015', '20190323_4_V016', '20190323_4_A007', '20190323_5_A249', '20190323_3_T372', '20190323_1_A319', '20190323_3_A163', '20190323_2_T431', '20190323_1_B374', '20190323_5_B146', '20190323_3_A254', '20190323_2_A081', '20190323_5_V149']


race_result is missing: ['20160518_9_T371', '20150527_2_S300', '20180121_9_B105', '20180207_5_A330', '20180909_6_A305'

In [6]:
# random check the corresponding missing in race_result
# it should refer to withdrawn in racecard

print('race_result is missing:', len(list(set(racecard['check'].unique()) - set(race_result_tmp['check'].unique()))))

l = list(set(racecard['check'].unique()) - set(race_result_tmp['check'].unique()))
print(racecard.loc[racecard['check'].isin(l)]['Horse'].value_counts(dropna=False), '\n')

# All the missing in race result is withdrawn horses
print(racecard.loc[racecard['check'].isin(l)]['Horse'].str.contains("Withdrawn").value_counts())


race_result is missing: 288
PENANG HALL(Withdrawn)          3
TRAVEL FIRST(Withdrawn)         3
BORN IN CHINA(Withdrawn)        3
JOYFUL PARK(Withdrawn)          2
CHEERFUL STAR(Withdrawn)        2
                               ..
GIANT TREASURE(Withdrawn)       1
BALAY BALAY(Withdrawn)          1
SWISS WARRIOR(Withdrawn)        1
INVINCIBLE DRAGON(Withdrawn)    1
SUPER CHIC(Withdrawn)           1
Name: Horse, Length: 262, dtype: int64 

True    288
Name: Horse, dtype: int64


In [7]:
'''
# start merging
# as WV/WX is all blank in racecard, it contains no addtional information for race_result
# so we only keep row in race_result

print(race_result.shape)
print(racecard.shape)

race_result = race_result.merge(racecard, on = ['check'], how='left', suffixes=('', '_dup'))

print(race_result.shape)
'''

"\n# start merging\n# as WV/WX is all blank in racecard, it contains no addtional information for race_result\n# so we only keep row in race_result\n\nprint(race_result.shape)\nprint(racecard.shape)\n\nrace_result = race_result.merge(racecard, on = ['check'], how='left', suffixes=('', '_dup'))\n\nprint(race_result.shape)\n"

In [8]:
#race_result.columns

In [9]:
'''race_result = race_result[['Plc.', 'Horse No.', 'Horse', 'Jockey', 'Trainer', 'Actual Wt.',
       'Declar. Horse Wt.', 'Draw', 'LBW', 'RunningPosition', 'Finish Time',
       'Win Odds', 'date', 'match', 'cdr', 'horse_name', 'horseid',
       'date_match', 'prize_money', 'brand_no', 'check', 
        'Brand No.', 'Wt.', 'Over Wt.', 'Rtg.', 'Rtg.+/-', 'Horse Wt. (Declaration)',
       'Wt.+/- (vs Declaration)', 'Best Time', 'Age', 'WFA',
       'Season Stakes', 'Priority', 'Gear']]'''

"race_result = race_result[['Plc.', 'Horse No.', 'Horse', 'Jockey', 'Trainer', 'Actual Wt.',\n       'Declar. Horse Wt.', 'Draw', 'LBW', 'RunningPosition', 'Finish Time',\n       'Win Odds', 'date', 'match', 'cdr', 'horse_name', 'horseid',\n       'date_match', 'prize_money', 'brand_no', 'check', \n        'Brand No.', 'Wt.', 'Over Wt.', 'Rtg.', 'Rtg.+/-', 'Horse Wt. (Declaration)',\n       'Wt.+/- (vs Declaration)', 'Best Time', 'Age', 'WFA',\n       'Season Stakes', 'Priority', 'Gear']]"

In [10]:
#race_result.to_csv (r'C:\Users\chunl\Desktop\race_result_racecard.csv', index = False, header=True)

In [11]:
'''
Reaching here means we have successfully merged race_result, prize_money (which is a mistake) and racecard.
'''

# Only form_record and sectional time are left out.
# We can do FE on form_record with the aid of race_result_comp first in order to maintain a full df for race_result

'\nReaching here means we have successfully merged race_result, prize_money (which is a mistake) and racecard.\n'

In [12]:
# As the FE on form_record may vary, it may be good to output the current df (race_result + prize_money + racecard)

### form_record ###

In [13]:
form_record.head()

,RaceIndex,Pla,Date,RC/Track/Course,Dist,Ground,RaceClass,Draw,Rating,Trainer,Jockey,LBW,WinOdds,ActWt,RunPo,FinishTime,Declare_Horse_Wt,Gear,VideoReplay,horseid
0,583,11,2016/04/24,"ST / Turf / ""A""",1400,G,5,9,36,A Schutz,N Rawiller,5 1/2,9.2,130,10 11 10 11,1.23.72,1055,B,NaN,HK_2011_N208
1,546,2,2016/04/10,"ST / Turf / ""C""",1400,GY,5,9,35,A Schutz,N Rawiller,3/4,15,128,8 9 9 2,1.23.80,1054,B,NaN,HK_2011_N208
2,508,8,2016/03/28,ST / AWT,1200,GD,5,7,37,A Schutz,N Rawiller,4 1/2,31,130,8 7 8,1.10.31,1062,B,NaN,HK_2011_N208
3,417,8,2016/02/21,"ST / Turf / ""A""",1400,G,5,11,39,A Schutz,N Rawiller,3 1/4,31,124,13 12 12 8,1.24.28,1064,B,NaN,HK_2011_N208
4,359,12,2016/01/31,"ST / Turf / ""A+3""",1200,G,4,13,41,A Schutz,S de Sousa,8 1/4,39,115,11 11 12,1.12.17,1075,B,NaN,HK_2011_N208


In [14]:
# create a formatted date on form_record

form_record['formatted_date'] = form_record['Date'].str[0:4] + form_record['Date'].str[5:7] + form_record['Date'].str[8:11]
form_record['formatted_date'] = form_record['formatted_date'].astype(int)
form_record.head()

,RaceIndex,Pla,Date,RC/Track/Course,Dist,Ground,RaceClass,Draw,Rating,Trainer,...,LBW,WinOdds,ActWt,RunPo,FinishTime,Declare_Horse_Wt,Gear,VideoReplay,horseid,formatted_date
0,583,11,2016/04/24,"ST / Turf / ""A""",1400,G,5,9,36,A Schutz,...,5 1/2,9.2,130,10 11 10 11,1.23.72,1055,B,NaN,HK_2011_N208,20160424
1,546,2,2016/04/10,"ST / Turf / ""C""",1400,GY,5,9,35,A Schutz,...,3/4,15,128,8 9 9 2,1.23.80,1054,B,NaN,HK_2011_N208,20160410
2,508,8,2016/03/28,ST / AWT,1200,GD,5,7,37,A Schutz,...,4 1/2,31,130,8 7 8,1.10.31,1062,B,NaN,HK_2011_N208,20160328
3,417,8,2016/02/21,"ST / Turf / ""A""",1400,G,5,11,39,A Schutz,...,3 1/4,31,124,13 12 12 8,1.24.28,1064,B,NaN,HK_2011_N208,20160221
4,359,12,2016/01/31,"ST / Turf / ""A+3""",1200,G,4,13,41,A Schutz,...,8 1/4,39,115,11 11 12,1.12.17,1075,B,NaN,HK_2011_N208,20160131


### race_result_comp

In [15]:
# Find all the date of historical races before 2015/01/01 which horses in our intereseted period are involved
# You can think of we are building a database that contain a full record of our interested horses.

# races that needed to be crawl as complement
print("race_result_comp:", list(form_record[form_record['formatted_date'] < 20150101].formatted_date.sort_values().unique()))

# number of races
len(form_record[form_record['formatted_date'] < 20150101].formatted_date.unique())

race_result_comp: [20071215, 20080209, 20080309, 20080921, 20081019, 20081026, 20081108, 20081116, 20081130, 20090101, 20090104, 20090107, 20090128, 20090211, 20090215, 20090222, 20090301, 20090307, 20090315, 20090322, 20090328, 20090405, 20090413, 20090422, 20090426, 20090501, 20090506, 20090517, 20090521, 20090531, 20090603, 20090607, 20090613, 20090621, 20090624, 20090628, 20090701, 20090913, 20090920, 20091001, 20091004, 20091007, 20091011, 20091017, 20091021, 20091025, 20091101, 20091104, 20091107, 20091115, 20091118, 20091122, 20091125, 20091129, 20091202, 20091206, 20091209, 20091213, 20091216, 20091219, 20091223, 20091227, 20100101, 20100106, 20100110, 20100113, 20100116, 20100120, 20100131, 20100203, 20100207, 20100210, 20100216, 20100221, 20100224, 20100228, 20100303, 20100306, 20100310, 20100314, 20100321, 20100324, 20100328, 20100331, 20100404, 20100410, 20100418, 20100421, 20100425, 20100501, 20100505, 20100508, 20100512, 20100516, 20100519, 20100523, 20100526, 20100530, 2

471

In [16]:
race_result.columns

Index(['Plc.', 'Horse No.', 'Horse', 'Jockey', 'Trainer', 'Actual Wt.',
       'Declar. Horse Wt.', 'Draw', 'LBW', 'RunningPosition', 'Finish Time',
       'Win Odds', 'date', 'match', 'cdr', 'prize_money', 'horse_name',
       'horseid', 'brand_no', 'check'],
      dtype='object')

In [17]:
# check any horse is missing first
# It should be blank

print('form_record is missing:', list(set(race_result['horseid'].unique()) - set(form_record['horseid'].unique())), '\n')
print('race_result is missing:', list(set(form_record['horseid'].unique()) - set(race_result['horseid'].unique())))

form_record is missing: [] 

race_result is missing: []


In [18]:
# check if any races between 20150101 to 2020 in race_result is missing from form_record

form_record_tmp = form_record[form_record['formatted_date'] >= 20150101]

# create for checking only
form_record_tmp['ind'] = form_record_tmp['horseid'] + "_" + form_record_tmp['formatted_date'].astype(str)
race_result['ind'] = race_result['horseid'] + "_" + race_result['date'].astype(str)

# form record should only miss 20161026 match 4 (VOID match) and race result should have no missing
# https://racing.hkjc.com/racing/information/english/Racing/LocalResults.aspx?RaceDate=2016/10/26&Racecourse=HV&RaceNo=4
print('form_record is missing:', list(set(race_result['ind'].unique()) - set(form_record_tmp['ind'].unique())))
print('race_result is missing:', list(set(form_record_tmp['ind'].unique()) - set(race_result['ind'].unique())))

form_record is missing: ['HK_2015_V201_20161026', 'HK_2015_V271_20161026', 'HK_2015_V221_20161026', 'HK_2015_V243_20161026', 'HK_2014_T157_20161026', 'HK_2015_V387_20161026', 'HK_2014_T361_20161026', 'HK_2015_V344_20161026', 'HK_2015_V315_20161026', 'HK_2015_V197_20161026', 'HK_2015_V286_20161026', 'HK_2014_T076_20161026']
race_result is missing: []


C:\Users\carlos.lau\Anaconda3\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [19]:
# check the whole form_record (not temp) which should be much longer

form_record['ind'] = form_record['horseid'] + "_" + form_record['formatted_date'].astype(str)
race_result['ind'] = race_result['horseid'] + "_" + race_result['date'].astype(str)

print('form_record is missing:', list(set(race_result['ind'].unique()) - set(form_record['ind'].unique()))) # void match
print("\n")
print('race_result is missing:', len(list(set(form_record['ind'].unique()) - set(race_result['ind'].unique()))))

form_record is missing: ['HK_2015_V201_20161026', 'HK_2015_V271_20161026', 'HK_2015_V221_20161026', 'HK_2015_V243_20161026', 'HK_2014_T157_20161026', 'HK_2015_V387_20161026', 'HK_2014_T361_20161026', 'HK_2015_V344_20161026', 'HK_2015_V315_20161026', 'HK_2015_V197_20161026', 'HK_2015_V286_20161026', 'HK_2014_T076_20161026']


race_result is missing: 16087


In [20]:
print(len(form_record))
print(len(race_result))

# (len diff) form_record - race_result = houses' past record before 20150101
len(form_record) - len(race_result)

72291
56216


16075

In [21]:
# record before 20150101 (missing in race_result) - missing in form_record = form record diff
16087 - 16075

12

### check race_result_comp and form_record ###

In [22]:
# check race_result_comp and form_record

race_result_comp = pd.read_csv('../scarping/race_result/race_result_comp.csv')

C:\Users\carlos.lau\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3418: DtypeWarning: Columns (5,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [23]:
# race_result_comp contains matches in 2007-2014

# Races 20100530 match 10 and 11 are cancelled race
# https://racing.hkjc.com/racing/information/english/Racing/LocalResults.aspx?RaceDate=2010/05/30&Racecourse=ST&RaceNo=10
race_result_comp.loc[race_result_comp['Plc.'].isna()]

,Plc.,Horse No.,Horse,Jockey,Trainer,Actual Wt.,Declar. Horse Wt.,Draw,LBW,RunningPosition,Finish Time,Win Odds,date,match,cdr,prize_money,Unnamed: 16,horseid
46854,NaN,1.0,SOARING DRAGON(H183),B Prebble,J Moore,,,10,,,20100530,10,Class 3 - 1600M - (80-60),"850,000",SOARING DRAGON,HK_2006_H183,NaN,NaN
46855,NaN,2.0,CHAMPION MOUNT(K087),Z Purton,S Woods,,,13,,,20100530,10,Class 3 - 1600M - (80-60),"850,000",CHAMPION MOUNT,HK_2008_K087,NaN,NaN
46856,NaN,3.0,MASTER ELITE(J210),J Lloyd,C W Chang,,,14,,,20100530,10,Class 3 - 1600M - (80-60),"850,000",MASTER ELITE,HK_2007_J210,NaN,NaN
46857,NaN,4.0,OCEAN PRINCE(K066),B Doyle,D Cruz,,,3,,,20100530,10,Class 3 - 1600M - (80-60),"850,000",OCEAN PRINCE,HK_2008_K066,NaN,NaN
46858,NaN,5.0,AMAZING JOURNEY(H281),K C Leung,P O'Sullivan,,,1,,,20100530,10,Class 3 - 1600M - (80-60),"850,000",AMAZING JOURNEY,HK_2006_H281,NaN,NaN
46859,NaN,6.0,BRAVE SPIRIT(J258),C W Wong,A S Cruz,,,9,,,20100530,10,Class 3 - 1600M - (80-60),"850,000",BRAVE SPIRIT,HK_2007_J258,NaN,NaN
46860,NaN,7.0,VANQUISH DELIGHT(J257),M Du Plessis,C Fownes,,,6,,,20100530,10,Class 3 - 1600M - (80-60),"850,000",VANQUISH DELIGHT,HK_2007_J257,NaN,NaN
46861,NaN,8.0,GIVE WELL(J109),G Mosse,A Lee,,,4,,,20100530,10,Class 3 - 1600M - (80-60),"850,000",GIVE WELL,HK_2007_J109,NaN,NaN
46862,NaN,9.0,SILVER SUN(CE126),K W Leung,Y S Tsui,,,7,,,20100530,10,Class 3 - 1600M - (80-60),"850,000",SILVER SUN,HK_2004_E126,NaN,NaN
46863,NaN,10.0,THUNDER DANCING(G250),Y T Cheng,Y O Wong,,,5,,,20100530,10,Class 3 - 1600M - (80-60),"850,000",THUNDER DANCING,HK_2005_G250,NaN,NaN


In [24]:
# after adjustment it should be blank
race_result_comp.dropna(subset=['Plc.'], inplace=True)

# for checking only
form_record['ind'] = form_record['horseid'] + "_" + form_record['formatted_date'].astype(str)
race_result_comp['ind'] = race_result_comp['horseid'] + "_" + race_result_comp['date'].astype(str)

# race result comp should have no missing
form_record_tmp = form_record[form_record['formatted_date'] < 20150101]
print('race_result_comp is missing:', list(set(form_record_tmp['ind'].unique()) - set(race_result_comp['ind'].unique())))

race_result_comp is missing: []


In [25]:
# Check the columns of two dataframe see if it is identical
# If not make it identical and then concat two df


# Old races did not have running position
# (Ignore)It is a great chance to see if your new script can address the issue
# (It is after all two different dataframe)
# Also this observation (no running position) can also be mentioned in the final presentation.

print(race_result.columns)
print(race_result_comp.columns)

Index(['Plc.', 'Horse No.', 'Horse', 'Jockey', 'Trainer', 'Actual Wt.',
       'Declar. Horse Wt.', 'Draw', 'LBW', 'RunningPosition', 'Finish Time',
       'Win Odds', 'date', 'match', 'cdr', 'prize_money', 'horse_name',
       'horseid', 'brand_no', 'check', 'ind'],
      dtype='object')
Index(['Plc.', 'Horse No.', 'Horse', 'Jockey', 'Trainer', 'Actual Wt.',
       'Declar. Horse Wt.', 'Draw', 'LBW', 'RunningPosition', 'Finish Time',
       'Win Odds', 'date', 'match', 'cdr', 'prize_money', 'Unnamed: 16',
       'horseid', 'ind'],
      dtype='object')


In [26]:
race_result_comp.rename(columns={'Unnamed: 16': "horse_name"}, inplace=True)
race_result.drop(['brand_no', 'check', 'ind'], axis=1, inplace=True)
race_result_comp.drop('ind', axis=1, inplace=True)

print("\n")
print(race_result.columns)
print(race_result_comp.columns)



Index(['Plc.', 'Horse No.', 'Horse', 'Jockey', 'Trainer', 'Actual Wt.',
       'Declar. Horse Wt.', 'Draw', 'LBW', 'RunningPosition', 'Finish Time',
       'Win Odds', 'date', 'match', 'cdr', 'prize_money', 'horse_name',
       'horseid'],
      dtype='object')
Index(['Plc.', 'Horse No.', 'Horse', 'Jockey', 'Trainer', 'Actual Wt.',
       'Declar. Horse Wt.', 'Draw', 'LBW', 'RunningPosition', 'Finish Time',
       'Win Odds', 'date', 'match', 'cdr', 'prize_money', 'horse_name',
       'horseid'],
      dtype='object')


In [27]:
race_result_concat = pd.concat([race_result, race_result_comp], axis = 0,  ignore_index=True)

### Things take from race_result_concat
### 1) Number of partcipating horses
### 2) Prize money
From 1) we can also get:

Save the following the feature engineering:
### 3) Normalized finishing position
### 4) Normalized break position
### 5) Normalized final call position

In [28]:
# prize_money

In [29]:
form_record['ind'] = form_record['horseid'] + "_" + form_record['formatted_date'].astype(str)
race_result_concat['ind'] = race_result_concat['horseid'] + "_" + race_result_concat['date'].astype(str)

print('race_result is missing:', len(list(set(form_record['ind'].unique()) - set(race_result_concat['ind'].unique()))))

race_result is missing: 0


In [30]:
form_record = form_record.merge(race_result_concat[['ind', 'prize_money']].drop_duplicates(subset=['ind']), on = 'ind', how = 'left')

In [31]:
# Number of participating horse -> normalize

# Check Na
race_result_concat.loc[race_result_concat['Plc.'].isna()]

,Plc.,Horse No.,Horse,Jockey,Trainer,Actual Wt.,Declar. Horse Wt.,Draw,LBW,RunningPosition,Finish Time,Win Odds,date,match,cdr,prize_money,horse_name,horseid,ind


In [32]:
print(race_result_concat['Plc.'].value_counts())


# Horse that in the same finishing position
race_result_concat['Plc.'] = race_result_concat['Plc.'].str.replace(' DH', '')

race_result_concat = race_result_concat.loc[race_result_concat['Plc.'].str.contains('^[0-9]+$')]
print(race_result_concat['Plc.'].value_counts())

1        8851
2        8830
3        8817
6        8799
4        8784
5        8782
7        8767
8        8711
9        8600
10       8408
11       8063
12       7374
13       3712
14       3112
WV       1838
WV A      356
4 DH      109
WX         91
PU         77
UR         75
3 DH       64
5 DH       58
WX A       56
6 DH       52
7 DH       46
2 DH       42
1 DH       38
9 DH       36
8 DH       36
10 DH      32
FE         27
DNF        23
11 DH      18
TNP        17
DISQ       13
VOID       12
12 DH       4
WXNR        2
Name: Plc., dtype: int64
4     8893
1     8889
3     8881
2     8872
6     8851
5     8840
7     8813
8     8747
9     8636
10    8440
11    8081
12    7378
13    3712
14    3112
Name: Plc., dtype: int64


In [33]:
'''
Why seperate two df (race_result_concat & race_result_concat_runpo)?
Is there any information can be drawn if a horse dropout in the middle of the race? (-> Is dropping invalid finshing position enough?)
How many are these case? (have finishing positive but do not have running position)
'''
# have finishing positive but do not have running position
# -> all are races in 2007 to early 2008 that no running position is recorded
race_result_concat.loc[race_result_concat['RunningPosition'].isna() & race_result_concat['Plc.'].notna()]['date'].value_counts()

20080209    148
20080309    137
20071215    135
Name: date, dtype: int64

In [34]:
race_result_concat[['RunPo_1','RunPo_2','RunPo_3','RunPo_4','RunPo_5','RunPo_6']] = race_result_concat.RunningPosition.str.split(expand=True)

In [35]:
race_result_concat.loc[race_result_concat['RunningPosition'].isna()][['RunPo_1','RunPo_2','RunPo_3','RunPo_4','RunPo_5','RunPo_6']]

,RunPo_1,RunPo_2,RunPo_3,RunPo_4,RunPo_5,RunPo_6
109796,NaN,NaN,NaN,NaN,NaN,NaN
109797,NaN,NaN,NaN,NaN,NaN,NaN
109798,NaN,NaN,NaN,NaN,NaN,NaN
109799,NaN,NaN,NaN,NaN,NaN,NaN
109800,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
110219,NaN,NaN,NaN,NaN,NaN,NaN
110220,NaN,NaN,NaN,NaN,NaN,NaN
110221,NaN,NaN,NaN,NaN,NaN,NaN
110222,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
race_result_concat.groupby(['date', 'match']).size()

date      match
20090301  1         6
          2        12
          3        14
          4        12
          5        12
                   ..
20141214  5        14
          6        13
          7        10
          8        12
          9        14
Length: 8871, dtype: int64

In [ ]:
# break position
'''
copy race_result_concat_runpo from race_result_concat
We seperate race_result_concat_runpo from race_result_concat to avoid missing running position contaminate n_fp calc. 
(matches in 2007-2009 do not have running position but have fp)
'''
print(race_result_concat['Plc.'].value_counts())

# .copy() is new added
race_result_concat_runpo = race_result_concat.copy()
race_result_concat['Place'] = race_result_concat['Plc.']

# Dropping Invalid finishing position
# Normalize based on horses that completed the race
race_result_concat = race_result_concat[race_result_concat['Plc.'] != 'WV']
race_result_concat = race_result_concat[race_result_concat['Plc.'] != 'WV A']
race_result_concat = race_result_concat[race_result_concat['Plc.'] != 'PU']
race_result_concat = race_result_concat[race_result_concat['Plc.'] != 'UR']
race_result_concat = race_result_concat[race_result_concat['Plc.'] != 'WX A']
race_result_concat = race_result_concat[race_result_concat['Plc.'] != 'WX']
race_result_concat = race_result_concat[race_result_concat['Plc.'] != 'FE']
race_result_concat = race_result_concat[race_result_concat['Plc.'] != 'VOID']
race_result_concat = race_result_concat[race_result_concat['Plc.'] != 'DNF']
race_result_concat = race_result_concat[race_result_concat['Plc.'] != 'TNP']
race_result_concat = race_result_concat[race_result_concat['Plc.'] != 'DISQ']
race_result_concat = race_result_concat[race_result_concat['Plc.'] != 'WXNR']

# Horse that in the same finishing position
race_result_concat['Plc.'] = race_result_concat['Plc.'].str.replace(' DH', '')

# Na for 20100530 races 10 and 11
race_result_concat.dropna(subset=['Plc.'], inplace=True)

print(race_result_concat['Plc.'].value_counts())

race_result_concat['Plc.'] = race_result_concat['Plc.'].astype(int)

race_result_concat = race_result_concat.sort_values(by=['horseid', 'date'])


# normalized finishing position from 0 to 1
race_result_concat['n_fp'] = 1 - ( (race_result_concat['Plc.'] - race_result_concat.groupby(['date', 'match'])['Plc.'].transform('min'))
                                     / (race_result_concat.groupby(['date', 'match'])['Plc.'].transform('max') - race_result_concat.groupby(['date', 'match'])['Plc.'].transform('min')) )

# number of horses in the same race
race_result_concat['max_horse'] = race_result_concat.groupby(['date', 'match'])['Plc.'].transform('max')

In [ ]:
# matches in 2007-2009 do not have running position
race_result_concat_runpo[['RunPo_1','RunPo_2','RunPo_3','RunPo_4','RunPo_5','RunPo_6']] = race_result_concat_runpo.RunningPosition.str.split(expand=True)

# if the first running position is na, it is the races that did not record running position
# we just calculate the running position that have recorded and put it back to form record for time-lag features
# since only races in 2007-2009 do not have running position, it still have enough time (A five year buffer) to create time-lag features for our interested period

race_result_concat_runpo.dropna(subset=['RunPo_1'], inplace=True)

race_result_concat_runpo['RunPo_1'] = race_result_concat_runpo['RunPo_1'].astype(int)

# normalization
race_result_concat_runpo['n_break_po'] = 1 - ( (race_result_concat_runpo['RunPo_1'] - race_result_concat_runpo.groupby(['date', 'match'])['RunPo_1'].transform('min'))
                                     / (race_result_concat_runpo.groupby(['date', 'match'])['RunPo_1'].transform('max') - race_result_concat_runpo.groupby(['date', 'match'])['RunPo_1'].transform('min')) )



In [ ]:
# Final Call Position

# get the last running position as final call position
race_result_concat_runpo['final_call_po'] = race_result_concat_runpo[['RunPo_1', 'RunPo_2', 'RunPo_3', 'RunPo_4', 'RunPo_5', 'RunPo_6']].ffill(axis=1).iloc[:, -1]

#Check na
print(race_result_concat_runpo['final_call_po'].isna().value_counts())
# race_result_concat_runpo.dropna(subset=['final_call_po'], inplace=True)

# Check
race_result_concat_runpo[['RunPo_1', 'RunPo_2', 'RunPo_3', 'RunPo_4', 'RunPo_5', 'RunPo_6', 'final_call_po']]

In [ ]:
# casting for calculation
race_result_concat_runpo['final_call_po'] = race_result_concat_runpo['final_call_po'].astype(int)

# normalization
race_result_concat_runpo['n_final_call_po'] = 1 - ( (race_result_concat_runpo['final_call_po'] - race_result_concat_runpo.groupby(['date', 'match'])['final_call_po'].transform('min'))
                                     / (race_result_concat_runpo.groupby(['date', 'match'])['final_call_po'].transform('max') - race_result_concat_runpo.groupby(['date', 'match'])['final_call_po'].transform('min')) )

In [ ]:
# feature get from race result:
# 1. number of participating horses
# 2. prizemoney


# merge those features back to form_record

form_record = form_record.merge(race_result_concat[['ind', 'Place', 'max_horse', 'n_fp', 'Horse']].drop_duplicates(subset=['ind']), on = 'ind', how = 'left')
form_record = form_record.merge(race_result_concat_runpo[['ind', 'n_break_po', 'n_final_call_po', 'RunPo_1']].drop_duplicates(subset=['ind']), on = 'ind', how = 'left')

### Feature engineering on form_record
__Adapting support vector machine methods for horserace odds prediction (Edelman, 2007)__
1. finishing position of previous race
2. odds of previous race
3. prizemoney in previous race
4. final call position of previous race
5. indicator, no call position of previous race
6. distance of previous race
7. odds * prizemoney, previous race
8. weight carried in previous race
9. days since previous race
10. total prizemoney in current race
11. weight carried in current race
12. distance of current race


__Searching for Positive Returns at the Track (Bolton&Chapman, 1986)__
 
13. LIFE_WIN: percentage of races won of those entered in the past two years
14. AVESPRAT: an average speed rating for the last four races of each horse 
> (A speed rating for a horse compares it time in a race with the **track record** for that distance)
15. W_RACE: winnings per race in the current year (A proxy for recent winning potential)
16. LSPEDRAT: a **track-adjusted** speed rating for the previous race
17. JOCK%WIN: the perecentage of jockey winning rides over the current year
18. JOCK#WIN: the number of jockey winning rides over the current year
19. JMISDATA: indicator variable, which takes on the value 1 when the other jockey variables are missing, and is 0 otherwise
20. WEIGHT: (included in race result)
21. POSTPOS: post position (included in race result)
22. NEWDIST: indicator variable, 1 if a horses had run three of four of its last four races at distance levels of less than one miles(25% difference in context), and 0 otherwise

__Identifying winners of competitive events: A SVM-based classi-fication model for horserace prediction (Lessmann, et al., 2009)__
> race-wise normalisation, to provide the first stage  SVM  model  with  some information concerning within-race competition

> these two  transformations complement  each  other  and  capture different aspects of  racing  data

23. ln_q: the natural logarithm of the normalized track probablities
24. pre_s_ra: Speed rating for the previous race in which the horse ran (~ 16)
~25. avgsr4: The average of a horse’s speed rating in its last 4 races; value of zero when there is no past run (= 14)~
26. disavesr: The average speed rating of the past runs of each horse at this distance; value of zero when no previous run
27. go_avesr: The average speed rating of all past runs of the horse on this going; value of zero when no previous run
~28. draw (= 21)~
29. eps: Total prize money earnings (finishing first, second or third) to date/Number of races entered (~ 15)
30. newdis: 1 indicates a horse that ran three or four of its last four races at a distance of 80% less than current distance, and 0 otherwise (~ 22)
~31. Weight carried by the horse in current race (= 20)~
32. win_run: The percentage of the races won by the horse in its career (~ 13)
33. jnowin: The number of wins by the jockey in career to date of race (~ 17)
34. jwinper: The winning percentage of the jockey in career to date of race (~ 18)
35. jst1miss: 1 indicates when the other jockey variables are missing; 0 otherwise (~ 19)

__Parameter Estimation for Ranking Data with Markov Chain Monte Carlo Stochastic Approximation (Huang, 2002)__

~36. LOGODD: log of the odds in this race (= 23)~

~37. WTCARRIED weight carried in this race (= 11)~

33. WTCARRIEDDIST: the WTCARRIED times the distance of this race
34. WTCARRIEDADJ: net adjustment of WTCARRIED in this race
35. AGET
>      = 1     if age = 4 or 5
>      = 0.95  if age = 3 or 6
>      = 0.9   if age = 7
>      = 0.85  if age = 8
>      = 0.8   if age = 2 or 9
>      = 0.75  if otherwise

~36. DRAWING (= 21)~

~37. AVESPRAT (= 14)~

~38. HWINPER: the winning percentage of a horse in the past two years (= 13)~

36. HNOWIN: the winning race number of a horse in the past two years
37. JWINPER: the winning percentage of a jockey in the past two years (~ 34)
38. JNOWIN: the winning race number of a jockey in the past two years (~ 33)
39. LOGDAYSINCE: log of days since a horse entered the last race (~ 9)
40. AGEBYDIST: the original age times the distance
41. DRAWBYTRACK: the DRAWING times the TRACK (1 if Shatin, 2 if Happy Valley)
    
    
__A SVM-based Committee Machine for Prediction of Hong Kong Horse Racin (Chung, et al., 2017)__

42. Class
~43. Distance (= 12)~
~44. Horse's win rate (= 32)~
45. Horse's place rate
46. Horse's show rate
~47. Jockey's win rate (= 33)~
48. Jockey's place rate
49. Jockey's show rate
50. Trainer's win rate
51. Trainer's place rate
52. Trainer's show rate
~53. Actual weight (= 20)~
54. Declared horse weight
~55. Draw (= 21)~
56. Average break position
57. Fastest finish time **(in racecard)**
~58. Win-odds~
~59. place of last races (= 1)~
60. Average place of last 3 races



__STILL SEARCHING FOR POSITIVE RETURNS AT THE TRACK: EMPIRICAL RESULTS FROM 2,000 HONG KONG RACES (Chapman, 2008)__

54. FINISH1: Recency-weighted mean of past normalized finishing position
55. WEIGHT(REL): Today's weight carried minus mean weight carried on all horses in this race.
56. WEIGHT(REL)DIST: WEIGHT(REL) times distance of this race
57. FIRSTCALL: Recency-weighted mean of past first call position (an early speed factor)
58. FIRSTCALLDIST: FIRSTCALL times distance of this race (in combination with FIRSTCALL, this enhances the impact of early speed in short races).
59. WINHISTORY: Recency-weighted mean of past win history (binary coding with 1 for wins and 0 otherwise)
60. LENGTHSBEHIND: Recency-weighted mean of past lengths beaten, distance normalized
~61. DAYSSINCE: Days since last race minus median days between races (= 9)~
62. CAREERSTARTS:  Number of career starts
63. FINISH2: Average past normalized finishing position
64. COMPETITORS1: A complicated current-competitorfactor which serves to boost horses who have actually raced with and beaten other horses in this race in a recent past race
> enhancement
65. COMPETITORS2: Recency-weighted mean of average lengths beaten of all other horses in this horse's last races
> enhancement
66. LASTRACE: Normalized finish of this horse in its last race times the recency weight of thal race
67. SPECIALDIFF: Recency-weighted mean of past officially reported "trip" difficulties this horse has experienced (e.g., "was bumped and lost two lengths in last race")
> enhancement
68. DISTANCEPREF: Preference (positive or negative) for this race's distance
> enhancement
69. TRACKPREF: Preference for this race's track
> enhancement
70. JOCKEYHISTORY: Recency-weighted past skill of the jockey that rode this horse in past races (~ 33,34)
71. JOCKEYCURRENT: This race's jockey advantage factor (based on an **auxiliary regression model**)
> enhancement
72. TRACKWORK: Trackwork factor (based on an auxiliary regression model)
> enhancement
73. STRENGTH: Recency-weighted estimated strength of **other horses** in this horse's past races
> enhancemnt





120 recency

In [ ]:
form_record['Date'] =  pd.to_datetime(form_record['Date'], format='%Y/%m/%d')

# make sure the form record is in order since we are about to create time-lag features
form_record = form_record.sort_values(by=['horseid', 'Date'])

In [ ]:
#1. finishing position of previous race

#1.1 indicator, horses withdrew from the last match

#2. odds of previous race

#3. prizemoney in previous race

#4. final call position of previous race

#5. indicator, no call position of previous race

#6. distance of previous race

#7. odds * prizemoney, previous race

#8. weight carried in previous race

#9. days since previous race

#10. total prizemoney in current race

#11. weight carried in current race

#12. distance of current race





In [ ]:
form_record

In [ ]:
# This cell is for WV indicator

#form_record['n_fp'] = form_record.groupby(['horseid'])['n_fp'].fillna(method='ffill')

form_record['place_last'] = form_record.groupby(['horseid'])['n_fp'].shift(+1, fill_value=0)
form_record['place_secondlast'] = form_record.groupby(['horseid'])['n_fp'].shift(+2, fill_value=0)
form_record['place_thirdlast'] = form_record.groupby(['horseid'])['n_fp'].shift(+3, fill_value=0)
#form_record['place_last'] = form_record.groupby(['horseid'])['place_last'].fillna(method='ffill')

# average place of last 3 races

form_record['place_last_3_check'] = form_record.groupby(['horseid'])['n_fp'].apply(lambda x: x.rolling(3).mean())
form_record['place_last_3'] = form_record.groupby(['horseid'])['place_last_3_check'].shift(+1, fill_value=0)

In [ ]:
# bookmakers' odds of previous race
form_record['odd_pre'] = form_record.groupby(['horseid'])['WinOdds'].shift(+1, fill_value=0)
#form_record["odd_pre"].mask(form_record['odd_pre'] == '  ', 0, inplace=True)
form_record["odd_pre"].mask(form_record['odd_pre'] == '  ', np.nan, inplace=True)
form_record['odd_pre'] = form_record.groupby(['horseid'])['odd_pre'].fillna(method='ffill')
form_record['odd_pre'] = form_record['odd_pre'].astype(float)

# total prizemoney in previous race
form_record['prize_money'] = form_record['prize_money'].str.replace(",", "")
form_record['prize_money'] = form_record['prize_money'].astype(int)
form_record['pm_pre'] = form_record.groupby(['horseid'])['prize_money'].shift(+1, fill_value=0)

# final call position of previous race = last running position of previous race (done)
form_record['n_final_call_po'] = form_record.groupby(['horseid'])['n_final_call_po'].fillna(method='ffill')
form_record['nfcp_pre'] = form_record.groupby(['horseid'])['n_final_call_po'].shift(+1, fill_value=0)

# indicator, no call position of previous race (done)
'''
= WV_ind
'''

# distance of previous race
form_record['Dist_pre'] = form_record.groupby(['horseid'])['Dist'].shift(+1, fill_value=0)

# odd * prizemoney of previous race
form_record["WinOdds"].mask(form_record['WinOdds'] == '  ', 0, inplace=True)
form_record['WinOdds'] = form_record['WinOdds'].astype(float)
form_record['oddpm_pre'] = form_record['pm_pre'] * form_record['odd_pre']


# weight carried in previous race
form_record['WTCARRIED_pre'] = form_record.groupby(['horseid'])['ActWt'].shift(+1, fill_value=0)

# days since previous race

In [ ]:
form_record.Date

In [ ]:
form_record['Date'] =  pd.to_datetime(form_record['Date'], format='%Y/%m/%d')

In [ ]:
form_record.Pla.value_counts(dropna=False)

In [ ]:
form_record = form_record.loc[form_record.Pla.str.contains('^[ 0-9]+$')]

In [ ]:
form_record.Pla.value_counts(dropna=False)

In [ ]:
'''
THIS IS form_record_WV_drop
'''

form_record_WV_drop = form_record
form_record_WV_drop = form_record_WV_drop[form_record_WV_drop['Pla'] != 'WV']
form_record_WV_drop = form_record_WV_drop[form_record_WV_drop['Pla'] != 'WV A']
form_record_WV_drop = form_record_WV_drop[form_record_WV_drop['Pla'] != 'PU']
form_record_WV_drop = form_record_WV_drop[form_record_WV_drop['Pla'] != 'UR']
form_record_WV_drop = form_record_WV_drop[form_record_WV_drop['Pla'] != 'WX A']
form_record_WV_drop = form_record_WV_drop[form_record_WV_drop['Pla'] != 'WX']
form_record_WV_drop = form_record_WV_drop[form_record_WV_drop['Pla'] != 'FE']
form_record_WV_drop = form_record_WV_drop[form_record_WV_drop['Pla'] != 'VOID']
form_record_WV_drop = form_record_WV_drop[form_record_WV_drop['Pla'] != 'DNF']
form_record_WV_drop = form_record_WV_drop[form_record_WV_drop['Pla'] != 'TNP']
form_record_WV_drop = form_record_WV_drop[form_record_WV_drop['Pla'] != 'DISQ']
form_record_WV_drop = form_record_WV_drop[form_record_WV_drop['Pla'] != 'WXNR']
form_record_WV_drop.Pla.value_counts(dropna=False)

In [ ]:
# make sure the form record is in order since we are about to create time-lag features

form_record_WV_drop = form_record_WV_drop.sort_values(by=['horseid', 'formatted_date'])

In [ ]:
import numpy as np

# Horse

# Number of races
form_record_WV_drop['careerstart'] = form_record_WV_drop.groupby('horseid').cumcount()+1

'''
unnecessary
'''
form_record_WV_drop['win1'] = np.where(form_record_WV_drop.Pla == '1', 1, 0)
form_record_WV_drop['win2'] = np.where(form_record_WV_drop.Pla == '2', 1, 0)
form_record_WV_drop['win3'] = np.where(form_record_WV_drop.Pla == '3', 1, 0)

# horse win rate
form_record_WV_drop['win'] = form_record_WV_drop.groupby(['horseid'])['win1'].apply(lambda x: x.cumsum())
form_record_WV_drop['win_rate_check'] = (form_record_WV_drop['win']/form_record_WV_drop['careerstart'])

form_record_WV_drop['win_rate'] = form_record_WV_drop.groupby(['horseid'])['win_rate_check'].shift(+1, fill_value=0)


# horse place rate
form_record_WV_drop['place'] = form_record_WV_drop.groupby(['horseid'])['win2'].apply(lambda x: x.cumsum())
form_record_WV_drop['place_rate_check'] = (form_record_WV_drop['place']/form_record_WV_drop['careerstart'])

form_record_WV_drop['place_rate'] = form_record_WV_drop.groupby(['horseid'])['place_rate_check'].shift(+1, fill_value=0)


# horse show rate
form_record_WV_drop['show'] = form_record_WV_drop.groupby(['horseid'])['win3'].apply(lambda x: x.cumsum())
form_record_WV_drop['show_rate_check'] = (form_record_WV_drop['show']/form_record_WV_drop['careerstart'])

form_record_WV_drop['show_rate'] = form_record_WV_drop.groupby(['horseid'])['show_rate_check'].shift(+1, fill_value=0)

form_record_WV_drop['HNOWIN'] = form_record_WV_drop.groupby(['horseid'])['win'].shift(+1, fill_value=0)
form_record_WV_drop['HNOWIN'] = form_record_WV_drop.groupby(['horseid'])['HNOWIN'].fillna(method='ffill')

form_record_WV_drop['HNOPLACE'] = form_record_WV_drop.groupby(['horseid'])['place'].shift(+1, fill_value=0)
form_record_WV_drop['HNOPLACE'] = form_record_WV_drop.groupby(['horseid'])['HNOPLACE'].fillna(method='ffill')

form_record_WV_drop['HNOSHOW'] = form_record_WV_drop.groupby(['horseid'])['show'].shift(+1, fill_value=0)
form_record_WV_drop['HNOSHOW'] = form_record_WV_drop.groupby(['horseid'])['HNOSHOW'].fillna(method='ffill')

In [ ]:
form_record = form_record.merge(form_record_WV_drop[['ind', 'careerstart', 'win1', 'win2', 'win3', 'win', 'win_rate_check', 'win_rate',
                                                     'place', 'place_rate_check', 'place_rate', 'show', 'show_rate_check', 'show_rate',
                                                    'HNOWIN', 'HNOPLACE', 'HNOSHOW']], on = 'ind', how = 'left')

In [ ]:
form_record.columns

In [ ]:
# Jockey 

'''
count is built on form_record_WV_drop
'''

form_record_WV_drop = form_record_WV_drop.sort_values(by=['Jockey', 'formatted_date', 'RaceIndex'])

form_record_WV_drop['occur'] = 1

# count: 以日為單位 -> all shift happen here
count = form_record_WV_drop.groupby(['Jockey', 'formatted_date']).sum().groupby(level=0).cumsum().reset_index()
count.rename(columns={'win1': "j_win", 'win2': "j_place", 'win3': "j_show", 'occur': "J_started"}, inplace=True)

count['Date'] =  pd.to_datetime(count['formatted_date'], format='%Y%m%d')

def rate_recency(frame):
    df = frame[['Jockey', 'Date', 'j_win_rate','j_place_rate', 'j_show_rate']].reset_index(drop=True)
    df = df.sort_values(by=['Date'])
    
    # j_win_rate, j_place_rate, j_show_rate shifted already
    df['j_win_rate_1yr'] = df.groupby(['Jockey'])['j_win_rate'].transform(lambda x: x.ewm(halflife='365 days', times=df['Date']).mean())
    df['j_place_rate_1yr'] = df.groupby(['Jockey'])['j_place_rate'].transform(lambda x: x.ewm(halflife='365 days', times=df['Date']).mean())
    df['j_show_rate_1yr'] = df.groupby(['Jockey'])['j_show_rate'].transform(lambda x: x.ewm(halflife='365 days', times=df['Date']).mean())
    df = df.drop(['j_win_rate','j_place_rate', 'j_show_rate'], axis = 1)
    frame = frame.merge(df, on = ['Jockey', 'Date'])
    return frame

# jockey win rate
count['j_win_rate_check'] = (count['j_win']/count['J_started'])
count['j_win_rate'] = count.groupby(['Jockey'])['j_win_rate_check'].shift(+1, fill_value=0)

# no. of win
count['JNOWIN'] = count.groupby(['Jockey'])['j_win'].shift(+1, fill_value=0)

# jockey place rate
count['j_place_rate_check'] = (count['j_place']/count['J_started'])
count['j_place_rate'] = count.groupby(['Jockey'])['j_place_rate_check'].shift(+1, fill_value=0)

# no. of place
count['JNOPLACE'] = count.groupby(['Jockey'])['j_place'].shift(+1, fill_value=0)

# jockey show rate
count['j_show_rate_check'] = (count['j_show']/count['J_started'])
count['j_show_rate'] = count.groupby(['Jockey'])['j_show_rate_check'].shift(+1, fill_value=0)

# no. of show
count['JNOSHOW'] = count.groupby(['Jockey'])['j_show'].shift(+1, fill_value=0)

# 1yr recency
count = count.groupby('Jockey').apply(rate_recency)
count.reset_index(inplace=True, drop=True)

print(form_record.shape)

form_record = form_record.merge(count[['Jockey', 'formatted_date', 'J_started', 'j_win', 'j_place', 'j_show', 'j_win_rate_check', 'j_win_rate', 
                                       'j_place_rate_check', 'j_place_rate', 'j_show_rate_check', 'j_show_rate', 'JNOWIN', 'JNOPLACE', 'JNOSHOW', 'j_win_rate_1yr', 'j_place_rate_1yr', 'j_show_rate_1yr']], on = ['Jockey', 'formatted_date'], how='left')


print(form_record.shape)

In [ ]:
# Trainer 

'''
t_count is built on form_record_WV_drop
'''
form_record_WV_drop = form_record_WV_drop.sort_values(by=['Trainer', 'formatted_date', 'RaceIndex'])

form_record_WV_drop['occur'] = 1

# count: 以日為單位 -> all shift happen here
t_count = form_record_WV_drop.groupby(['Trainer', 'formatted_date']).sum().groupby(level=0).cumsum().reset_index()
t_count.rename(columns={'win1': "t_win", 'win2': "t_place", 'win3': "t_show", 'occur': "T_started"}, inplace=True)

t_count['Date'] =  pd.to_datetime(t_count['formatted_date'], format='%Y%m%d')

def rate_recency(frame):
    df = frame[['Trainer', 'Date', 't_win_rate','t_place_rate', 't_show_rate']].reset_index(drop=True)
    df = df.sort_values(by=['Date'])
    
    df['t_win_rate_1yr'] = df.groupby(['Trainer'])['t_win_rate'].transform(lambda x: x.ewm(halflife='365 days', times=df['Date']).mean())
    df['t_place_rate_1yr'] = df.groupby(['Trainer'])['t_place_rate'].transform(lambda x: x.ewm(halflife='365 days', times=df['Date']).mean())
    df['t_show_rate_1yr'] = df.groupby(['Trainer'])['t_show_rate'].transform(lambda x: x.ewm(halflife='365 days', times=df['Date']).mean())
    df = df.drop(['t_win_rate','t_place_rate', 't_show_rate'], axis = 1)
    frame = frame.merge(df, on = ['Trainer', 'Date'])
    return frame

# trainer win rate
t_count['t_win_rate_check'] = (t_count['t_win']/t_count['T_started'])
t_count['t_win_rate'] = t_count.groupby(['Trainer'])['t_win_rate_check'].shift(+1, fill_value=0)

# no. of win
t_count['TNOWIN'] = t_count.groupby(['Trainer'])['t_win'].shift(+1, fill_value=0)

# trainer place rate
t_count['t_place_rate_check'] = (t_count['t_place']/t_count['T_started'])
t_count['t_place_rate'] = t_count.groupby(['Trainer'])['t_place_rate_check'].shift(+1, fill_value=0)

# no. of place
t_count['TNOPLACE'] = t_count.groupby(['Trainer'])['t_place'].shift(+1, fill_value=0)

# trainer show rate
t_count['t_show_rate_check'] = (t_count['t_show']/t_count['T_started'])
t_count['t_show_rate'] = t_count.groupby(['Trainer'])['t_show_rate_check'].shift(+1, fill_value=0)

# no. of show
t_count['TNOSHOW'] = t_count.groupby(['Trainer'])['t_show'].shift(+1, fill_value=0)

# 1yr recency
t_count = t_count.groupby('Trainer').apply(rate_recency)
t_count.reset_index(inplace=True, drop=True)

print(form_record.shape)

form_record = form_record.merge(t_count[['Trainer', 'formatted_date', 'T_started', 't_win', 't_place', 't_show', 't_win_rate_check', 't_win_rate', 
                                       't_place_rate_check', 't_place_rate', 't_show_rate_check', 't_show_rate', 'TNOWIN', 'TNOPLACE', 'TNOSHOW', 't_win_rate_1yr', 't_place_rate_1yr', 't_show_rate_1yr']], on = ['Trainer', 'formatted_date'], how='left')

print(form_record.shape)

In [ ]:
#form_record.to_csv (r'C:\Users\chunl\Desktop\form_record.csv', index = False, header=True)

In [ ]:
form_record.columns

In [ ]:
#form_record.to_csv (r'C:\Users\chunl\Desktop\form_record.csv', index = False, header=True)
form_record = form_record[['RaceIndex', 'Pla', 'Date', 'RC/Track/Course', 'Dist', 'Ground',
                           'RaceClass', 'Draw', 'Rating', 'Trainer', 'Jockey', 'LBW', 'WinOdds',
                           'ActWt', 'RunPo', 'FinishTime', 'Declare_Horse_Wt', 'horseid', 'formatted_date', 'ind', 'prize_money', 'Place',
                           'max_horse', 'n_fp', 'Horse', 'n_break_po', 'n_final_call_po', 'RunPo_1', 
                           
                           'careerstart',
                           
                           'win_rate', 'place_rate', 'show_rate', 'HNOWIN', 'HNOPLACE', 'HNOSHOW', 
                           'j_win_rate', 'j_place_rate', 'j_show_rate', 'JNOWIN', 'JNOPLACE', 'JNOSHOW', 
                           'j_win_rate_1yr', 'j_place_rate_1yr', 'j_show_rate_1yr', 
                           't_win_rate', 't_place_rate', 't_show_rate', 'TNOWIN', 'TNOPLACE', 'TNOSHOW', 
                           't_win_rate_1yr', 't_place_rate_1yr', 't_show_rate_1yr']]

### Average break position
### place of last races
### Average place of last 3 races

In [ ]:
form_record_runpo = form_record.dropna(subset=['RunPo_1'])
form_record_runpo = form_record_runpo.sort_values(by=['horseid', 'formatted_date'])

# div is bulit on form_record_runpo
form_record_runpo['div'] = form_record_runpo.groupby('horseid').cumcount()+1

'''
div is for position (form_record_runpo)
occur is for rate (form_record_WV_drop)
有名次 =/= 有 running position
'''
form_record = form_record.merge(form_record_runpo[['ind', 'div']], on = ['ind'], how = 'left')

In [ ]:
# average break position

In [ ]:
form_record = form_record.sort_values(by=['horseid', 'formatted_date'])

# We use form_record_WV_drop to calculate the div as div is for avg_break_po

form_record['sum_break_po'] = form_record.groupby(['horseid'])['n_break_po'].apply(lambda x: x.cumsum())

form_record['avg_break_po_check'] = (form_record['sum_break_po']/form_record['div'])
form_record['avg_break_po_check'] = form_record.groupby(['horseid'])['avg_break_po_check'].fillna(method='ffill')

form_record['avg_break_po'] = form_record.groupby(['horseid'])['avg_break_po_check'].shift(+1, fill_value=0)

In [ ]:
def avg_break_po_recency(frame):
    df = frame[['horseid', 'Date', 'n_break_po']].reset_index(drop=True)
    df = df.sort_values(by=['Date'])
    
    df['avg_break_po_120'] = df.groupby(['horseid'])['n_break_po'].transform(lambda x: x.ewm(halflife='120 days', times=df['Date']).mean())
    df = df.drop(['n_break_po'], axis = 1)
    frame = frame.merge(df, on = ['horseid', 'Date'])
    return frame

form_record = form_record.groupby('horseid').apply(avg_break_po_recency)
form_record.reset_index(inplace=True, drop=True)
form_record['avg_break_po_120'] = form_record.groupby(['horseid'])['avg_break_po_120'].shift(+1, fill_value=0)

In [ ]:
# place of last races
# Average place of last 3 races

In [ ]:
# This cell is for WV indicator

#form_record['n_fp'] = form_record.groupby(['horseid'])['n_fp'].fillna(method='ffill')

form_record['place_last'] = form_record.groupby(['horseid'])['n_fp'].shift(+1, fill_value=0)
form_record['place_secondlast'] = form_record.groupby(['horseid'])['n_fp'].shift(+2, fill_value=0)
form_record['place_thirdlast'] = form_record.groupby(['horseid'])['n_fp'].shift(+3, fill_value=0)
#form_record['place_last'] = form_record.groupby(['horseid'])['place_last'].fillna(method='ffill')

# average place of last 3 races

form_record['place_last_3_check'] = form_record.groupby(['horseid'])['n_fp'].apply(lambda x: x.rolling(3).mean())
form_record['place_last_3'] = form_record.groupby(['horseid'])['place_last_3_check'].shift(+1, fill_value=0)

In [ ]:
# WV ind

form_record['WV_ind_check'] = np.where( ((form_record['Pla'] == '1') | (form_record['Pla'] == '2') | (form_record['Pla'] == '3') | (form_record['Pla'] == '4') | (form_record['Pla'] == '5') | (form_record['Pla'] == '6')
                                | (form_record['Pla'] == '7') | (form_record['Pla'] == '8') | (form_record['Pla'] == '9') | (form_record['Pla'] == '10') | (form_record['Pla'] == '11') | (form_record['Pla'] == '12')
                                | (form_record['Pla'] == '13') | (form_record['Pla'] == '14')), 0, 1)

form_record['WV_ind'] = form_record.groupby(['horseid'])['WV_ind_check'].shift(+1, fill_value=0)
form_record['WVsecond_ind'] = form_record.groupby(['horseid'])['WV_ind_check'].shift(+2, fill_value=0)
form_record['WVthird_ind'] = form_record.groupby(['horseid'])['WV_ind_check'].shift(+3, fill_value=0)

# WV 3 ind

form_record['WV_3_ind'] = np.where(form_record['place_last_3'].isna(), 1, 0)

In [ ]:
# place of last races

form_record['n_fp'] = form_record.groupby(['horseid'])['n_fp'].fillna(method='ffill')

form_record['place_last'] = form_record.groupby(['horseid'])['n_fp'].shift(+1, fill_value=0)
form_record['place_last'] = form_record.groupby(['horseid'])['place_last'].fillna(method='ffill')

# average place of last 3 races

form_record['place_last_3_check'] = form_record.groupby(['horseid'])['n_fp'].apply(lambda x: x.rolling(3, min_periods=1).mean())
form_record['place_last_3'] = form_record.groupby(['horseid'])['place_last_3_check'].shift(+1, fill_value=0)

In [ ]:
# second last and third last
form_record['place_secondlast'] = form_record.groupby(['horseid'])['n_fp'].shift(+2, fill_value=0)
form_record['place_secondlast'] = form_record.groupby(['horseid'])['place_secondlast'].fillna(method='ffill')

form_record['place_thirdlast'] = form_record.groupby(['horseid'])['n_fp'].shift(+3, fill_value=0)
form_record['place_thirdlast'] = form_record.groupby(['horseid'])['place_thirdlast'].fillna(method='ffill')

In [ ]:
form_record.columns

In [ ]:
### Average break position

form_record[['Date', 'horseid', 'RunPo', 'max_horse', 'div', 'n_break_po', 'sum_break_po', 'avg_break_po_check', 'avg_break_po',
       'avg_break_po_120']].to_csv (r'C:\Users\chunl\Desktop\form_record.csv', index = False, header=True)

In [ ]:
### place of last races, second last and third last
### Average place of last 3 races

form_record[['Date', 'horseid', 'Pla', 'max_horse', 'n_fp', 'place_last', 'place_secondlast', 'place_thirdlast', 'WV_ind_check', 'WV_ind', 'WVsecond_ind', 'WVthird_ind', 'place_last_3_check', 'place_last_3',
       'WV_3_ind']].to_csv (r'C:\Users\chunl\Desktop\form_record.csv', index = False, header=True)

In [ ]:
form_record = form_record[['RaceIndex', 'Pla', 'Date', 'RC/Track/Course', 'Dist', 'Ground',
                           'RaceClass', 'Draw', 'Rating', 'Trainer', 'Jockey', 'LBW', 'WinOdds',
                           'ActWt', 'RunPo', 'FinishTime', 'Declare_Horse_Wt', 'horseid', 'formatted_date', 'ind', 'prize_money', 'Place',
                           'max_horse', 'n_fp', 'Horse', 'n_break_po', 'n_final_call_po', 'RunPo_1', 'careerstart',
                           
                           'win_rate', 'place_rate', 'show_rate', 'HNOWIN', 'HNOPLACE', 'HNOSHOW', 
                           'j_win_rate', 'j_place_rate', 'j_show_rate', 'JNOWIN', 'JNOPLACE', 'JNOSHOW', 
                           'j_win_rate_1yr', 'j_place_rate_1yr', 'j_show_rate_1yr', 
                           't_win_rate', 't_place_rate', 't_show_rate', 'TNOWIN', 'TNOPLACE', 'TNOSHOW', 
                           't_win_rate_1yr', 't_place_rate_1yr', 't_show_rate_1yr',
                           
                           'avg_break_po', 'avg_break_po_120', 'place_last', 'place_secondlast', 'place_thirdlast', 'place_last_3', 'WV_ind', 'WVsecond_ind', 'WVthird_ind', 'WV_3_ind']]

In [ ]:
# TW done

In [ ]:
'''
next paper here
'''

### AVESPRAT
### LOGDAYSINCE

In [ ]:
# debut
# form_record_WV_drop

form_record_WV_drop['debut'] = 0
form_record_WV_drop = form_record_WV_drop.sort_values(by= ['formatted_date'])
idx = form_record_WV_drop.groupby('horseid').head(1).index
form_record_WV_drop.loc[idx,'debut'] = 1

# DAYSINCE
# use form_record_WV_drop
form_record_WV_drop['DAYSINCE'] = form_record_WV_drop.groupby(['horseid'])['Date'].diff().dt.days.fillna(0)
form_record_WV_drop['LOGDAYSINCE'] = np.log(form_record_WV_drop.DAYSINCE)
form_record_WV_drop['LOGDAYSINCE'].replace(-np.inf, 0, inplace=True)

form_record = form_record.merge(form_record_WV_drop[['ind', 'debut', 'DAYSINCE', 'LOGDAYSINCE']], on = 'ind', how = 'left')
form_record.columns

In [ ]:
form_record[['Date', 'horseid', 'Pla', 'debut', 'DAYSINCE', 'LOGDAYSINCE']].to_csv (r'C:\Users\chunl\Desktop\form_record.csv', index = False, header=True)

In [ ]:
# AVESPRAT

In [ ]:
form_record.RaceClass.value_counts()

In [ ]:
form_record['class'] = form_record['RaceClass']
form_record.replace({'class' : { 'G1' : 'G', 'G3' : 'G', 'HKG3' : 'G', 'G2' : 'G', 'HKG1' : 'G', 'HKG2' : 'G' }}, inplace=True)
form_record.replace({'class' : { '4R' : '4', '4S' : '4', '4B' : '4'}}, inplace=True)
form_record.replace({'class' : { '3S' : '3', '3B' : '3'}}, inplace=True)

#ST / Turf / "C+3"
form_record[['RC', 'Track', 'Course']] = form_record['RC/Track/Course'].str.split(" / ", expand=True)

In [ ]:
form_record['class'].value_counts()

In [ ]:
#form_record.loc[form_record['RaceClass'] == '3B'][['Date', 'RC/Track/Course', 'Dist', 'Ground', 'RaceClass', 'cdr', 'RC/Track/Course', 'FinishTime', 'Finish Time']]

In [ ]:
def track_record(course_record):
    m, s = course_record.split(".", 1)
    cr = int(m) * 60 + float(s)
    return cr

In [ ]:
ST_turf_cr_1000_G = '0.55.16'
ST_turf_cr_1000_1 = '0.54.70'
ST_turf_cr_1000_2 = '0.54.69'
ST_turf_cr_1000_3 = '0.54.89'
ST_turf_cr_1000_4 = '0.55.07'
ST_turf_cr_1000_5 = '0.55.64'
ST_turf_cr_1000_Griffin = '0.56.00'

ST_turf_cr_1200_G = '1.07.50'
ST_turf_cr_1200_1 = '1.07.80'
ST_turf_cr_1200_2 = '1.07.89'
ST_turf_cr_1200_3 = '1.08.31'
ST_turf_cr_1200_4 = '1.08.45'
ST_turf_cr_1200_5 = '1.09.24'
ST_turf_cr_1200_Griffin = '1.08.80'

ST_turf_cr_1400_G = '1.20.05'
ST_turf_cr_1400_1 = '1.19.92'
ST_turf_cr_1400_2 = '1.20.29'
ST_turf_cr_1400_3 = '1.20.62'
ST_turf_cr_1400_4 = '1.20.80'
ST_turf_cr_1400_5 = '1.21.53'
ST_turf_cr_1400_Griffin = '1.22.30'

ST_turf_cr_1600_G = '1.32.64'
ST_turf_cr_1600_1 = '1.32.82'
ST_turf_cr_1600_2 = '1.32.96'
ST_turf_cr_1600_3 = '1.33.27'
ST_turf_cr_1600_4 = '1.33.78'
ST_turf_cr_1600_5 = '1.34.38'
ST_turf_cr_1600_Griffin = '1.35.80'

ST_turf_cr_1800_G = '1.45.25'
ST_turf_cr_1800_1 = '1.46.60'
ST_turf_cr_1800_2 = '1.45.88'
ST_turf_cr_1800_3 = '1.46.18'
ST_turf_cr_1800_4 = '1.46.54'
ST_turf_cr_1800_5 = '1.47.60'

ST_turf_cr_2000_G = '1.59.30'
ST_turf_cr_2000_1 = '1.59.66'
ST_turf_cr_2000_2 = '1.59.53'
ST_turf_cr_2000_3 = '2.00.37'
ST_turf_cr_2000_4 = '2.01.62'
ST_turf_cr_2000_5 = '2.01.56'

ST_turf_cr_2200_1 = '2.14.60'
ST_turf_cr_2200_2 = '2.12.42'
ST_turf_cr_2200_3 = '2.16.06'
ST_turf_cr_2200_4 = '2.13.69'
ST_turf_cr_2200_5 = '2.15.42'

ST_turf_cr_2400_G = '2.24.60'
ST_turf_cr_2400_1 = '2.26.10'
ST_turf_cr_2400_2 = '2.29.70'

ST_turf_cr_2400_4 = '2.31.00'

In [ ]:
HV_turf_cr_1000_1 = '0.56.30'
HV_turf_cr_1000_2 = '0.56.25'
HV_turf_cr_1000_3 = '0.56.11'
HV_turf_cr_1000_4 = '0.56.39'
HV_turf_cr_1000_5 = '0.56.90'
HV_turf_cr_1000_Griffin = '0.58.30'

HV_turf_cr_1200_G = '1.09.30'
HV_turf_cr_1200_1 = '1.08.70'
HV_turf_cr_1200_2 = '1.08.52'
HV_turf_cr_1200_3 = '1.08.68'
HV_turf_cr_1200_4 = '1.08.92'
HV_turf_cr_1200_5 = '1.09.73'
HV_turf_cr_1200_Griffin = '1.10.50'

HV_turf_cr_1650_1 = '1.38.70'
HV_turf_cr_1650_2 = '1.38.58'
HV_turf_cr_1650_3 = '1.38.61'
HV_turf_cr_1650_4 = '1.38.83'
HV_turf_cr_1650_5 = '1.39.11'

HV_turf_cr_1800_G = '1.48.10'
HV_turf_cr_1800_1 = '1.47.80'
HV_turf_cr_1800_2 = '1.47.61'
HV_turf_cr_1800_3 = '1.48.30'
HV_turf_cr_1800_4 = '1.48.10'
HV_turf_cr_1800_5 = '1.49.33'

HV_turf_cr_2200_1 = '2.15.44'
HV_turf_cr_2200_2 = '2.15.90'
HV_turf_cr_2200_3 = '2.14.88'
HV_turf_cr_2200_4 = '2.16.06'
HV_turf_cr_2200_5 = '2.16.06'

HV_turf_cr_2400_2 = '2.30.40'
HV_turf_cr_2400_3 = '2.31.20'
HV_turf_cr_2400_4 = '2.31.00'
HV_turf_cr_2400_5 = '2.31.00'

In [ ]:
ST_allw_cr_1200_1 = '1.07.45'
ST_allw_cr_1200_2 = '1.07.27'
ST_allw_cr_1200_3 = '1.07.40'
ST_allw_cr_1200_4 = '1.08.14'
ST_allw_cr_1200_5 = '1.08.46'

ST_allw_cr_1650_1 = '1.36.34'
ST_allw_cr_1650_2 = '1.36.75'
ST_allw_cr_1650_3 = '1.37.22'
ST_allw_cr_1650_4 = '1.38.02'
ST_allw_cr_1650_5 = '1.38.60'

ST_allw_cr_1800_1 = '1.49.40'
ST_allw_cr_1800_2 = '1.47.67'
ST_allw_cr_1800_3 = '1.46.68'
ST_allw_cr_1800_4 = '1.47.54'
ST_allw_cr_1800_5 = '1.47.67'

ST_allw_cr_2000_3 = '2.01.30'
ST_allw_cr_2000_4 = '2.01.80'

ST_allw_cr_2400_2 = '2.31.80'
ST_allw_cr_2400_3 = '2.30.00'
ST_allw_cr_2400_4 = '2.29.10'
ST_allw_cr_2400_5 = '2.31.50'

In [ ]:
# Create a df for track_record

data = [['ST', 'Turf', 1000, 'G'], ['ST', 'Turf', 1000, '1'], ['ST', 'Turf', 1000, '2'], ['ST', 'Turf', 1000, '3'], ['ST', 'Turf', 1000, '4'], ['ST', 'Turf', 1000, '5'], ['ST', 'Turf', 1000, 'GRIFFIN'],
        ['ST', 'Turf', 1200, 'G'], ['ST', 'Turf', 1200, '1'], ['ST', 'Turf', 1200, '2'], ['ST', 'Turf', 1200, '3'], ['ST', 'Turf', 1200, '4'], ['ST', 'Turf', 1200, '5'], ['ST', 'Turf', 1200, 'GRIFFIN'],
        ['ST', 'Turf', 1400, 'G'], ['ST', 'Turf', 1400, '1'], ['ST', 'Turf', 1400, '2'], ['ST', 'Turf', 1400, '3'], ['ST', 'Turf', 1400, '4'], ['ST', 'Turf', 1400, '5'], ['ST', 'Turf', 1400, 'GRIFFIN'],
        ['ST', 'Turf', 1600, 'G'], ['ST', 'Turf', 1600, '1'], ['ST', 'Turf', 1600, '2'], ['ST', 'Turf', 1600, '3'], ['ST', 'Turf', 1600, '4'], ['ST', 'Turf', 1600, '5'], ['ST', 'Turf', 1600, 'GRIFFIN'],
        ['ST', 'Turf', 1800, 'G'], ['ST', 'Turf', 1800, '1'], ['ST', 'Turf', 1800, '2'], ['ST', 'Turf', 1800, '3'], ['ST', 'Turf', 1800, '4'], ['ST', 'Turf', 1800, '5'],
        ['ST', 'Turf', 2000, 'G'], ['ST', 'Turf', 2000, '1'], ['ST', 'Turf', 2000, '2'], ['ST', 'Turf', 2000, '3'], ['ST', 'Turf', 2000, '4'], ['ST', 'Turf', 2000, '5'],
        ['ST', 'Turf', 2200, '1'], ['ST', 'Turf', 2200, '2'], ['ST', 'Turf', 2200, '3'], ['ST', 'Turf', 2200, '4'], ['ST', 'Turf', 2200, '5'],
        ['ST', 'Turf', 2400, 'G'], ['ST', 'Turf', 2400, '1'], ['ST', 'Turf', 2400, '2'], ['ST', 'Turf', 2400, '4'],
        
        ['HV', 'Turf', 1000, '1'], ['HV', 'Turf', 1000, '2'], ['HV', 'Turf', 1000, '3'], ['HV', 'Turf', 1000, '4'], ['HV', 'Turf', 1000, '5'], ['HV', 'Turf', 1000, 'GRIFFIN'],
        ['HV', 'Turf', 1200, 'G'], ['HV', 'Turf', 1200, '1'], ['HV', 'Turf', 1200, '2'], ['HV', 'Turf', 1200, '3'], ['HV', 'Turf', 1200, '4'], ['HV', 'Turf', 1200, '5'], ['HV', 'Turf', 1200, 'GRIFFIN'],
        ['HV', 'Turf', 1650, '1'], ['HV', 'Turf', 1650, '2'], ['HV', 'Turf', 1650, '3'], ['HV', 'Turf', 1650, '4'], ['HV', 'Turf', 1650, '5'],
        ['HV', 'Turf', 1800, 'G'], ['HV', 'Turf', 1800, '1'], ['HV', 'Turf', 1800, '2'], ['HV', 'Turf', 1800, '3'], ['HV', 'Turf', 1800, '4'], ['HV', 'Turf', 1800, '5'],
        ['HV', 'Turf', 2200, '1'], ['HV', 'Turf', 2200, '2'], ['HV', 'Turf', 2200, '3'], ['HV', 'Turf', 2200, '4'], ['HV', 'Turf', 2200, '5'],
        ['HV', 'Turf', 2400, '2'], ['HV', 'Turf', 2400, '3'], ['HV', 'Turf', 2400, '4'], ['HV', 'Turf', 2400, '5'],
        
        ['ST', 'AWT', 1200, '1'], ['ST', 'AWT', 1200, '2'], ['ST', 'AWT', 1200, '3'], ['ST', 'AWT', 1200, '4'], ['ST', 'AWT', 1200, '5'],
        ['ST', 'AWT', 1650, '1'], ['ST', 'AWT', 1650, '2'], ['ST', 'AWT', 1650, '3'], ['ST', 'AWT', 1650, '4'], ['ST', 'AWT', 1650, '5'],
        ['ST', 'AWT', 1800, '1'], ['ST', 'AWT', 1800, '2'], ['ST', 'AWT', 1800, '3'], ['ST', 'AWT', 1800, '4'], ['ST', 'AWT', 1800, '5'],
                                                            ['ST', 'AWT', 2000, '3'], ['ST', 'AWT', 2000, '4'],
                                  ['ST', 'AWT', 2400, '2'], ['ST', 'AWT', 2400, '3'], ['ST', 'AWT', 2400, '4'], ['ST', 'AWT', 2400, '5']] 
  
# Create the pandas DataFrame 
record = pd.DataFrame(data, columns = ['RC', 'Track', 'Dist', 'class']) 

record['recordtime'] = [ST_turf_cr_1000_G, ST_turf_cr_1000_1, ST_turf_cr_1000_2 ,ST_turf_cr_1000_3, ST_turf_cr_1000_4, ST_turf_cr_1000_5, ST_turf_cr_1000_Griffin
                        , ST_turf_cr_1200_G, ST_turf_cr_1200_1, ST_turf_cr_1200_2, ST_turf_cr_1200_3, ST_turf_cr_1200_4, ST_turf_cr_1200_5, ST_turf_cr_1200_Griffin
                        , ST_turf_cr_1400_G, ST_turf_cr_1400_1, ST_turf_cr_1400_2, ST_turf_cr_1400_3, ST_turf_cr_1400_4, ST_turf_cr_1400_5, ST_turf_cr_1400_Griffin
                        , ST_turf_cr_1600_G, ST_turf_cr_1600_1, ST_turf_cr_1600_2, ST_turf_cr_1600_3, ST_turf_cr_1600_4, ST_turf_cr_1600_5, ST_turf_cr_1600_Griffin
                        , ST_turf_cr_1800_G, ST_turf_cr_1800_1, ST_turf_cr_1800_2, ST_turf_cr_1800_3, ST_turf_cr_1800_4, ST_turf_cr_1800_5
                        , ST_turf_cr_2000_G, ST_turf_cr_2000_1, ST_turf_cr_2000_2, ST_turf_cr_2000_3, ST_turf_cr_2000_4, ST_turf_cr_2000_5
                        , ST_turf_cr_2200_1, ST_turf_cr_2200_2, ST_turf_cr_2200_3, ST_turf_cr_2200_4, ST_turf_cr_2200_5
                        , ST_turf_cr_2400_G, ST_turf_cr_2400_1, ST_turf_cr_2400_2, ST_turf_cr_2400_4
                        
                        , HV_turf_cr_1000_1, HV_turf_cr_1000_2, HV_turf_cr_1000_3, HV_turf_cr_1000_4, HV_turf_cr_1000_5, HV_turf_cr_1000_Griffin
                        , HV_turf_cr_1200_G, HV_turf_cr_1200_1, HV_turf_cr_1200_2, HV_turf_cr_1200_3, HV_turf_cr_1200_4, HV_turf_cr_1200_5, HV_turf_cr_1200_Griffin
                        , HV_turf_cr_1650_1, HV_turf_cr_1650_2, HV_turf_cr_1650_3, HV_turf_cr_1650_4, HV_turf_cr_1650_5
                        , HV_turf_cr_1800_G, HV_turf_cr_1800_1, HV_turf_cr_1800_2, HV_turf_cr_1800_3, HV_turf_cr_1800_4, HV_turf_cr_1800_5
                        , HV_turf_cr_2200_1, HV_turf_cr_2200_2, HV_turf_cr_2200_3, HV_turf_cr_2200_4, HV_turf_cr_2200_5
                        , HV_turf_cr_2400_2, HV_turf_cr_2400_3, HV_turf_cr_2400_4, HV_turf_cr_2400_5
                        
                        , ST_allw_cr_1200_1, ST_allw_cr_1200_2, ST_allw_cr_1200_3, ST_allw_cr_1200_4, ST_allw_cr_1200_5
                        , ST_allw_cr_1650_1, ST_allw_cr_1650_2, ST_allw_cr_1650_3, ST_allw_cr_1650_4, ST_allw_cr_1650_5
                        , ST_allw_cr_1800_1, ST_allw_cr_1800_2, ST_allw_cr_1800_3, ST_allw_cr_1800_4, ST_allw_cr_1800_5
                        , ST_allw_cr_2000_3, ST_allw_cr_2000_4
                        , ST_allw_cr_2400_2, ST_allw_cr_2400_3, ST_allw_cr_2400_4, ST_allw_cr_2400_5]

In [ ]:
record['recordtime'] = record['recordtime'].apply(track_record)

In [ ]:
# pre-processing on form_record

form_record['FinishTime'] = form_record['FinishTime'].str.replace('  ', '00.00.00')
form_record[['FinishTime']] = form_record[['FinishTime']].fillna(value='00.00.00')

#form_record[['FinishTime_m', 'FinishTime_s']] = form_record['FinishTime'].str.split(".", 1, expand=True)
#form_record['FinishTime_m'] = form_record['FinishTime_m'].astype(int)
#form_record['FinishTime_s'] = form_record['FinishTime_s'].astype(float)
#form_record[['FinishTime_m', 'FinishTime_s']]

form_record['FT'] = form_record['FinishTime'].apply(track_record)

form_record = form_record.merge(record, on = ['RC', 'Track', 'Dist', 'class'], how='left')

form_record['SPRAT'] = 100 - (form_record['FT'] - form_record['recordtime'])/ 0.2
# avoid distortion from WV matches
form_record["SPRAT"].mask(form_record['FT'] == 0, np.nan, inplace=True)

form_record["SPRAT_ind"] = np.where( form_record["SPRAT"].isna(), 1, 0)
form_record["SPRAT_ind"] = form_record["SPRAT_ind"].shift(+1, fill_value=0)

In [ ]:
print(form_record.shape)
form_record_R4YO = form_record[form_record['class'] != 'R']
form_record_R4YO = form_record_R4YO[form_record_R4YO['class'] != '4YO']
print(form_record_R4YO.shape)
print(form_record.shape)

In [ ]:
# for ind
form_record_R4YO['AVESPRAT_check'] = form_record_R4YO.groupby(['horseid'])['SPRAT'].apply(lambda x: x.rolling(4).mean())
form_record_R4YO['AVESPRAT'] = form_record_R4YO.groupby(['horseid'])['AVESPRAT_check'].shift(+1, fill_value=0)

form_record_R4YO['AVESPRAT_ind'] = np.where(form_record_R4YO['AVESPRAT'].isna(), 1, 0)
form_record_R4YO["AVESPRAT_ind"].mask(form_record_R4YO['debut'] == 1, 0, inplace=True)

# this is the calculation
form_record_R4YO['AVESPRAT_check'] = form_record_R4YO.groupby(['horseid'])['SPRAT'].apply(lambda x: x.rolling(4, min_periods=1).mean())
form_record_R4YO['AVESPRAT'] = form_record_R4YO.groupby(['horseid'])['AVESPRAT_check'].shift(+1, fill_value=0)

In [ ]:
form_record = form_record.merge(form_record_R4YO[['ind', 'AVESPRAT_check', 'AVESPRAT', 'AVESPRAT_ind']], on = 'ind', how = 'left')

In [ ]:
form_record.columns.values

In [ ]:
#form_record.to_csv (r'C:\Users\chunl\Desktop\form_record.csv', index = False, header=True)

In [ ]:
#(form_record['WV_ind'] != form_record['SPRAT_ind']).sum()
'''
Difference between 4 binary variables we have used so far

'WV_ind': place of last races
'WV_3_ind': Average place of last 3 races
'SPRAT_ind': rating of last races
'AVESPRAT_ind': Average ratings of last 4 races

place =/= ratings
* Ratings is not calculable from special matches(e.g. R, 4YO) *
'''

In [ ]:
form_record[['Date', 'horseid', 'Pla', 'FinishTime', 'FT', 'recordtime', 'SPRAT', 'SPRAT_ind', 'AVESPRAT_ind', 'AVESPRAT_check', 'AVESPRAT']].to_csv (r'C:\Users\chunl\Desktop\form_record.csv', index = False, header=True)

In [ ]:
# consolidate variables                           
form_record = form_record[['RaceIndex', 'Pla', 'Date', 'RC/Track/Course', 'Dist', 'Ground',
                           'RaceClass', 'Draw', 'Rating', 'Trainer', 'Jockey', 'LBW', 'WinOdds',
                           'ActWt', 'RunPo', 'FinishTime', 'Declare_Horse_Wt', 'horseid', 'formatted_date', 'ind', 'prize_money', 'Place',
                           'max_horse', 'n_fp', 'Horse', 'n_break_po', 'n_final_call_po', 'RunPo_1', 'careerstart',
                           
                           'win_rate', 'place_rate', 'show_rate', 'HNOWIN', 'HNOPLACE', 'HNOSHOW', 
                           'j_win_rate', 'j_place_rate', 'j_show_rate', 'JNOWIN', 'JNOPLACE', 'JNOSHOW', 
                           'j_win_rate_1yr', 'j_place_rate_1yr', 'j_show_rate_1yr', 
                           't_win_rate', 't_place_rate', 't_show_rate', 'TNOWIN', 'TNOPLACE', 'TNOSHOW', 
                           't_win_rate_1yr', 't_place_rate_1yr', 't_show_rate_1yr',
                           
                           'avg_break_po', 'avg_break_po_120', 'place_last', 'place_secondlast', 'place_thirdlast', 'place_last_3', 'WV_ind', 'WVsecond_ind', 'WVthird_ind', 'WV_3_ind',
                           
                           'debut', 'DAYSINCE', 'LOGDAYSINCE', 'SPRAT', 'SPRAT_ind', 'AVESPRAT_ind', 'AVESPRAT']]
                           
# SPRAT_ind is for last speed rating

In [ ]:
'''
next paper here
'''

### Previous odds
### Previous prizemoney
### Previous final call position
### Indicator, no previous final call position
### previous distance
### previous odds*prizemoney
### previous weight carried
### days since previous race (done)

In [ ]:
# bookmakers' odds of previous race
form_record['odd_pre'] = form_record.groupby(['horseid'])['WinOdds'].shift(+1, fill_value=0)
#form_record["odd_pre"].mask(form_record['odd_pre'] == '  ', 0, inplace=True)
form_record["odd_pre"].mask(form_record['odd_pre'] == '  ', np.nan, inplace=True)
form_record['odd_pre'] = form_record.groupby(['horseid'])['odd_pre'].fillna(method='ffill')
form_record['odd_pre'] = form_record['odd_pre'].astype(float)

# total prizemoney in previous race
form_record['prize_money'] = form_record['prize_money'].str.replace(",", "")
form_record['prize_money'] = form_record['prize_money'].astype(int)
form_record['pm_pre'] = form_record.groupby(['horseid'])['prize_money'].shift(+1, fill_value=0)

# final call position of previous race = last running position of previous race (done)
form_record['n_final_call_po'] = form_record.groupby(['horseid'])['n_final_call_po'].fillna(method='ffill')
form_record['nfcp_pre'] = form_record.groupby(['horseid'])['n_final_call_po'].shift(+1, fill_value=0)

# indicator, no call position of previous race (done)
'''
= WV_ind
'''

# distance of previous race
form_record['Dist_pre'] = form_record.groupby(['horseid'])['Dist'].shift(+1, fill_value=0)

# odd * prizemoney of previous race
form_record["WinOdds"].mask(form_record['WinOdds'] == '  ', 0, inplace=True)
form_record['WinOdds'] = form_record['WinOdds'].astype(float)
form_record['oddpm_pre'] = form_record['pm_pre'] * form_record['odd_pre']


# weight carried in previous race
form_record['WTCARRIED_pre'] = form_record.groupby(['horseid'])['ActWt'].shift(+1, fill_value=0)

# days since previous race

In [ ]:
form_record[['Date', 'horseid', 'Pla', 'WinOdds', 'odd_pre', 'prize_money', 'pm_pre', 'n_final_call_po', 'nfcp_pre', 'WV_ind', 'Dist', 'Dist_pre', 'oddpm_pre', 'ActWt', 'WTCARRIED_pre']].to_csv (r'C:\Users\chunl\Desktop\form_record.csv', index = False, header=True)

In [ ]:
# consolidate variables
form_record = form_record[['RaceIndex', 'Pla', 'Date', 'RC/Track/Course', 'Dist', 'Ground',
                           'RaceClass', 'Draw', 'Rating', 'Trainer', 'Jockey', 'LBW', 'WinOdds',
                           'ActWt', 'RunPo', 'FinishTime', 'Declare_Horse_Wt', 'horseid', 'formatted_date', 'ind', 'prize_money', 'Place',
                           'max_horse', 'n_fp', 'Horse', 'n_break_po', 'n_final_call_po', 'RunPo_1', 'careerstart',
                           
                           'win_rate', 'place_rate', 'show_rate', 'HNOWIN', 'HNOPLACE', 'HNOSHOW', 
                           'j_win_rate', 'j_place_rate', 'j_show_rate', 'JNOWIN', 'JNOPLACE', 'JNOSHOW', 
                           'j_win_rate_1yr', 'j_place_rate_1yr', 'j_show_rate_1yr', 
                           't_win_rate', 't_place_rate', 't_show_rate', 'TNOWIN', 'TNOPLACE', 'TNOSHOW', 
                           't_win_rate_1yr', 't_place_rate_1yr', 't_show_rate_1yr',
                           
                           'avg_break_po', 'avg_break_po_120', 'place_last', 'place_secondlast', 'place_thirdlast', 'place_last_3', 'WV_ind', 'WVsecond_ind', 'WVthird_ind', 'WV_3_ind',
                           
                           'debut', 'DAYSINCE', 'LOGDAYSINCE', 'SPRAT', 'SPRAT_ind', 'AVESPRAT_ind', 'AVESPRAT',
                           
                           'odd_pre', 'pm_pre', 'nfcp_pre', 'Dist_pre', 'oddpm_pre', 'WTCARRIED_pre']]

# SPRAT_ind is for last speed rating

In [ ]:
'''
next paper here
'''

In [ ]:
form_record.Place.value_counts()

### W/RACE
### LSPEDRAT - previous speed rating
### new distance

In [ ]:
# W/RACE
import numpy as np
'''
Season Indicatior to calculate the CUMULATIVE season PRIZE MONEY (check)
'''

#form_record['month'] = form_record['Date'].str[5:7]
form_record['season'] = np.where(form_record['Date'].dt.month >= 9, form_record['Date'].dt.year +1, form_record['Date'].dt.year )

# change prize_money to int
#form_record['prize_money'] = form_record['prize_money'].str.replace(',', '')
#form_record['prize_money'] = form_record['prize_money'].astype(int)

# winnings per match
#form_record['winnings_per_match1'] = np.where(form_record.Place == '1', form_record['prize_money'] * 0.57, 0)
#form_record['winnings_per_match2'] = np.where(form_record.Pla == '2', form_record['prize_money'] * 0.22, 0)
#form_record['winnings_per_match3'] = np.where(form_record.Pla == '3', form_record['prize_money'] * 0.115, 0)
#form_record['winnings_per_match4'] = np.where(form_record.Pla == '4', form_record['prize_money'] * 0.06, 0)
#form_record['winnings_per_match5'] = np.where(form_record.Pla == '5', form_record['prize_money'] * 0.035, 0)

form_record['winnings_per_match'] = (np.where(form_record.Place == '1', form_record['prize_money'] * 0.57, 0) + np.where(form_record.Place == '1 DH', form_record['prize_money'] * 0.57 / 2, 0) + 
                                     np.where(form_record.Place == '2', form_record['prize_money'] * 0.22, 0) + np.where(form_record.Place == '2 DH', form_record['prize_money'] * 0.22 / 2 , 0) + 
                                     np.where(form_record.Place == '3', form_record['prize_money'] * 0.115, 0) + np.where(form_record.Place == '3 DH', form_record['prize_money'] * 0.115 / 2 , 0) +
                                     np.where(form_record.Place == '4', form_record['prize_money'] * 0.06, 0) + np.where(form_record.Place == '4 DH', form_record['prize_money'] * 0.06 / 2 , 0) +
                                     np.where(form_record.Place == '5', form_record['prize_money'] * 0.035, 0) + np.where(form_record.Place == '5 DH', form_record['prize_money'] * 0.035 / 2 , 0))
#form_record['winnings_per_match1'] + form_record['winnings_per_match2'] + form_record['winnings_per_match3'] + form_record['winnings_per_match4'] + form_record['winnings_per_match5']

form_record['RACE'] = form_record.groupby(['horseid', 'season']).cumcount()

# sum over the season for checking
form_record['season_pm'] = form_record.groupby(['horseid', 'season'])['winnings_per_match'].transform('sum')

# calculate the rolling prize money (W/RACE)
form_record = form_record.sort_values(by=['horseid', 'formatted_date'])

form_record['WINNINGS'] = form_record.groupby(['horseid', 'season'])['winnings_per_match'].apply(lambda x: x.cumsum())
form_record['W/RACE'] = form_record.groupby(['horseid', 'season'])['WINNINGS'].shift(+1, fill_value=0)
form_record['W/RACE'] = form_record['W/RACE'] / form_record['RACE']

In [ ]:
# LSPEDRAT
form_record['LSPEDRAT'] = form_record.groupby(['horseid'])['SPRAT'].shift(+1, fill_value=0)
form_record['LSPEDRAT'] = form_record.groupby(['horseid'])['LSPEDRAT'].fillna(method='ffill')

In [ ]:
# NEWDIST
form_record['Dist_1'] = form_record.groupby(['horseid'])['Dist'].shift(+1)
form_record['Dist_2'] = form_record.groupby(['horseid'])['Dist'].shift(+2)
form_record['Dist_3'] = form_record.groupby(['horseid'])['Dist'].shift(+3)
form_record['Dist_4'] = form_record.groupby(['horseid'])['Dist'].shift(+4)

form_record['NEWDIST_1'] = np.where( (form_record['Dist'] - form_record['Dist_1']) > 200, 1, 0)
form_record['NEWDIST_2'] = np.where( (form_record['Dist'] - form_record['Dist_2']) > 200, 1, 0)
form_record['NEWDIST_3'] = np.where( (form_record['Dist'] - form_record['Dist_3']) > 200, 1, 0)
form_record['NEWDIST_4'] = np.where( (form_record['Dist'] - form_record['Dist_4']) > 200, 1, 0)

form_record['NEWDIST_NO'] = form_record['NEWDIST_1'] + form_record['NEWDIST_2'] + form_record['NEWDIST_3'] + form_record['NEWDIST_4']
form_record['NEWDIST'] = np.where(form_record['NEWDIST_NO'] >= 3, 1, 0)

In [ ]:
form_record[['Date', 'horseid', 'Pla', 'season', 'RACE', 'winnings_per_match', 'season_pm', 'WINNINGS', 'W/RACE', 'SPRAT', 'LSPEDRAT', 'SPRAT_ind'
            , 'Dist', 'NEWDIST_1', 'NEWDIST_2', 'NEWDIST_3', 'NEWDIST_4', 'NEWDIST']].to_csv (r'C:\Users\chunl\Desktop\form_record.csv', index = False, header=True)

In [ ]:
# consolidate variables
form_record = form_record[['RaceIndex', 'Pla', 'Date', 'RC/Track/Course', 'Dist', 'Ground',
                           'RaceClass', 'Draw', 'Rating', 'Trainer', 'Jockey', 'LBW', 'WinOdds',
                           'ActWt', 'RunPo', 'FinishTime', 'Declare_Horse_Wt', 'horseid', 'formatted_date', 'ind', 'prize_money', 'Place',
                           'max_horse', 'n_fp', 'Horse', 'n_break_po', 'n_final_call_po', 'RunPo_1', 'careerstart',
                           
                           'win_rate', 'place_rate', 'show_rate', 'HNOWIN', 'HNOPLACE', 'HNOSHOW', 
                           'j_win_rate', 'j_place_rate', 'j_show_rate', 'JNOWIN', 'JNOPLACE', 'JNOSHOW', 
                           'j_win_rate_1yr', 'j_place_rate_1yr', 'j_show_rate_1yr', 
                           't_win_rate', 't_place_rate', 't_show_rate', 'TNOWIN', 'TNOPLACE', 'TNOSHOW', 
                           't_win_rate_1yr', 't_place_rate_1yr', 't_show_rate_1yr',
                           
                           'avg_break_po', 'avg_break_po_120', 'place_last', 'place_secondlast', 'place_thirdlast', 'place_last_3', 'WV_ind', 'WVsecond_ind', 'WVthird_ind', 'WV_3_ind',
                           
                           'debut', 'DAYSINCE', 'LOGDAYSINCE', 'SPRAT', 'SPRAT_ind', 'AVESPRAT_ind', 'AVESPRAT',
                           
                           'odd_pre', 'pm_pre', 'nfcp_pre', 'Dist_pre', 'oddpm_pre', 'WTCARRIED_pre',
                           
                           'season', 'RACE', 'season_pm', 'W/RACE', 'LSPEDRAT', 'NEWDIST']]

# SPRAT_ind is for last speed rating
# 'season', 'RACE', 'season_pm' for checking only ('SPRAT' as well)

In [ ]:
'''
next paper here
'''

### FINISH1 - recency-weighted mean of past normalized finishing position
### FIRSTCALL - recency-weighted mean of past first call position
### FIRSTCALLDIST
### LENGTHBEHIND - recency-weighted mean of past lengths beaten, distance normalized
### FINISH2 - Average past normalized finishing position
### LASTRACE - normalized finish of this horse in its race * the recency weight of that race

In [ ]:
oneday = pd.Timedelta(days=1)

def FIRSTCALL(frame):
    df = frame[['Horse', 'Date', 'n_break_po']].reset_index(drop=True)
    df = df.sort_values(by=['Date'])
    #FINISH1 = []
    for i in range(0, df.shape[0]):
        day = ( (df["Date"][i] - df["Date"][0:i]) / oneday).to_numpy()
        if np.sum(1/day) == 0:
            firstcall = 0.0
        else:
            firstcall = np.sum( ( 1/day * df["n_break_po"][0:i].to_numpy().T) ) / np.sum(1/day)
        df.loc[df.index[i],'FIRSTCALL'] = firstcall

    df.loc[df.index[0],'FIRSTCALL'] = 0
    df['FIRSTCALL120'] = df.groupby(['Horse'])['n_break_po'].transform(lambda x: x.ewm(halflife='120 days', times=df['Date']).mean())
    df['FIRSTCALL120'] = df.groupby(['Horse'])['FIRSTCALL120'].shift(+1, fill_value=0)
    df = df.drop(['n_break_po'], axis = 1)
    frame = frame.merge(df, on = ['Horse', 'Date'])
    return frame

In [ ]:
form_record_FIRSTCALL = form_record.dropna(subset=['n_break_po'])
import time
start = time.time()

form_record_FIRSTCALL = form_record_FIRSTCALL.groupby('Horse').apply(FIRSTCALL)

print(time.time() - start)

In [ ]:
form_record_FIRSTCALL.reset_index(inplace=True, drop=True)
form_record = form_record.merge(form_record_FIRSTCALL[['Horse', 'Date', 'FIRSTCALL', 'FIRSTCALL120']], on = ['Horse', 'Date'])

In [ ]:
'''# FIRSTCALL

import time
start = time.time()

form_record = form_record.groupby('Horse').apply(FIRSTCALL)

print(time.time() - start)'''

In [ ]:
form_record['FIRSTCALLDIST'] = form_record['FIRSTCALL'] * form_record['Dist']
form_record['FIRSTCALL120DIST'] = form_record['FIRSTCALL120'] * form_record['Dist']

In [ ]:
form_record[['horseid', 'Date', 'n_break_po', 'Dist', 'FIRSTCALL', 'FIRSTCALL120', 'FIRSTCALLDIST', 'FIRSTCALL120DIST']]
#.to_csv (r'C:\Users\chunl\Desktop\form_record.csv', index = False, header=True)

In [ ]:
import fractions
from fractions import Fraction

def mixed_to_float(x):
    return float(sum(fractions.Fraction(term) for term in x.split()))

In [ ]:
# LENGTHBEHIND

# Handleing abnormal LBW
form_record["LBW"].mask(form_record['ind'] == 'HK_2011_N157_20121219', "N", inplace=True)
form_record["LBW"].mask(form_record['ind'] == 'HK_2013_S394_20150426', "NOSE", inplace=True)

form_record['LBW_float'] = form_record['LBW']

# pre-process
form_record['LBW_float'] = form_record['LBW_float'].str.replace("-", " ")
form_record['LBW_float'].mask(form_record['LBW_float'] == " ", '0', inplace=True)

# This is for applying mixed_to_float
#race_result_concat['LBW_float'].mask(race_result_concat['Plc.'] == '1', '0', inplace=True)
#race_result_concat['LBW_float'].mask(race_result_concat['Plc.'] == '1 DH', '0', inplace=True)
#race_result_concat['LBW_float'] = np.where(race_result_concat['Plc.'] == '1', '0', race_result_concat['LBW_float'])
#race_result_concat['LBW_float'] = np.where(race_result_concat['Plc.'] == '1 DH', '0', race_result_concat['LBW_float'])

# pre-process
form_record['LBW_float'] = form_record['LBW_float'].str.replace('+', "")
form_record['LBW_float'] = form_record['LBW_float'].str.replace("NOSE", "1/50")
form_record['LBW_float'] = form_record['LBW_float'].str.replace("SH", "1/20")
form_record['LBW_float'] = form_record['LBW_float'].str.replace("HD", "1/10")
form_record['LBW_float'] = form_record['LBW_float'].str.replace("N", "1/4")
form_record['LBW_float'] = form_record['LBW_float'].str.replace("DH", "")
form_record['LBW_float'] = form_record['LBW_float'].str.replace("ML", '99')
form_record['LBW_float'] = form_record['LBW_float'].str.replace("TO", '99')
form_record[['LBW_float']] = form_record[['LBW_float']].fillna(value=99)
form_record['LBW_float'] = form_record['LBW_float'].astype(str)

# apply the function
form_record['LBW_float'] = form_record['LBW_float'].apply(mixed_to_float)
form_record['n_LBW'] = form_record['LBW_float'] / form_record['Dist']


In [ ]:
oneday = pd.Timedelta(days=1)

def recency(frame):
    df = frame[['Horse', 'Date', 'n_fp', 'n_LBW']].reset_index(drop=True)
    df = df.sort_values(by=['Date'])
    df['days'] = df.groupby(['Horse'])['Date'].diff().dt.days.fillna(0)
    #df = frame.sort_values(by=['horse_name', 'Date'])[['Horse', 'Date', 'n_fp', 'date', 'days', 'rw', 'rw_t', 'NO', 'break_po', 'n_LBW']].reset_index(drop=True)
    #FINISH1 = []
    for i in range(0, df.shape[0]):
        day = ( (df["Date"][i] - df["Date"][0:i]) / oneday).to_numpy()
        recency = np.sum( ( 1/day * df["n_fp"][0:i].to_numpy().T ) / np.sum(1/day) )
        #firstcall = np.sum( ( 1/day * df["break_po"][0:i].to_numpy().T) ) / np.sum(1/day)
        #WIN = np.sum( ( df["rw_t"][0:i].to_numpy() * df[[" "]][0:i].to_numpy()[::-1].T) ) / np.sum(df["rw_t"][0:i].to_numpy())
        FINISH2 = df["n_fp"][0:i].mean()
        LASTRACE = (1 / df.loc[df.index[i],'days']) * df.loc[df.index[i-1],'n_fp']
        LENGTHBEHIND = np.sum( ( 1/day * df["n_LBW"][0:i].to_numpy().T) ) / np.sum(1/day)
        #np.sum( ( grp.iloc[0:i, 5].to_numpy() * grp.iloc[0:i, 1].to_numpy()[::-1].T) ) / np.sum(grp.iloc[0:i, 5].to_numpy())
        #FINISH1 = np.append(FINISH1, recency)
        #grp['FINISH1'] = [FINISH1]
        df.loc[df.index[i],'FINISH1'] = recency
        #df.loc[df.index[i],'FIRSTCALL'] = firstcall
        df.loc[df.index[i],'FINISH2'] = FINISH2
        df.loc[df.index[i],'LASTRACE'] = LASTRACE
        df.loc[df.index[i],'LENGTHBEHIND'] = LENGTHBEHIND

    df.loc[df.index[0],'FINISH1'] = 0
    #df.loc[df.index[0],'FIRSTCALL'] = 0
    df.loc[df.index[0],'FINISH2'] = 0
    df.loc[df.index[0],'LASTRACE'] = 0
    df.loc[df.index[0],'LENGTHBEHIND'] = 0
    
    df['FINISH120'] = df.groupby(['Horse'])['n_fp'].transform(lambda x: x.ewm(halflife='120 days', times=df['Date']).mean())
    df['LENGTHBEHIND120'] = df.groupby(['Horse'])['n_LBW'].transform(lambda x: x.ewm(halflife='120 days', times=df['Date']).mean())
    
    df['FINISH120'] = df.groupby(['Horse'])['FINISH120'].shift(+1, fill_value=0)
    df['LENGTHBEHIND120'] = df.groupby(['Horse'])['LENGTHBEHIND120'].shift(+1, fill_value=0)
    
    df = df.drop(['n_fp', 'n_LBW'], axis = 1)
    frame = frame.merge(df, on = ['Horse', 'Date'])
    return frame

In [ ]:
# FINISH1, (WINHISTORY,) FINISH2, LASTRACE, LENGTHBEHIND
import time
start = time.time()

form_record.reset_index(inplace=True, drop=True)
form_record = form_record.groupby('Horse').apply(recency)

print(time.time() - start)

In [ ]:
form_record[['Horse', 'Date', 'days', 'LBW', 'LBW_float', 'Dist', 'n_fp', 'n_LBW', 'FINISH1', 'FINISH120', 'FINISH2', 'LASTRACE', 'LENGTHBEHIND', 'LENGTHBEHIND120']]

In [ ]:
form_record.columns

In [ ]:
# consolidate variables
form_record = form_record[['RaceIndex', 'Pla', 'Date', 'RC/Track/Course', 'Dist', 'Ground',
                           'RaceClass', 'Draw', 'Rating', 'Trainer', 'Jockey', 'LBW', 'WinOdds',
                           'ActWt', 'RunPo', 'FinishTime', 'Declare_Horse_Wt', 'horseid', 'formatted_date', 'ind', 'prize_money', 'Place',
                           'max_horse', 'n_fp', 'Horse', 'n_break_po', 'n_final_call_po', 'RunPo_1', 'careerstart',
                           
                           'win_rate', 'place_rate', 'show_rate', 'HNOWIN', 'HNOPLACE', 'HNOSHOW', 
                           'j_win_rate', 'j_place_rate', 'j_show_rate', 'JNOWIN', 'JNOPLACE', 'JNOSHOW', 
                           'j_win_rate_1yr', 'j_place_rate_1yr', 'j_show_rate_1yr', 
                           't_win_rate', 't_place_rate', 't_show_rate', 'TNOWIN', 'TNOPLACE', 'TNOSHOW', 
                           't_win_rate_1yr', 't_place_rate_1yr', 't_show_rate_1yr',
                           
                           'avg_break_po', 'avg_break_po_120', 'place_last', 'place_secondlast', 'place_thirdlast', 'place_last_3', 'WV_ind', 'WVsecond_ind', 'WVthird_ind', 'WV_3_ind',
                           
                           'debut', 'DAYSINCE', 'LOGDAYSINCE', 'SPRAT', 'SPRAT_ind', 'AVESPRAT_ind', 'AVESPRAT',
                           
                           'odd_pre', 'pm_pre', 'nfcp_pre', 'Dist_pre', 'oddpm_pre', 'WTCARRIED_pre',
                           
                           'season', 'RACE', 'season_pm', 'W/RACE', 'LSPEDRAT', 'NEWDIST',
                           
                           'FIRSTCALL', 'FIRSTCALL120', 'FIRSTCALLDIST', 'FIRSTCALL120DIST', 'FINISH1', 'FINISH2', 'LASTRACE', 'LENGTHBEHIND', 'FINISH120', 'LENGTHBEHIND120']]

# SPRAT_ind is for last speed rating
# 'season', 'RACE', 'season_pm' for checking only ('SPRAT' as well)

In [ ]:
'''
next paper here
'''
form_record.reset_index(inplace=True, drop=True)
form_record = form_record.sort_values(by=['Date'])

### disavesr
### go_avesr

In [ ]:
# disavesr
form_record['disavesr_check'] = form_record.groupby(['horseid', 'Dist'])['SPRAT'].apply(lambda x: x.expanding().mean())
form_record['disavesr'] = form_record.groupby(['horseid', 'Dist'])['disavesr_check'].shift(+1, fill_value=0)
form_record['disavesr'] = form_record.groupby(['horseid'])['disavesr'].fillna(method='ffill')

In [ ]:
# go_avesr
form_record['go_avesr_check'] = form_record.groupby(['horseid', 'Ground'])['SPRAT'].apply(lambda x: x.expanding().mean())
form_record['go_avesr'] = form_record.groupby(['horseid', 'Ground'])['go_avesr_check'].shift(+1, fill_value=0)
form_record['go_avesr'] = form_record.groupby(['horseid'])['go_avesr'].fillna(method='ffill')

In [ ]:
form_record[['Horse', 'Date', 'SPRAT', 'disavesr_check', 'disavesr', 'go_avesr_check', 'go_avesr']]

In [ ]:
form_record.columns

In [ ]:
# consolidate variables
form_record = form_record[['RaceIndex', 'Pla', 'Date', 'RC/Track/Course', 'Dist', 'Ground',
                           'RaceClass', 'Draw', 'Rating', 'Trainer', 'Jockey', 'LBW', 'WinOdds',
                           'ActWt', 'RunPo', 'FinishTime', 'Declare_Horse_Wt', 'horseid', 'formatted_date', 'ind', 'prize_money', 'Place',
                           'max_horse', 'n_fp', 'Horse', 'n_break_po', 'n_final_call_po', 'RunPo_1', 'careerstart',
                           
                           'win_rate', 'place_rate', 'show_rate', 'HNOWIN', 'HNOPLACE', 'HNOSHOW', 
                           'j_win_rate', 'j_place_rate', 'j_show_rate', 'JNOWIN', 'JNOPLACE', 'JNOSHOW', 
                           'j_win_rate_1yr', 'j_place_rate_1yr', 'j_show_rate_1yr', 
                           't_win_rate', 't_place_rate', 't_show_rate', 'TNOWIN', 'TNOPLACE', 'TNOSHOW', 
                           't_win_rate_1yr', 't_place_rate_1yr', 't_show_rate_1yr',
                           
                           'avg_break_po', 'avg_break_po_120', 'place_last', 'place_secondlast', 'place_thirdlast', 'place_last_3', 'WV_ind', 'WVsecond_ind', 'WVthird_ind', 'WV_3_ind',
                           
                           'debut', 'DAYSINCE', 'LOGDAYSINCE', 'SPRAT', 'SPRAT_ind', 'AVESPRAT_ind', 'AVESPRAT',
                           
                           'odd_pre', 'pm_pre', 'nfcp_pre', 'Dist_pre', 'oddpm_pre', 'WTCARRIED_pre',
                           
                           'season', 'RACE', 'season_pm', 'W/RACE', 'LSPEDRAT', 'NEWDIST',
                           
                           'FIRSTCALL', 'FIRSTCALL120', 'FIRSTCALLDIST', 'FIRSTCALL120DIST', 'FINISH1', 'FINISH2', 'LASTRACE', 'LENGTHBEHIND', 'FINISH120', 'LENGTHBEHIND120',
                          
                           'disavesr', 'go_avesr']]

# SPRAT_ind is for last speed rating
# 'season', 'RACE', 'season_pm' for checking only ('SPRAT' as well)

### merge form_record (after FE) into race_result 
### Start to build the final df

In [ ]:
form_record['ind'] = form_record['horseid'] + "_" + form_record['formatted_date'].astype(str)
race_result['ind'] = race_result['horseid'] + "_" + race_result['date'].astype(str)

print('form_record is missing:', list(set(race_result['ind'].unique()) - set(form_record['ind'].unique()))) # void match
print("\n")
print('race_result is missing:', len(list(set(form_record['ind'].unique()) - set(race_result['ind'].unique()))))

In [ ]:
print('form_record is missing:', len(list(set(race_result['ind'].unique()) - set(form_record['ind'].unique()))))

l = list(set(race_result['ind'].unique()) - set(form_record['ind'].unique()))
print(race_result.loc[race_result['ind'].isin(l)]['Plc.'].value_counts(dropna=False))

In [ ]:
print(race_result.shape)

race_result = race_result.merge(form_record, on = ['ind'], how='left', suffixes=('', '_dup'))
print(race_result.shape)
race_result.head()

In [ ]:
race_result.columns.values

In [ ]:
#checking
print((race_result['Jockey'] != race_result['Jockey_dup']).sum())
print((race_result['Trainer'] != race_result['Trainer_dup']).sum())
print((race_result['horseid'] != race_result['horseid_dup']).sum(), race_result.loc[race_result['horseid'] != race_result['horseid_dup']]['Plc.'].value_counts())
print(race_result.loc[race_result['LBW'] != race_result['LBW_dup']]['Plc.'].value_counts())
race_result.loc[race_result['LBW'] != race_result['LBW_dup']][['Plc.', 'LBW', 'LBW_dup']]

In [ ]:
print((race_result['prize_money'] != race_result['prize_money_dup']).sum())
print((race_result['Win Odds'] != race_result['WinOdds']).sum())
print((race_result['Draw'] != race_result['Draw_dup']).sum(), race_result.loc[race_result['Draw'] != race_result['Draw_dup']]['Plc.'].value_counts())

### Take features from racecard and merge into race_result

In [ ]:
race_result["brand_no"] = race_result['horseid'].str[-4:]

# make sure race_result is completed first
# by comparing it to racecard

race_result['check'] = race_result['date'].astype(str) + "_" + race_result['match'].astype(str) + "_" + race_result['brand_no'].astype(str)
racecard['check'] = racecard['date'].astype(str) + "_" +  racecard['match'].astype(str) + "_" + racecard['Brand No.'].astype(str)

In [ ]:
print('racecard is missing:', list(set(race_result['check'].unique()) - set(racecard['check'].unique())))
print("\n")
# This should be blank
print('race_result is missing:', list(set(racecard['check'].unique()) - set(race_result['check'].unique())))

print('racecard is missing:', len(list(set(race_result['check'].unique()) - set(racecard['check'].unique()))))

l = list(set(race_result['check'].unique()) - set(racecard['check'].unique()))
print(race_result.loc[race_result['check'].isin(l)]['Plc.'].value_counts(dropna=False))

# matches with place is the match in 20190323
race_result.loc[(race_result['check'].isin(l)) & (race_result['Plc.'] != 'WV') & (race_result['Plc.'] != 'WX')][['Date']]

In [ ]:
racecard.columns

In [ ]:
racecard['Best Time'] = racecard['Best Time'].str.replace("-", '00.00.00')
racecard[['Best Time']] = racecard[['Best Time']].fillna(value='00.00.00')

In [ ]:
# Turn Best Time into a value
import numpy as np
racecard[['Best_Time_m', 'Best_Time_s']] = racecard['Best Time'].str.split(".", 1, expand=True)
racecard['Best_Time_m'] = racecard['Best_Time_m'].astype(int)
racecard['Best_Time_s'] = racecard['Best_Time_s'].astype(float)
racecard['besttime'] = racecard['Best_Time_m'] * 60 + racecard['Best_Time_s']
racecard['besttime_miss'] = np.where(racecard['besttime'] == 0, 1, 0)
racecard[['Best Time', 'Best_Time_m', 'Best_Time_s', 'besttime', 'besttime_miss']]

In [ ]:
'''
We need 'Wt.+/- (vs Declaration)', 'Best Time', 'Age', 'Season Stakes', 'Priority'
'''
racecard.rename(columns={'Wt.+/- (vs Declaration)': "WTCARRIEDADJ"}, inplace=True)
racecard.rename(columns={'Season Stakes': "Season_Stakes"}, inplace=True)

In [ ]:
print(race_result.shape)
race_result = race_result.merge(racecard[['besttime', 'besttime_miss', 'WTCARRIEDADJ', 'Age', 'Season_Stakes', 'Priority', 'check']], on = ['check'], how='left')
print(race_result.shape)

In [ ]:
race_result['Priority'].value_counts(dropna=False)

In [ ]:
race_result['Priority'] = race_result['Priority'].fillna('0')

In [ ]:
race_result[['Horse', 'Date', 'Season_Stakes', 'season', 'RACE', 'season_pm', 'W/RACE']].to_csv (r'C:\Users\chunl\Desktop\race_result.csv', index = False, header=True)

### Final cleasing 
### And generating features we left to do in this stage

In [ ]:
print(race_result['Plc.'].value_counts(dropna=False))
print(race_result['Pla'].value_counts(dropna=False))

race_result = race_result[race_result['Plc.'] != 'WV']
race_result = race_result[race_result['Plc.'] != 'WV A']
race_result = race_result[race_result['Plc.'] != 'PU']
race_result = race_result[race_result['Plc.'] != 'UR']
race_result = race_result[race_result['Plc.'] != 'WX A']
race_result = race_result[race_result['Plc.'] != 'WX']
race_result = race_result[race_result['Plc.'] != 'FE']
race_result = race_result[race_result['Plc.'] != 'VOID']
race_result = race_result[race_result['Plc.'] != 'DNF']
race_result = race_result[race_result['Plc.'] != 'TNP']
race_result = race_result[race_result['Plc.'] != 'DISQ']
race_result = race_result[race_result['Plc.'] != 'WXNR']
race_result['Plc.'] = race_result['Plc.'].str.replace(' DH', '')

race_result = race_result[race_result['date'] != 20190323]
race_result = race_result[race_result['RaceClass'] != 'R']
race_result = race_result[race_result['RaceClass'] != '4YO']

print(race_result['Plc.'].value_counts(dropna=False))
print(race_result['Pla'].value_counts(dropna=False))

In [ ]:
race_result.columns.values

In [ ]:
race_result['RaceClass'].value_counts() # catergorical
print(race_result['Dist'].dtype)

race_result['Actual Wt.'] = race_result['Actual Wt.'].astype(int)
print('actual weight:', (race_result['Actual Wt.'] != race_result['ActWt']).sum())

race_result['Declar. Horse Wt.'] = race_result['Declar. Horse Wt.'].astype(int)
race_result['Declare_Horse_Wt'] = race_result['Declare_Horse_Wt'].astype(int)
print('Declare_Horse_Wt:', (race_result['Declar. Horse Wt.'] != race_result['Declare_Horse_Wt']).sum())

race_result['Draw'] = race_result['Draw'].astype(int)
race_result['Draw_dup'] = race_result['Draw_dup'].astype(int)
print('Draw:', (race_result['Draw'] != race_result['Draw_dup']).sum())

race_result['Win Odds'] = race_result['Win Odds'].astype(float)
race_result['WinOdds'] = race_result['WinOdds'].astype(float)
print('WinOdds:', (race_result['Win Odds'] != race_result['WinOdds']).sum())

# WTCARRIEDIST
race_result['WTCARRIEDIST'] = race_result['ActWt'] * race_result['Dist']

# WEIGHTDIST
race_result['WEIGHTDIST'] = race_result['Declare_Horse_Wt'] * race_result['Dist']

# WTCARRIEDADJ
race_result['WTCARRIEDADJ'] = race_result['WTCARRIEDADJ'].str.replace("-", "0")
race_result['WTCARRIEDADJ'] = race_result['WTCARRIEDADJ'].astype(int)

# AGET
race_result['Age'] = race_result['Age'].astype(int)
race_result['AGET'] = 0.75
race_result['AGET'] = np.where( ( (race_result['Age'] == 4) | (race_result['Age'] == 5) ), 1, race_result['AGET'])
race_result['AGET'] = np.where( ( (race_result['Age'] == 3) | (race_result['Age'] == 6) ), 0.95, race_result['AGET'])
race_result['AGET'] = np.where( (race_result['Age'] == 7), 0.9, race_result['AGET'])
race_result['AGET'] = np.where( (race_result['Age'] == 8), 0.85, race_result['AGET'])
race_result['AGET'] = np.where( ( (race_result['Age'] == 1) | (race_result['Age'] == 9) ), 0.8, race_result['AGET'])

# AGEBYDIST
race_result['AGEBYDIST'] = race_result['Age'] * race_result['Dist']

# DRAWBYTRACK
race_result[['RC', 'Track', 'Course']] = race_result['RC/Track/Course'].str.split(" / ", expand=True)
race_result['DRAWBYTRACK'] = np.where(race_result['RC'] == 'ST', 1 * race_result['Draw'], 2 * race_result['Draw'])

# prize_money
race_result['prize_money'] = race_result['prize_money'].str.replace(",", "")
race_result['prize_money'] = race_result['prize_money'].astype(int)
print('prize_money:', (race_result['prize_money'] != race_result['prize_money_dup']).sum())

# WEIGHT(REL)
race_result['WEIGHT(REL)'] = race_result['ActWt'] - race_result.groupby(['date', 'match'])['ActWt'].transform('mean')

# WEIGHT(REL)DIST
race_result['WEIGHT(REL)DIST'] = race_result['WEIGHT(REL)'] * race_result['Dist']

In [ ]:
#race_result[['Horse', 'date', 'match', 'ActWt', 'WEIGHT(REL)']].to_csv (r'C:\Users\chunl\Desktop\race_result.csv', index = False, header=True)#

In [ ]:
'''
Check before proceed
'''

In [ ]:
# fillna
print(race_result.shape)

race_result['LBW'].mask(race_result['Plc.'] == '1', '0', inplace=True)

race_result['disavesr'] = np.where(race_result['debut'] == 1, 0, race_result['disavesr'])
race_result['go_avesr'] = np.where(race_result['debut'] == 1, 0, race_result['go_avesr'])
race_result['nfcp_pre'] = np.where(race_result['debut'] == 1, 0, race_result['nfcp_pre'])
race_result['odd_pre'] = np.where(race_result['debut'] == 1, 0, race_result['odd_pre'])
race_result['AVESPRAT'] = np.where(race_result['debut'] == 1, 0, race_result['AVESPRAT'])
race_result['place_last_3'] = np.where(race_result['debut'] == 1, 0, race_result['place_last_3'])
race_result['avg_break_po'] = np.where(race_result['debut'] == 1, 0, race_result['avg_break_po'])
race_result['avg_break_po_120'] = np.where(race_result['debut'] == 1, 0, race_result['avg_break_po_120'])

race_result = race_result[race_result['date'] != 20190323]
race_result = race_result[race_result['RaceClass'] != 'R']
race_result = race_result[race_result['RaceClass'] != '4YO']

# drop course
race_result.drop(columns=['Course'], inplace = True)

print(race_result.shape)

In [ ]:
race_result_na = race_result[race_result.isnull().any(axis=1)]
print(race_result_na.shape)

# NaN because of divided by zero
print(race_result_na['RACE'].value_counts())
race_result_na[['Horse', 'W/RACE', 'season', 'RACE', 'Date']]

In [ ]:
race_result['W/RACE'].mask(race_result['RACE'] == 0, 0, inplace=True)
race_result_na = race_result[race_result.isnull().any(axis=1)]
race_result_na

In [ ]:
print(race_result_na.shape)
link = race_result_na['horseid'].to_numpy()
for i in range(0, race_result_na.shape[0]):
    print('https://racing.hkjc.com/racing/information/Chinese/Horse/OtherHorse.aspx?HorseId=' + link[i])

In [ ]:
race_result.to_csv (r'C:\Users\chunl\Desktop\export_dataframe.csv', index = False, header=True)

In [ ]:
'''
hooray
'''

In [ ]:
race_result_comp = pd.read_csv('C:/Users/chunl/Desktop/code_data/scarping/race_result_comp.csv')
#race_result = pd.read_csv('C:/Users/chunl/Desktop/code_data/scarping/done/race_result.csv', encoding= 'unicode_escape')
print(race_result.shape)
print(race_result_comp.shape)

print(race_result.columns)
print(race_result_comp.columns)
print('\n')
race_result_comp['date_match'] = race_result_comp['date'].astype(str) + race_result_comp['match'].astype(str)

In [ ]:
race_result.rename(columns={'Unnamed: 15': "horse_name"}, inplace=True)
race_result_comp.rename(columns={'Unnamed: 16': "horse_name"}, inplace=True)
# race_result.drop('date_match', axis=1, inplace=True)
race_result = race_result[['Plc.', 'Horse No.', 'Horse', 'Jockey', 'Trainer', 'Actual Wt.',
       'Declar. Horse Wt.', 'Draw', 'LBW', 'RunningPosition', 'Finish Time',
       'Win Odds', 'date', 'match', 'cdr', 'prize_money', 'horse_name',
       'horseid', 'date_match']]

print(race_result.columns)
print(race_result_comp.columns)

In [ ]:
print(race_result.shape)
print(race_result_comp.shape)

### race_result_concat ###

In [ ]:
race_result_concat = pd.concat([race_result, race_result_comp], axis = 0,  ignore_index=True)

In [ ]:
race_result_concat.shape

In [ ]:
race_result_concat.columns

In [ ]:
form_record['ind'] = form_record['horseid'] + "_" + form_record['formatted_date'].astype(str)
race_result_concat['ind'] = race_result_concat['horseid'] + "_" + race_result_concat['date'].astype(str)

#print('form_record is missing:', list(set(race_result_concat['ind'].unique()) - set(form_record['ind'].unique()))) # void match
#print("\n")
print('race_result is missing:', len(list(set(form_record['ind'].unique()) - set(race_result_concat['ind'].unique()))))

In [ ]:
# merge prize money to form_record first to avoid missing resulted from the subsequent drop

form_record = form_record.merge(race_result_concat[['ind', 'prize_money']].drop_duplicates(subset=['ind']), on = 'ind', how = 'left')

In [ ]:
# break position (calculated in race_result_concat)

'''
copy race_result_concat_runpo from race_result_concat
We seperate race_result_concat_runpo from race_result_concat to avoid missing running position contaminate n_fp calc. (matches in 2007-2009 do not have running position but have fp)
'''
race_result_concat_runpo = race_result_concat

race_result_concat = race_result_concat[race_result_concat['Plc.'] != 'WV']
race_result_concat = race_result_concat[race_result_concat['Plc.'] != 'WV A']
race_result_concat = race_result_concat[race_result_concat['Plc.'] != 'PU']
race_result_concat = race_result_concat[race_result_concat['Plc.'] != 'UR']
race_result_concat = race_result_concat[race_result_concat['Plc.'] != 'WX A']
race_result_concat = race_result_concat[race_result_concat['Plc.'] != 'WX']
race_result_concat = race_result_concat[race_result_concat['Plc.'] != 'FE']
race_result_concat = race_result_concat[race_result_concat['Plc.'] != 'VOID']
race_result_concat = race_result_concat[race_result_concat['Plc.'] != 'DNF']
race_result_concat = race_result_concat[race_result_concat['Plc.'] != 'TNP']
race_result_concat = race_result_concat[race_result_concat['Plc.'] != 'DISQ']
race_result_concat = race_result_concat[race_result_concat['Plc.'] != 'WXNR']
race_result_concat['Plc.'] = race_result_concat['Plc.'].str.replace(' DH', '')

# Na for 20100530 races 10 and 11
race_result_concat.dropna(subset=['Plc.'], inplace=True)
print(race_result_concat['Plc.'].value_counts())

race_result_concat['Plc.'] = race_result_concat['Plc.'].astype(int)

race_result_concat = race_result_concat.sort_values(by=['horseid', 'date'])

# place of last races

race_result_concat['n_fp'] = 1 - ( (race_result_concat['Plc.'] - race_result_concat.groupby(['date_match'])['Plc.'].transform('min'))
                                     / (race_result_concat.groupby(['date_match'])['Plc.'].transform('max') - race_result_concat.groupby(['date_match'])['Plc.'].transform('min')) )

In [ ]:
# matches in 2007-2009 do not have running position
race_result_concat_runpo[['RunPo_1','RunPo_2','RunPo_3','RunPo_4','RunPo_5','RunPo_6']] = race_result_concat_runpo.RunningPosition.str.split(expand=True)

# get n_fp from race_result_concat
race_result_concat_runpo = race_result_concat_runpo.merge(race_result_concat[['horseid', 'date', 'n_fp']], on = ['horseid', 'date'], how = 'left')

race_result_concat_runpo.dropna(subset=['RunPo_1'], inplace=True)

race_result_concat_runpo['RunPo_1'] = race_result_concat_runpo['RunPo_1'].astype(int)
race_result_concat_runpo['break_po'] = 1 - ( (race_result_concat_runpo['RunPo_1'] - race_result_concat_runpo.groupby(['date_match'])['RunPo_1'].transform('min'))
                                     / (race_result_concat_runpo.groupby(['date_match'])['RunPo_1'].transform('max') - race_result_concat_runpo.groupby(['date_match'])['RunPo_1'].transform('min')) )

# Final Call Position

race_result_concat_runpo['final_call_po'] = race_result_concat_runpo[['RunPo_1', 'RunPo_2', 'RunPo_3', 'RunPo_4', 'RunPo_5', 'RunPo_6']].ffill(axis=1).iloc[:, -1]

race_result_concat_runpo.dropna(subset=['final_call_po'], inplace=True)

race_result_concat_runpo['final_call_po'] = race_result_concat_runpo['final_call_po'].astype(int)
race_result_concat_runpo['n_final_call_po'] = 1 - ( (race_result_concat_runpo['final_call_po'] - race_result_concat_runpo.groupby(['date_match'])['final_call_po'].transform('min'))
                                     / (race_result_concat_runpo.groupby(['date_match'])['final_call_po'].transform('max') - race_result_concat_runpo.groupby(['date_match'])['final_call_po'].transform('min')) )

In [ ]:
'''
Think about should we drop matches with no RunPo
'''

race_result_concat_runpo['Date'] =  pd.to_datetime(race_result_concat_runpo['date'], format='%Y%m%d')

# Both do on race_result_concat. We can try
# 1) function
# 2) apply it on race_result_concat_runpo
# 3) merge and check

In [ ]:
oneday = pd.Timedelta(days=1)

def FIRSTCALL(frame):
    df = frame.sort_values(by=['horse_name', 'Date'])[['Horse', 'Date', 'break_po']].reset_index()
    #FINISH1 = []
    for i in range(0, df.shape[0]):
        day = ( (df["Date"][i] - df["Date"][0:i]) / oneday).to_numpy()
        firstcall = np.sum( ( 1/day * df["break_po"][0:i].to_numpy().T) ) / np.sum(1/day)
        df.loc[df.index[i],'FIRSTCALL'] = firstcall

    df.loc[df.index[0],'FIRSTCALL'] = 0
    df['FIRSTCALL120'] = df.groupby(['Horse'])['break_po'].transform(lambda x: x.ewm(halflife='120 days', times=df['Date']).mean())
    df['FIRSTCALL120'] = df.groupby(['Horse'])['FIRSTCALL120'].shift(+1, fill_value=0)
    df = df.drop(['break_po'], axis = 1)
    frame = frame.merge(df, on = ['Horse', 'Date'])
    return frame

In [ ]:
race_result_concat_runpo.columns

In [ ]:
# FIRSTCALL

import time
start = time.time()

race_result_concat_runpo = race_result_concat_runpo.groupby('Horse').apply(FIRSTCALL)

print(time.time() - start)

In [ ]:
race_result_concat_runpo[['horseid', 'date', 'FIRSTCALL', 'FIRSTCALL120']]

In [ ]:
# Merge after calc.
race_result_concat = race_result_concat.merge(race_result_concat_runpo[['horseid', 'date', 'RunPo_1', 'break_po', 'final_call_po', 'n_final_call_po', 'FIRSTCALL', 'FIRSTCALL120']], on = ['horseid', 'date'], how = 'left')

In [ ]:
#race_result_concat = race_result_concat.sort_values(by=['date_match', 'Plc.'])
#race_result_concat.to_csv (r'C:\Users\chunl\Desktop\race_result_concat.csv', index = False, header=True)
print(race_result_concat.columns)
print(race_result_concat.shape)

### FE on form_record ###

In [ ]:
print(form_record.shape)
form_record = form_record.merge(race_result_concat, on = ['ind'], how='left', suffixes=('', '_dup'))
print(form_record.shape)

In [ ]:
form_record.columns.values

In [ ]:
form_record_runpo = form_record.dropna(subset=['RunPo_1'])
form_record_runpo = form_record_runpo.sort_values(by=['horseid', 'formatted_date'])

# div is bulit on form_record_runpo
form_record_runpo['div'] = form_record_runpo.groupby('horseid').cumcount()+1

'''
div is for position (form_record_runpo)
occur is for rate (form_record_WV_drop)
'''
form_record = form_record.merge(form_record_runpo[['ind', 'div']], on = ['ind'], how = 'left')

In [ ]:
# average break position (calculated in form_record AFTER merge)

form_record = form_record.sort_values(by=['horseid', 'formatted_date'])

# We use form_record_WV_drop to calculate the div as div is for avg_break_po

form_record['sum_break_po'] = form_record.groupby(['horseid'])['break_po'].apply(lambda x: x.cumsum())

form_record['avg_break_po_check'] = (form_record['sum_break_po']/form_record['div'])
form_record['avg_break_po_check'] = form_record.groupby(['horseid'])['avg_break_po_check'].fillna(method='ffill')

form_record['avg_break_po'] = form_record.groupby(['horseid'])['avg_break_po_check'].shift(+1, fill_value=0)

In [ ]:
# This cell is for WV indicator

#form_record['n_fp'] = form_record.groupby(['horseid'])['n_fp'].fillna(method='ffill')

form_record['place_last'] = form_record.groupby(['horseid'])['n_fp'].shift(+1, fill_value=0)
#form_record['place_last'] = form_record.groupby(['horseid'])['place_last'].fillna(method='ffill')

# average place of last 3 races

form_record['place_last_3_check'] = form_record.groupby(['horseid'])['n_fp'].apply(lambda x: x.rolling(3).mean())
form_record['place_last_3'] = form_record.groupby(['horseid'])['place_last_3_check'].shift(+1, fill_value=0)

In [ ]:
# WV ind

form_record['WV_ind_check'] = np.where( ((form_record['Pla'] == '1') | (form_record['Pla'] == '2') | (form_record['Pla'] == '3') | (form_record['Pla'] == '4') | (form_record['Pla'] == '5') | (form_record['Pla'] == '6')
                                | (form_record['Pla'] == '7') | (form_record['Pla'] == '8') | (form_record['Pla'] == '9') | (form_record['Pla'] == '10') | (form_record['Pla'] == '11') | (form_record['Pla'] == '12')
                                | (form_record['Pla'] == '13') | (form_record['Pla'] == '14')), 0, 1)

form_record['WV_ind'] = form_record.groupby(['horseid'])['WV_ind_check'].shift(+1, fill_value=0)

# WV 3 ind

form_record['WV_3_ind'] = np.where(form_record['place_last_3'].isna(), 1, 0)

In [ ]:
# place of last races (calculated in form_record AFTER merge)

form_record['n_fp'] = form_record.groupby(['horseid'])['n_fp'].fillna(method='ffill')

form_record['place_last'] = form_record.groupby(['horseid'])['n_fp'].shift(+1, fill_value=0)
form_record['place_last'] = form_record.groupby(['horseid'])['place_last'].fillna(method='ffill')

# average place of last 3 races

form_record['place_last_3_check'] = form_record.groupby(['horseid'])['n_fp'].apply(lambda x: x.rolling(3, min_periods=1).mean())
form_record['place_last_3'] = form_record.groupby(['horseid'])['place_last_3_check'].shift(+1, fill_value=0)

In [ ]:
# Final Call Position

form_record['n_final_call_po'] = form_record.groupby(['horseid'])['n_final_call_po'].fillna(method='ffill')
form_record['nfcp_pre'] = form_record.groupby(['horseid'])['n_final_call_po'].shift(+1, fill_value=0)

In [ ]:
form_record.columns.values

In [ ]:
# debut we use form_record_WV_drop

form_record_WV_drop['debut'] = 0
form_record_WV_drop = form_record_WV_drop.sort_values(by= ['formatted_date'])
idx = form_record_WV_drop.groupby('horseid').head(1).index
form_record_WV_drop.loc[idx,'debut'] = 1

# DAYSINCE we use form_record_WV_drop
form_record_WV_drop['DAYSINCE'] = form_record_WV_drop.groupby(['horseid'])['Date'].diff().dt.days.fillna(0)

form_record = form_record.merge(form_record_WV_drop[['ind', 'debut', 'DAYSINCE']], on = 'ind', how = 'left')

In [ ]:
#form_record_WV_drop.loc[form_record_WV_drop['horseid'] == 'HK_2013_S382']
form_record.shape

In [ ]:
form_record_TW = form_record[['RaceIndex', 'Pla', 'Date', 'RC/Track/Course', 'Dist', 'Ground',
                              'RaceClass', 'Draw', 'Rating', 'Trainer', 'Jockey', 'LBW', 'WinOdds',
                              'ActWt', 'RunPo', 'FinishTime', 'Declare_Horse_Wt', 'horseid', 'formatted_date', 'ind', 'careerstart', 'win1', 'win2', 'win3', 'win', 'win_rate_check',
                              'win_rate', 'place', 'place_rate_check', 'place_rate', 'show',
                              'show_rate_check', 'show_rate', 'J_started', 'j_win',
                              'j_place', 'j_show', 'j_win_rate_check', 'j_win_rate',
                              'j_place_rate_check', 'j_place_rate', 'j_show_rate_check',
                              'j_show_rate', 'JNOWIN', 'JNOPLACE', 'JNOSHOW', 'T_started',
                              't_win', 't_place', 't_show', 't_win_rate_check', 't_win_rate',
                              't_place_rate_check', 't_place_rate', 't_show_rate_check',
                              't_show_rate', 'TNOWIN', 'TNOPLACE', 'TNOSHOW', 'RunPo_1',
                              'break_po', 'sum_break_po', 'div', 'avg_break_po_check', 'avg_break_po',
                              'final_call_po', 'n_final_call_po', 'nfcp_pre',
                              'n_fp', 'place_last', 'place_last_3_check', 'place_last_3', 'WV_ind_check', 'WV_ind', 'WV_3_ind',
                              'debut']]

In [ ]:
form_record_TW.to_csv (r'C:\Users\chunl\Desktop\form_record_TW.csv', index = False, header=True)

### CU stat

In [ ]:
# LOGODD


# WTCARRIED ('Actual Wt.') in result
form_record.rename(columns={'ActWt': "WTCARRIED"}, inplace=True)

# WTCARRIEDIST
form_record['WTCARRIEDIST'] = form_record['WTCARRIED'] * form_record['Dist'] 

# WEIGHTDIST ('Declar. Horse Wt.') in result
form_record['Declare_Horse_Wt'] = form_record['Declare_Horse_Wt'].fillna(value = 0)
form_record['Declare_Horse_Wt'] = form_record['Declare_Horse_Wt'].str.replace('  ', '0')
form_record['WEIGHTDIST'] = form_record['Declare_Horse_Wt'].astype(int) * form_record['Dist']

# WTCARRIEDADJ
'''
In racecard
'''

# AGET
'''
In racecard
'''

# DRAWING
'''
form_record['Draw']
'''

In [ ]:
#form_record[['RaceIndex', 'Pla', 'Date', 'Dist', 'Draw', 'WTCARRIED', 'RunPo', 'FinishTime', 'Declare_Horse_Wt', 'Actual Wt.', 'Declar. Horse Wt.', 'WTCARRIEDIST', 'WEIGHTDIST']]

In [ ]:
#form_record[['RC/Track/Course', 'Dist', 'Ground', 'RaceClass', 'cdr', 'RC/Track/Course', 'FinishTime', 'Finish Time']]

In [ ]:
form_record.RaceClass.value_counts()

In [ ]:
form_record['class'] = form_record['RaceClass']
form_record.replace({'class' : { 'G1' : 'G', 'G3' : 'G', 'HKG3' : 'G', 'G2' : 'G', 'HKG1' : 'G', 'HKG2' : 'G' }}, inplace=True)
form_record.replace({'class' : { '4R' : '4', '4S' : '4', '4B' : '4'}}, inplace=True)
form_record.replace({'class' : { '3S' : '3', '3B' : '3'}}, inplace=True)

#ST / Turf / "C+3"
form_record[['RC', 'Track', 'Course']] = form_record['RC/Track/Course'].str.split(" / ", expand=True)

In [ ]:
form_record['class'].value_counts()

In [ ]:
#form_record.loc[form_record['RaceClass'] == '3B'][['Date', 'RC/Track/Course', 'Dist', 'Ground', 'RaceClass', 'cdr', 'RC/Track/Course', 'FinishTime', 'Finish Time']]

In [ ]:
def track_record(course_record):
    m, s = course_record.split(".", 1)
    cr = int(m) * 60 + float(s)
    return cr

In [ ]:
ST_turf_cr_1000_G = '0.55.16'
ST_turf_cr_1000_1 = '0.54.70'
ST_turf_cr_1000_2 = '0.54.69'
ST_turf_cr_1000_3 = '0.54.89'
ST_turf_cr_1000_4 = '0.55.07'
ST_turf_cr_1000_5 = '0.55.64'
ST_turf_cr_1000_Griffin = '0.56.00'

ST_turf_cr_1200_G = '1.07.50'
ST_turf_cr_1200_1 = '1.07.80'
ST_turf_cr_1200_2 = '1.07.89'
ST_turf_cr_1200_3 = '1.08.31'
ST_turf_cr_1200_4 = '1.08.45'
ST_turf_cr_1200_5 = '1.09.24'
ST_turf_cr_1200_Griffin = '1.08.80'

ST_turf_cr_1400_G = '1.20.05'
ST_turf_cr_1400_1 = '1.19.92'
ST_turf_cr_1400_2 = '1.20.29'
ST_turf_cr_1400_3 = '1.20.62'
ST_turf_cr_1400_4 = '1.20.80'
ST_turf_cr_1400_5 = '1.21.53'
ST_turf_cr_1400_Griffin = '1.22.30'

ST_turf_cr_1600_G = '1.32.64'
ST_turf_cr_1600_1 = '1.32.82'
ST_turf_cr_1600_2 = '1.32.96'
ST_turf_cr_1600_3 = '1.33.27'
ST_turf_cr_1600_4 = '1.33.78'
ST_turf_cr_1600_5 = '1.34.38'
ST_turf_cr_1600_Griffin = '1.35.80'

ST_turf_cr_1800_G = '1.45.25'
ST_turf_cr_1800_1 = '1.46.60'
ST_turf_cr_1800_2 = '1.45.88'
ST_turf_cr_1800_3 = '1.46.18'
ST_turf_cr_1800_4 = '1.46.54'
ST_turf_cr_1800_5 = '1.47.60'

ST_turf_cr_2000_G = '1.59.30'
ST_turf_cr_2000_1 = '1.59.66'
ST_turf_cr_2000_2 = '1.59.53'
ST_turf_cr_2000_3 = '2.00.37'
ST_turf_cr_2000_4 = '2.01.62'
ST_turf_cr_2000_5 = '2.01.56'

ST_turf_cr_2200_1 = '2.14.60'
ST_turf_cr_2200_2 = '2.12.42'
ST_turf_cr_2200_3 = '2.16.06'
ST_turf_cr_2200_4 = '2.13.69'
ST_turf_cr_2200_5 = '2.15.42'

ST_turf_cr_2400_G = '2.24.60'
ST_turf_cr_2400_1 = '2.26.10'
ST_turf_cr_2400_2 = '2.29.70'

ST_turf_cr_2400_4 = '2.31.00'

In [ ]:

HV_turf_cr_1000_1 = '0.56.30'
HV_turf_cr_1000_2 = '0.56.25'
HV_turf_cr_1000_3 = '0.56.11'
HV_turf_cr_1000_4 = '0.56.39'
HV_turf_cr_1000_5 = '0.56.90'
HV_turf_cr_1000_Griffin = '0.58.30'

HV_turf_cr_1200_G = '1.09.30'
HV_turf_cr_1200_1 = '1.08.70'
HV_turf_cr_1200_2 = '1.08.52'
HV_turf_cr_1200_3 = '1.08.68'
HV_turf_cr_1200_4 = '1.08.92'
HV_turf_cr_1200_5 = '1.09.73'
HV_turf_cr_1200_Griffin = '1.10.50'

HV_turf_cr_1650_1 = '1.38.70'
HV_turf_cr_1650_2 = '1.38.58'
HV_turf_cr_1650_3 = '1.38.61'
HV_turf_cr_1650_4 = '1.38.83'
HV_turf_cr_1650_5 = '1.39.11'

HV_turf_cr_1800_G = '1.48.10'
HV_turf_cr_1800_1 = '1.47.80'
HV_turf_cr_1800_2 = '1.47.61'
HV_turf_cr_1800_3 = '1.48.30'
HV_turf_cr_1800_4 = '1.48.10'
HV_turf_cr_1800_5 = '1.49.33'

HV_turf_cr_2200_1 = '2.15.44'
HV_turf_cr_2200_2 = '2.15.90'
HV_turf_cr_2200_3 = '2.14.88'
HV_turf_cr_2200_4 = '2.16.06'
HV_turf_cr_2200_5 = '2.16.06'

HV_turf_cr_2400_2 = '2.30.40'
HV_turf_cr_2400_3 = '2.31.20'
HV_turf_cr_2400_4 = '2.31.00'
HV_turf_cr_2400_5 = '2.31.00'

In [ ]:

ST_allw_cr_1200_1 = '1.07.45'
ST_allw_cr_1200_2 = '1.07.27'
ST_allw_cr_1200_3 = '1.07.40'
ST_allw_cr_1200_4 = '1.08.14'
ST_allw_cr_1200_5 = '1.08.46'

ST_allw_cr_1650_1 = '1.36.34'
ST_allw_cr_1650_2 = '1.36.75'
ST_allw_cr_1650_3 = '1.37.22'
ST_allw_cr_1650_4 = '1.38.02'
ST_allw_cr_1650_5 = '1.38.60'

ST_allw_cr_1800_1 = '1.49.40'
ST_allw_cr_1800_2 = '1.47.67'
ST_allw_cr_1800_3 = '1.46.68'
ST_allw_cr_1800_4 = '1.47.54'
ST_allw_cr_1800_5 = '1.47.67'

ST_allw_cr_2000_3 = '2.01.30'
ST_allw_cr_2000_4 = '2.01.80'

ST_allw_cr_2400_2 = '2.31.80'
ST_allw_cr_2400_3 = '2.30.00'
ST_allw_cr_2400_4 = '2.29.10'
ST_allw_cr_2400_5 = '2.31.50'

In [ ]:
# Create a df for track_record

data = [['ST', 'Turf', 1000, 'G'], ['ST', 'Turf', 1000, '1'], ['ST', 'Turf', 1000, '2'], ['ST', 'Turf', 1000, '3'], ['ST', 'Turf', 1000, '4'], ['ST', 'Turf', 1000, '5'], ['ST', 'Turf', 1000, 'GRIFFIN'],
        ['ST', 'Turf', 1200, 'G'], ['ST', 'Turf', 1200, '1'], ['ST', 'Turf', 1200, '2'], ['ST', 'Turf', 1200, '3'], ['ST', 'Turf', 1200, '4'], ['ST', 'Turf', 1200, '5'], ['ST', 'Turf', 1200, 'GRIFFIN'],
        ['ST', 'Turf', 1400, 'G'], ['ST', 'Turf', 1400, '1'], ['ST', 'Turf', 1400, '2'], ['ST', 'Turf', 1400, '3'], ['ST', 'Turf', 1400, '4'], ['ST', 'Turf', 1400, '5'], ['ST', 'Turf', 1400, 'GRIFFIN'],
        ['ST', 'Turf', 1600, 'G'], ['ST', 'Turf', 1600, '1'], ['ST', 'Turf', 1600, '2'], ['ST', 'Turf', 1600, '3'], ['ST', 'Turf', 1600, '4'], ['ST', 'Turf', 1600, '5'], ['ST', 'Turf', 1600, 'GRIFFIN'],
        ['ST', 'Turf', 1800, 'G'], ['ST', 'Turf', 1800, '1'], ['ST', 'Turf', 1800, '2'], ['ST', 'Turf', 1800, '3'], ['ST', 'Turf', 1800, '4'], ['ST', 'Turf', 1800, '5'],
        ['ST', 'Turf', 2000, 'G'], ['ST', 'Turf', 2000, '1'], ['ST', 'Turf', 2000, '2'], ['ST', 'Turf', 2000, '3'], ['ST', 'Turf', 2000, '4'], ['ST', 'Turf', 2000, '5'],
        ['ST', 'Turf', 2200, '1'], ['ST', 'Turf', 2200, '2'], ['ST', 'Turf', 2200, '3'], ['ST', 'Turf', 2200, '4'], ['ST', 'Turf', 2200, '5'],
        ['ST', 'Turf', 2400, 'G'], ['ST', 'Turf', 2400, '1'], ['ST', 'Turf', 2400, '2'], ['ST', 'Turf', 2400, '4'],
        
        ['HV', 'Turf', 1000, '1'], ['HV', 'Turf', 1000, '2'], ['HV', 'Turf', 1000, '3'], ['HV', 'Turf', 1000, '4'], ['HV', 'Turf', 1000, '5'], ['HV', 'Turf', 1000, 'GRIFFIN'],
        ['HV', 'Turf', 1200, 'G'], ['HV', 'Turf', 1200, '1'], ['HV', 'Turf', 1200, '2'], ['HV', 'Turf', 1200, '3'], ['HV', 'Turf', 1200, '4'], ['HV', 'Turf', 1200, '5'], ['HV', 'Turf', 1200, 'GRIFFIN'],
        ['HV', 'Turf', 1650, '1'], ['HV', 'Turf', 1650, '2'], ['HV', 'Turf', 1650, '3'], ['HV', 'Turf', 1650, '4'], ['HV', 'Turf', 1650, '5'],
        ['HV', 'Turf', 1800, 'G'], ['HV', 'Turf', 1800, '1'], ['HV', 'Turf', 1800, '2'], ['HV', 'Turf', 1800, '3'], ['HV', 'Turf', 1800, '4'], ['HV', 'Turf', 1800, '5'],
        ['HV', 'Turf', 2200, '1'], ['HV', 'Turf', 2200, '2'], ['HV', 'Turf', 2200, '3'], ['HV', 'Turf', 2200, '4'], ['HV', 'Turf', 2200, '5'],
        ['HV', 'Turf', 2400, '2'], ['HV', 'Turf', 2400, '3'], ['HV', 'Turf', 2400, '4'], ['HV', 'Turf', 2400, '5'],
        
        ['ST', 'AWT', 1200, '1'], ['ST', 'AWT', 1200, '2'], ['ST', 'AWT', 1200, '3'], ['ST', 'AWT', 1200, '4'], ['ST', 'AWT', 1200, '5'],
        ['ST', 'AWT', 1650, '1'], ['ST', 'AWT', 1650, '2'], ['ST', 'AWT', 1650, '3'], ['ST', 'AWT', 1650, '4'], ['ST', 'AWT', 1650, '5'],
        ['ST', 'AWT', 1800, '1'], ['ST', 'AWT', 1800, '2'], ['ST', 'AWT', 1800, '3'], ['ST', 'AWT', 1800, '4'], ['ST', 'AWT', 1800, '5'],
                                                            ['ST', 'AWT', 2000, '3'], ['ST', 'AWT', 2000, '4'],
                                  ['ST', 'AWT', 2400, '2'], ['ST', 'AWT', 2400, '3'], ['ST', 'AWT', 2400, '4'], ['ST', 'AWT', 2400, '5']] 
  
# Create the pandas DataFrame 
record = pd.DataFrame(data, columns = ['RC', 'Track', 'Dist', 'class']) 

record['recordtime'] = [ST_turf_cr_1000_G, ST_turf_cr_1000_1, ST_turf_cr_1000_2 ,ST_turf_cr_1000_3, ST_turf_cr_1000_4, ST_turf_cr_1000_5, ST_turf_cr_1000_Griffin
                        , ST_turf_cr_1200_G, ST_turf_cr_1200_1, ST_turf_cr_1200_2, ST_turf_cr_1200_3, ST_turf_cr_1200_4, ST_turf_cr_1200_5, ST_turf_cr_1200_Griffin
                        , ST_turf_cr_1400_G, ST_turf_cr_1400_1, ST_turf_cr_1400_2, ST_turf_cr_1400_3, ST_turf_cr_1400_4, ST_turf_cr_1400_5, ST_turf_cr_1400_Griffin
                        , ST_turf_cr_1600_G, ST_turf_cr_1600_1, ST_turf_cr_1600_2, ST_turf_cr_1600_3, ST_turf_cr_1600_4, ST_turf_cr_1600_5, ST_turf_cr_1600_Griffin
                        , ST_turf_cr_1800_G, ST_turf_cr_1800_1, ST_turf_cr_1800_2, ST_turf_cr_1800_3, ST_turf_cr_1800_4, ST_turf_cr_1800_5
                        , ST_turf_cr_2000_G, ST_turf_cr_2000_1, ST_turf_cr_2000_2, ST_turf_cr_2000_3, ST_turf_cr_2000_4, ST_turf_cr_2000_5
                        , ST_turf_cr_2200_1, ST_turf_cr_2200_2, ST_turf_cr_2200_3, ST_turf_cr_2200_4, ST_turf_cr_2200_5
                        , ST_turf_cr_2400_G, ST_turf_cr_2400_1, ST_turf_cr_2400_2, ST_turf_cr_2400_4
                        
                        , HV_turf_cr_1000_1, HV_turf_cr_1000_2, HV_turf_cr_1000_3, HV_turf_cr_1000_4, HV_turf_cr_1000_5, HV_turf_cr_1000_Griffin
                        , HV_turf_cr_1200_G, HV_turf_cr_1200_1, HV_turf_cr_1200_2, HV_turf_cr_1200_3, HV_turf_cr_1200_4, HV_turf_cr_1200_5, HV_turf_cr_1200_Griffin
                        , HV_turf_cr_1650_1, HV_turf_cr_1650_2, HV_turf_cr_1650_3, HV_turf_cr_1650_4, HV_turf_cr_1650_5
                        , HV_turf_cr_1800_G, HV_turf_cr_1800_1, HV_turf_cr_1800_2, HV_turf_cr_1800_3, HV_turf_cr_1800_4, HV_turf_cr_1800_5
                        , HV_turf_cr_2200_1, HV_turf_cr_2200_2, HV_turf_cr_2200_3, HV_turf_cr_2200_4, HV_turf_cr_2200_5
                        , HV_turf_cr_2400_2, HV_turf_cr_2400_3, HV_turf_cr_2400_4, HV_turf_cr_2400_5
                        
                        , ST_allw_cr_1200_1, ST_allw_cr_1200_2, ST_allw_cr_1200_3, ST_allw_cr_1200_4, ST_allw_cr_1200_5
                        , ST_allw_cr_1650_1, ST_allw_cr_1650_2, ST_allw_cr_1650_3, ST_allw_cr_1650_4, ST_allw_cr_1650_5
                        , ST_allw_cr_1800_1, ST_allw_cr_1800_2, ST_allw_cr_1800_3, ST_allw_cr_1800_4, ST_allw_cr_1800_5
                        , ST_allw_cr_2000_3, ST_allw_cr_2000_4
                        , ST_allw_cr_2400_2, ST_allw_cr_2400_3, ST_allw_cr_2400_4, ST_allw_cr_2400_5]

In [ ]:
record['recordtime'] = record['recordtime'].apply(track_record)

In [ ]:
# pre-processing on form_record

form_record['FinishTime'] = form_record['FinishTime'].str.replace('  ', '00.00.00')
form_record[['FinishTime']] = form_record[['FinishTime']].fillna(value='00.00.00')

#form_record[['FinishTime_m', 'FinishTime_s']] = form_record['FinishTime'].str.split(".", 1, expand=True)
#form_record['FinishTime_m'] = form_record['FinishTime_m'].astype(int)
#form_record['FinishTime_s'] = form_record['FinishTime_s'].astype(float)
#form_record[['FinishTime_m', 'FinishTime_s']]

form_record['FT'] = form_record['FinishTime'].apply(track_record)

form_record = form_record.merge(record, on = ['RC', 'Track', 'Dist', 'class'], how='left')

form_record['SPRAT'] = 100 - (form_record['FT'] - form_record['recordtime'])/ 0.2
# avoid distortion from WV matches
form_record["SPRAT"].mask(form_record['FT'] == 0, np.nan, inplace=True)

form_record["SPRAT_ind"] = np.where( form_record["SPRAT"].isna(), 1, 0)
form_record["SPRAT_ind"] = form_record["SPRAT_ind"].shift(+1, fill_value=0)

In [ ]:
print(form_record.shape)
form_record_R4YO = form_record[form_record['class'] != 'R']
form_record_R4YO = form_record_R4YO[form_record_R4YO['class'] != '4YO']
print(form_record_R4YO.shape)
print(form_record.shape)

In [ ]:
# for ind
form_record_R4YO['AVESPRAT_check'] = form_record_R4YO.groupby(['horseid'])['SPRAT'].apply(lambda x: x.rolling(4).mean())
form_record_R4YO['AVESPRAT'] = form_record_R4YO.groupby(['horseid'])['AVESPRAT_check'].shift(+1, fill_value=0)

form_record_R4YO['AVESPRAT_ind'] = np.where(form_record_R4YO['AVESPRAT'].isna(), 1, 0)
form_record_R4YO["AVESPRAT_ind"].mask(form_record_R4YO['debut'] == 1, 0, inplace=True)

# this is the calculation
form_record_R4YO['AVESPRAT_check'] = form_record_R4YO.groupby(['horseid'])['SPRAT'].apply(lambda x: x.rolling(4, min_periods=1).mean())
form_record_R4YO['AVESPRAT'] = form_record_R4YO.groupby(['horseid'])['AVESPRAT_check'].shift(+1, fill_value=0)

In [ ]:
form_record = form_record.merge(form_record_R4YO[['ind', 'AVESPRAT_check', 'AVESPRAT', 'AVESPRAT_ind']], on = 'ind', how = 'left')

In [ ]:
form_record.columns.values

In [ ]:
'''# AVESPRAT (last four race: Initial 100. Compared to track record (course, distance, class), a point is deducted for every 1/5 second)

#form_record.loc[form_record['FinishTime'] == '  ']
form_record['FinishTime'] = form_record['FinishTime'].str.replace('  ', '00.00.00')
form_record[['FinishTime']] = form_record[['FinishTime']].fillna(value='00.00.00')

#form_record[['FinishTime_m', 'FinishTime_s']] = form_record['FinishTime'].str.split(".", 1, expand=True)
#form_record['FinishTime_m'] = form_record['FinishTime_m'].astype(int)
#form_record['FinishTime_s'] = form_record['FinishTime_s'].astype(float)
#form_record[['FinishTime_m', 'FinishTime_s']]

form_record['FT'] = form_record['FinishTime'].apply(track_record)

form_record = form_record.merge(record, on = ['RC', 'Track', 'Dist', 'class'], how='left')

form_record['SPRAT'] = 100 - (form_record['FT'] - form_record['recordtime'])/ 0.2
# avoid distortion from WV matches
form_record["SPRAT"].mask(form_record['FT'] == 0, np.nan, inplace=True)

form_record['AVESPRAT_check'] = form_record.groupby(['horseid'])['SPRAT'].apply(lambda x: x.rolling(4).mean())
form_record['AVESPRAT'] = form_record.groupby(['horseid'])['AVESPRAT_check'].shift(+1, fill_value=0)

form_record['AVESPRAT_ind'] = np.where(form_record['AVESPRAT'].isna(), 1, 0)
form_record["AVESPRAT_ind"].mask(form_record['debut'] == 1, 0, inplace=True)'''

In [ ]:
'''form_record['AVESPRAT_check'] = form_record.groupby(['horseid'])['SPRAT'].apply(lambda x: x.rolling(4, min_periods=1).mean())
form_record['AVESPRAT'] = form_record.groupby(['horseid'])['AVESPRAT_check'].shift(+1, fill_value=0)'''

In [ ]:
#form_record.groupby(['RC', 'Dist', 'class']).describe()
#form_record.to_csv (r'C:\Users\chunl\Desktop\form_record.csv', index = False, header=True)

In [ ]:
# HWINPER = ('win_rate', 'place_rate', 'show_rate')

# HNOWIN = ('win', 'place', 'show') <- shifting needed
form_record = form_record.sort_values(by=['horseid', 'formatted_date'])
form_record['HNOWIN'] = form_record.groupby(['horseid'])['win'].shift(+1, fill_value=0)
form_record['HNOWIN'] = form_record.groupby(['horseid'])['HNOWIN'].fillna(method='ffill')

form_record['HNOPLACE'] = form_record.groupby(['horseid'])['place'].shift(+1, fill_value=0)
form_record['HNOPLACE'] = form_record.groupby(['horseid'])['HNOPLACE'].fillna(method='ffill')

form_record['HNOSHOW'] = form_record.groupby(['horseid'])['show'].shift(+1, fill_value=0)
form_record['HNOSHOW'] = form_record.groupby(['horseid'])['HNOSHOW'].fillna(method='ffill')

# JWINPER ('j_win_rate', 'j_place_rate', 'j_show_rate')

# JNOWIN ('JNOWIN', 'JNOPLACE', 'JNOSHOW')

# DAYSINCE
'''
Do in the final df
'''

# AGEBYDIST
'''
Do in the final df as AGE in racecard
'''

# DRAWBYTRACK
form_record['Draw'] = form_record['Draw'].str.replace('  ', "0")
form_record['Draw'] = form_record['Draw'].astype(int)
form_record['DRAWBYTRACK'] = np.where(form_record['RC'] == 'ST', 1 * form_record['Draw'], 2 * form_record['Draw'])

In [ ]:
form_record.shape

In [ ]:
form_record_CU = form_record[['RaceIndex', 'Pla', 'Date', 'RC/Track/Course', 'Dist', 'Ground',
                              'RaceClass', 'Draw', 'Rating', 'Trainer', 'Jockey', 'LBW',
                              'WinOdds', 'WTCARRIED', 'RunPo', 'FinishTime', 'Declare_Horse_Wt',
                              'WTCARRIEDIST', 'WEIGHTDIST', 'horseid', 'formatted_date', 'ind',
                              'class', 'RC', 'Track', 'Course', 'FT', 'recordtime',
                              'SPRAT', 'AVESPRAT_check', 'AVESPRAT', 'AVESPRAT_ind', 'WV_3_ind',
                              'careerstart', 'win1', 'win2', 'win3', 'win', 'win_rate_check',
                              'win_rate', 'place', 'place_rate_check', 'place_rate', 'show',
                              'show_rate_check', 'show_rate', 'HNOWIN', 'HNOPLACE', 'HNOSHOW', 'J_started', 'j_win',
                              'j_place', 'j_show', 'j_win_rate_check', 'j_win_rate',
                              'j_place_rate_check', 'j_place_rate', 'j_show_rate_check',
                              'j_show_rate', 'JNOWIN', 'JNOPLACE', 'JNOSHOW', 'debut',
                              'DRAWBYTRACK']]
form_record_CU.to_csv (r'C:\Users\chunl\Desktop\form_record_CU.csv', index = False, header=True)

### Edelman, 2007

In [ ]:
# finish position of previous race (done)

'''
= place_last
'''

# bookmakers' odds of previous race
form_record['odd_pre'] = form_record.groupby(['horseid'])['WinOdds'].shift(+1, fill_value=0)
form_record["odd_pre"].mask(form_record['odd_pre'] == '  ', np.nan, inplace=True)
form_record['odd_pre'] = form_record.groupby(['horseid'])['odd_pre'].fillna(method='ffill')
form_record['odd_pre'] = form_record['odd_pre'].astype(float)

# total prizemoney in previous race
form_record['prize_money'] = form_record['prize_money'].str.replace(",", "")
form_record['prize_money'] = form_record['prize_money'].astype(int)
form_record['pm_pre'] = form_record.groupby(['horseid'])['prize_money'].shift(+1, fill_value=0)
#form_record['pm_pre'] = form_record.groupby(['horseid'])['pm_pre'].fillna(method='ffill')

# final call position of previous race = last running position of previous race (done)
'''
= nfcp_pre
'''

# indicator, no call position of previous race (done)
'''
= WV_ind
'''

# distance of previous race
form_record['Dist_pre'] = form_record.groupby(['horseid'])['Dist'].shift(+1, fill_value=0)

# odd * prizemoney of previous race
'''
form_record['prize_money'] = form_record['prize_money'].str.replace(",", "")
form_record['prize_money'] = form_record['prize_money'].astype(int)

form_record["WinOdds"].mask(form_record['WinOdds'] == '  ', 0, inplace=True)
form_record['WinOdds'] = form_record['WinOdds'].astype(float)

form_record['oddpm'] = form_record['prize_money'] * form_record['WinOdds']
form_record['oddpm_pre'] = form_record.groupby(['horseid'])['oddpm'].shift(+1, fill_value=0)
'''
form_record["WinOdds"].mask(form_record['WinOdds'] == '  ', 0, inplace=True)
form_record['WinOdds'] = form_record['WinOdds'].astype(float)
form_record['oddpm_pre'] = form_record['pm_pre'] * form_record['odd_pre']


# weight carried in previous race
form_record['WTCARRIED_pre'] = form_record.groupby(['horseid'])['WTCARRIED'].shift(+1, fill_value=0)

# days since previous race
'''
Do in last stage
'''

# total prizemoney in current race
'''
Do in last stage
'''

# weighted carried in current race (done)
'''
= WTCARRIED
'''

# distance of current race (done)
'''
= Dist
'''

In [ ]:
form_record.shape

In [ ]:
# Export to check
form_record_Edelman = form_record[['RaceIndex', 'Pla', 'Date', 'RC/Track/Course', 'Ground',
       'RaceClass', 'Draw', 'Rating', 'Trainer', 'Jockey', 'LBW',
       'WinOdds', 'RunPo', 'FinishTime', 'Declare_Horse_Wt',
       'horseid', 'formatted_date', 'ind', 'place_last', 'odd_pre', 'pm_pre', 'nfcp_pre', 'WV_ind', 'Dist_pre', 'oddpm_pre', 'WTCARRIED_pre', 'prize_money', 'WTCARRIED', 'Dist',    
       'debut'
       ]]
form_record_Edelman.to_csv (r'C:\Users\chunl\Desktop\form_record_Edelman.csv', index = False, header=True)

### Bolton_Chapman

In [ ]:
# LIFE%WIN = Horse win rate (time span?)


In [ ]:
# W/RACE
import numpy as np
'''
Season Indicatior to calculate the CUMULATIVE season PRIZE MONEY (check)
'''

#form_record['month'] = form_record['Date'].str[5:7]
form_record['season'] = np.where( form_record['Date'].dt.month >= 9, form_record['Date'].dt.year +1, form_record['Date'].dt.year )

# change prize_money to int
#form_record['prize_money'] = form_record['prize_money'].str.replace(',', '')
#form_record['prize_money'] = form_record['prize_money'].astype(int)

# winnings per match
form_record['winnings_per_match1'] = np.where(form_record.Pla == '1', form_record['prize_money'] * 0.57, 0)
form_record['winnings_per_match2'] = np.where(form_record.Pla == '2', form_record['prize_money'] * 0.22, 0)
form_record['winnings_per_match3'] = np.where(form_record.Pla == '3', form_record['prize_money'] * 0.115, 0)
form_record['winnings_per_match4'] = np.where(form_record.Pla == '4', form_record['prize_money'] * 0.06, 0)
form_record['winnings_per_match5'] = np.where(form_record.Pla == '5', form_record['prize_money'] * 0.035, 0)
form_record['winnings_per_match'] = form_record['winnings_per_match1'] + form_record['winnings_per_match2'] + form_record['winnings_per_match3'] + form_record['winnings_per_match4'] + form_record['winnings_per_match5']

form_record['RACE'] = form_record.groupby(['horseid', 'season']).cumcount()

# sum over the season for checking
form_record['season_pm'] = form_record.groupby(['horseid', 'season'])['winnings_per_match'].transform('sum')

# calculate the rolling prize money (W/RACE)
form_record = form_record.sort_values(by=['horseid', 'formatted_date'])

form_record['WINNINGS'] = form_record.groupby(['horseid', 'season'])['winnings_per_match'].apply(lambda x: x.cumsum())
form_record['W/RACE'] = form_record.groupby(['horseid', 'season'])['WINNINGS'].shift(+1, fill_value=0)
form_record['W/RACE'] = form_record['W/RACE'] / form_record['RACE']

In [ ]:
# LSPEDRAT
form_record['LSPEDRAT'] = form_record.groupby(['horseid'])['SPRAT'].shift(+1, fill_value=0)
form_record['LSPEDRAT'] = form_record.groupby(['horseid'])['LSPEDRAT'].fillna(method='ffill')

In [ ]:
# WEIGHT (done)
# POSTPOS (done)
# NEWDIST
form_record['Dist_1'] = form_record.groupby(['horseid'])['Dist'].shift(+1)
form_record['Dist_2'] = form_record.groupby(['horseid'])['Dist'].shift(+2)
form_record['Dist_3'] = form_record.groupby(['horseid'])['Dist'].shift(+3)
form_record['Dist_4'] = form_record.groupby(['horseid'])['Dist'].shift(+4)

form_record['NEWDIST_1'] = np.where( (form_record['Dist'] - form_record['Dist_1']) > 200, 1, 0)
form_record['NEWDIST_2'] = np.where( (form_record['Dist'] - form_record['Dist_2']) > 200, 1, 0)
form_record['NEWDIST_3'] = np.where( (form_record['Dist'] - form_record['Dist_3']) > 200, 1, 0)
form_record['NEWDIST_4'] = np.where( (form_record['Dist'] - form_record['Dist_4']) > 200, 1, 0)

form_record['NEWDIST_NO'] = form_record['NEWDIST_1'] + form_record['NEWDIST_2'] + form_record['NEWDIST_3'] + form_record['NEWDIST_4']
form_record['NEWDIST'] = np.where(form_record['NEWDIST_NO'] >= 3, 1, 0)

#form_record.drop(['Dist_1', 'Dist_2', 'Dist_3', 'Dist_4', 'NEWDIST_1', 'NEWDIST_2', 'NEWDIST_3', 'NEWDIST_4', 'NEWDIST_NO'], axis=1, inplace=True)

In [ ]:
form_record_Chapman = form_record[['RaceIndex', 'Pla', 'Date', 'RC/Track/Course', 'Dist', 'Ground',
       'RaceClass', 'horseid', 'formatted_date', 'season', 'winnings_per_match1',
       'winnings_per_match2', 'winnings_per_match3',
       'winnings_per_match4', 'winnings_per_match5', 'winnings_per_match',
       'season_pm', 'WINNINGS', 'W/RACE', 'FT', 'SPRAT', 'LSPEDRAT', 'WV_ind', 'Dist_1', 'Dist_2',
       'Dist_3', 'Dist_4', 'NEWDIST_1', 'NEWDIST_2', 'NEWDIST_3',
       'NEWDIST_4', 'NEWDIST_NO', 'NEWDIST']]
form_record_Chapman.to_csv (r'C:\Users\chunl\Desktop\form_record_Chapman.csv', index = False, header=True)

In [ ]:
# JOCK%WIN (done)
# JOCK#WIN (done)
# JMISDATA
form_record.shape

In [ ]:
form_record.columns.values

In [ ]:
'''
2008
'''

# days for recency-weighted
from datetime import datetime
race_result_concat['Date'] =  pd.to_datetime(race_result_concat['date'], format='%Y%m%d')

race_result_concat = race_result_concat.sort_values(by=['Date', 'horse_name'])
race_result_concat['days'] = race_result_concat.groupby(['horse_name'])['Date'].diff().dt.days.fillna(0)
race_result_concat['rw'] = race_result_concat.groupby(['horse_name'])['days'].cumsum()
race_result_concat['rw_t'] = 0
race_result_concat["rw_t"].mask(race_result_concat['rw'] > 0, (1/race_result_concat['rw']), inplace=True)

race_result_concat['NO'] = race_result_concat.groupby('horseid').cumcount()

In [ ]:
#race_result_concat.sort_values(by=['Date', 'horse_name'])[['Horse', 'Date', 'n_fp', 'date', 'days', 'rw', 'rw_t', 'NO']]

In [ ]:
#trial = race_result_concat.loc[(race_result_concat.Horse == "SPEED VISION(B211)") | (race_result_concat.Horse == 'NAMJONG TURBO(N250)')][['Horse', 'Date', 'n_fp', 'date', 'days', 'rw', 'rw_t', 'NO']]
#trial

In [ ]:
#out = trial.groupby('Horse').apply(recency)
#trial = trial.merge(out, on = ['Date', 'n_fp', 'date', 'days', 'rw', 'rw_t', 'NO'])
#trial

In [ ]:
import fractions
from fractions import Fraction

def mixed_to_float(x):
    return float(sum(fractions.Fraction(term) for term in x.split()))

In [ ]:
# LENGTHBEHIND

# LBW
#print(race_result_concat.loc[race_result_concat.LBW == ' ']['Plc.'].value_counts())

race_result_concat['LBW_float'] = race_result_concat['LBW']

# pre-process
race_result_concat['LBW_float'] = race_result_concat['LBW_float'].str.replace("-", " ")
race_result_concat['LBW_float'].mask(race_result_concat['LBW_float'] == " ", '0', inplace=True)

# This is for applying mixed_to_float
#race_result_concat['LBW_float'].mask(race_result_concat['Plc.'] == '1', '0', inplace=True)
#race_result_concat['LBW_float'].mask(race_result_concat['Plc.'] == '1 DH', '0', inplace=True)
#race_result_concat['LBW_float'] = np.where(race_result_concat['Plc.'] == '1', '0', race_result_concat['LBW_float'])
#race_result_concat['LBW_float'] = np.where(race_result_concat['Plc.'] == '1 DH', '0', race_result_concat['LBW_float'])

# pre-process
race_result_concat['LBW_float'] = race_result_concat['LBW_float'].str.replace('+', "")
race_result_concat['LBW_float'] = race_result_concat['LBW_float'].str.replace("NOSE", "1/50")
race_result_concat['LBW_float'] = race_result_concat['LBW_float'].str.replace("SH", "1/20")
race_result_concat['LBW_float'] = race_result_concat['LBW_float'].str.replace("HD", "1/10")
race_result_concat['LBW_float'] = race_result_concat['LBW_float'].str.replace("N", "1/4")
race_result_concat['LBW_float'] = race_result_concat['LBW_float'].str.replace("DH", "")
race_result_concat['LBW_float'] = race_result_concat['LBW_float'].str.replace("ML", '99')
race_result_concat['LBW_float'] = race_result_concat['LBW_float'].str.replace("TO", '99')
race_result_concat[['LBW_float']] = race_result_concat[['LBW_float']].fillna(value=99)
race_result_concat['LBW_float'] = race_result_concat['LBW_float'].astype(str)

# extract distance for normalizing LBW
race_result_concat[['class', 'distance', 'rating']] = race_result_concat.cdr.str.split(" - ", expand=True)
race_result_concat['class'] = race_result_concat['class'].str.replace("Class ", "")
race_result_concat['distance'] = race_result_concat['distance'].str.replace("M", "")
race_result_concat['distance'] = race_result_concat['distance'].astype(int)

# apply the function
race_result_concat['LBW_float'] = race_result_concat['LBW_float'].apply(mixed_to_float)
race_result_concat['n_LBW'] = race_result_concat['LBW_float'] / race_result_concat['distance']


In [ ]:
race_result_concat[['Plc.', 'distance', 'LBW', 'LBW_float', 'n_LBW']]

In [ ]:
oneday = pd.Timedelta(days=1)

def recency(frame):
    df = frame[['Horse', 'Date', 'n_fp', 'date', 'days', 'rw', 'rw_t', 'NO', 'break_po', 'n_LBW']].reset_index(drop=True)
    df = df.sort_values(by=['Date'])
    #df = frame.sort_values(by=['horse_name', 'Date'])[['Horse', 'Date', 'n_fp', 'date', 'days', 'rw', 'rw_t', 'NO', 'break_po', 'n_LBW']].reset_index(drop=True)
    #FINISH1 = []
    for i in range(0, df.shape[0]):
        day = ( (df["Date"][i] - df["Date"][0:i]) / oneday).to_numpy()
        recency = np.sum( ( 1/day * df["n_fp"][0:i].to_numpy().T ) / np.sum(1/day) )
        #firstcall = np.sum( ( 1/day * df["break_po"][0:i].to_numpy().T) ) / np.sum(1/day)
        #WIN = np.sum( ( df["rw_t"][0:i].to_numpy() * df[[" "]][0:i].to_numpy()[::-1].T) ) / np.sum(df["rw_t"][0:i].to_numpy())
        FINISH2 = df["n_fp"][0:i].mean()
        LASTRACE = (1 / df.loc[df.index[i],'days']) * df.loc[df.index[i-1],'n_fp']
        LENGTHBEHIND = np.sum( ( 1/day * df["n_LBW"][0:i].to_numpy().T) ) / np.sum(1/day)
        #np.sum( ( grp.iloc[0:i, 5].to_numpy() * grp.iloc[0:i, 1].to_numpy()[::-1].T) ) / np.sum(grp.iloc[0:i, 5].to_numpy())
        #FINISH1 = np.append(FINISH1, recency)
        #grp['FINISH1'] = [FINISH1]
        df.loc[df.index[i],'FINISH1'] = recency
        #df.loc[df.index[i],'FIRSTCALL'] = firstcall
        df.loc[df.index[i],'FINISH2'] = FINISH2
        df.loc[df.index[i],'LASTRACE'] = LASTRACE
        df.loc[df.index[i],'LENGTHBEHIND'] = LENGTHBEHIND

    df.loc[df.index[0],'FINISH1'] = 0
    #df.loc[df.index[0],'FIRSTCALL'] = 0
    df.loc[df.index[0],'FINISH2'] = 0
    df.loc[df.index[0],'LASTRACE'] = 0
    df.loc[df.index[0],'LENGTHBEHIND'] = 0
    df = df.drop(['n_fp', 'date', 'days', 'rw', 'rw_t', 'NO', 'break_po', 'n_LBW'], axis = 1)
    frame = frame.merge(df, on = ['Horse', 'Date'])
    return frame

In [ ]:
'''
oneday = pd.Timedelta(days=1)

df = race_result_concat.sort_values(by=['horse_name', 'Date'])[['Horse', 'Date', 'n_fp', 'date', 'days', 'rw', 'rw_t', 'NO', 'break_po', 'n_LBW']].reset_index()

for i in range(0, 6):
    #print( (df["Date"][i], df["Date"][0:i]) )
    #print( (df["Date"][i] - df["Date"][0:i]) )
    day = ( (df["Date"][i] - df["Date"][0:i]) / oneday).to_numpy()
    print(day)
    print(df["n_fp"][0:i].to_numpy().T)
    recency = np.sum( ( 1/day * df[["n_fp"]][0:i].to_numpy().T ) / np.sum(1/day) )
    firstcall = np.sum( ( 1/day * df[["break_po"]][0:i].to_numpy().T) ) / np.sum(1/day)
    FINISH2 = df["n_fp"][0:i].mean()
    LASTRACE = (1 / df.loc[df.index[i],'days']) * df.loc[df.index[i-1],'n_fp']
    LENGTHBEHIND = np.sum( ( 1/day * df[["n_LBW"]][0:i].to_numpy().T) ) / np.sum(1/day)
    print(1/day * df["n_fp"][0:i].to_numpy().T)
    #print(np.sum( ( 1/day * df[["n_fp"]][0:i].to_numpy().T ) / np.sum(1/day) ) )
    #print('\n')
    df.loc[df.index[i],'FINISH1'] = recency
    df.loc[df.index[i],'FIRSTCALL'] = firstcall
    df.loc[df.index[i],'FINISH2'] = FINISH2
    df.loc[df.index[i],'LASTRACE'] = LASTRACE
    df.loc[df.index[i],'LENGTHBEHIND'] = LENGTHBEHIND
df'''

In [ ]:
# FINISH1, WINHISTORY, FINISH2, LASTRACE, LENGTHBEHIND
import time
start = time.time()

race_result_concat = race_result_concat.groupby('Horse').apply(recency)

print(time.time() - start)

In [ ]:
# AVESPRAT (~ beyes)
# need class, distance and course

In [ ]:
# 120-day half-life

race_result_concat['prize_money'] = race_result_concat['prize_money'].str.replace(',', '')
race_result_concat['prize_money'] = race_result_concat['prize_money'].astype(int)
race_result_concat['Draw'] = race_result_concat['Draw'].astype(int)
# do not normalize (n_Draw*Plc = vain)
# 
race_result_concat['POSTADV'] = race_result_concat.groupby(['date_match'])['Plc.'].transform('max') * race_result_concat['distance'] * race_result_concat['Draw']

# recency-weighted past normalized finishing position, recency-weighted past race competitive level, recency-weighted past post position advantage (post*distance*no. of competitors)(need research)

def recency120(frame):
    df = frame[['Horse', 'Date', 'n_fp', 'date', 'days', 'rw', 'rw_t', 'NO', 'break_po', 'n_LBW', 'prize_money', 'POSTADV']].reset_index(drop=True)
    df = df.sort_values(by=['Date'])
    #df = frame.sort_values(by=['horse_name', 'Date'])[['Horse', 'Date', 'n_fp', 'date', 'days', 'rw', 'rw_t', 'NO', 'break_po', 'n_LBW', 'prize_money', 'POSTADV']].reset_index(drop=True)
    df['FINISH120'] = df.groupby(['Horse'])['n_fp'].transform(lambda x: x.ewm(halflife='120 days', times=df['Date']).mean())
    df['COMP120'] = df.groupby(['Horse'])['prize_money'].transform(lambda x: x.ewm(halflife='120 days', times=df['Date']).mean())
    df['POSTADV120'] = df.groupby(['Horse'])['POSTADV'].transform(lambda x: x.ewm(halflife='120 days', times=df['Date']).mean())
    df['LENGTHBEHIND120'] = df.groupby(['Horse'])['n_LBW'].transform(lambda x: x.ewm(halflife='120 days', times=df['Date']).mean())
    #df.loc[df.index[0],'FINISH120'] = 0
    df = df.drop(['n_fp', 'date', 'days', 'rw', 'rw_t', 'NO', 'break_po', 'n_LBW', 'prize_money', 'POSTADV'], axis = 1)
    frame = frame.merge(df, on = ['Horse', 'Date'])
    return frame

In [ ]:
race_result_concat.reset_index(inplace=True, drop=True)
race_result_concat = race_result_concat.groupby('Horse').apply(recency120)
race_result_concat.reset_index(inplace=True, drop=True)
race_result_concat['FINISH120'] = race_result_concat.groupby(['horseid'])['FINISH120'].shift(+1, fill_value=0)

# prizemoney known before the race, no shift is needed
#race_result_concat['COMP120'] = race_result_concat.groupby(['horseid'])['COMP120'].shift(+1, fill_value=0)

# post known before the race, no shift is needed
#race_result_concat['POSTADV120'] = race_result_concat.groupby(['horseid'])['POSTADV120'].shift(+1, fill_value=0)

race_result_concat['LENGTHBEHIND120'] = race_result_concat.groupby(['horseid'])['LENGTHBEHIND120'].shift(+1, fill_value=0)

In [ ]:
race_result_concat[['Horse', 'Date' , 'n_fp', 'FINISH1', 'FINISH120', 'n_LBW', 'LENGTHBEHIND', 'LENGTHBEHIND120', 'prize_money', 'COMP120', 'POSTADV120']]

In [ ]:
# 1yr recency: Recency-weighted past jockey/trainer normalized finish.

def J_recency_1yr(frame):
    df = frame[['Jockey', 'Date', 'match', 'n_fp', 'date', 'days', 'rw', 'rw_t', 'NO', 'break_po', 'n_LBW', 'prize_money']].reset_index(drop=True)
    df = df.sort_values(by=['Jockey', 'Date', 'match'])
    # Jockey
    #df['J_FINISH_1yr'] = df.groupby(['Jockey'])['n_fp'].transform(lambda x: x.ewm(halflife='365 days', times=df['Date']).mean())
    df['J_COMP_1yr'] = df.groupby(['Jockey'])['prize_money'].transform(lambda x: x.ewm(halflife='365 days', times=df['Date']).mean())
    #df.loc[df.index[0],'FINISH120'] = 0
    df = df.drop(['n_fp', 'date', 'days', 'rw', 'rw_t', 'NO', 'break_po', 'n_LBW', 'prize_money'], axis = 1)
    frame = frame.merge(df, on = ['Jockey', 'Date', 'match'])
    return frame

In [ ]:
race_result_concat = race_result_concat.groupby('Jockey').apply(J_recency_1yr)
race_result_concat.reset_index(inplace=True, drop=True)
#race_result_concat['J_FINISH_1yr'] = race_result_concat.groupby(['Jockey'])['J_FINISH_1yr'].shift(+1, fill_value=0)

# prizemoney known before the race, no shift is needed
#race_result_concat['J_COMP_1yr'] = race_result_concat.groupby(['Jockey'])['J_COMP_1yr'].shift(+1, fill_value=0)

In [ ]:
race_result_concat.shape

In [ ]:
def T_recency_1yr(frame):
    df = frame[['Horse', 'Trainer', 'Date', 'match', 'n_fp', 'date', 'days', 'rw', 'rw_t', 'NO', 'break_po', 'n_LBW', 'prize_money']].reset_index(drop=True)
    df = df.sort_values(by=['Trainer', 'Date', 'match'])
    # Trainer
    #df['T_FINISH_1yr'] = df.groupby(['Trainer'])['n_fp'].transform(lambda x: x.ewm(halflife='365 days', times=df['Date']).mean())
    df['T_COMP_1yr'] = df.groupby(['Trainer'])['prize_money'].transform(lambda x: x.ewm(halflife='365 days', times=df['Date']).mean())
    #df.loc[df.index[0],'FINISH120'] = 0
    df = df.drop(['n_fp', 'date', 'days', 'rw', 'rw_t', 'NO', 'break_po', 'n_LBW', 'prize_money'], axis = 1)
    frame = frame.merge(df, on = ['Horse', 'Trainer', 'Date', 'match'])
    return frame

In [ ]:
race_result_concat = race_result_concat.groupby('Trainer').apply(T_recency_1yr)
race_result_concat.reset_index(inplace=True, drop=True)
#race_result_concat['T_FINISH_1yr'] = race_result_concat.groupby(['Trainer'])['T_FINISH_1yr'].shift(+1, fill_value=0)

# prizemoney known before the race, no shift is needed
#race_result_concat['T_COMP_1yr'] = race_result_concat.groupby(['Trainer'])['T_COMP_1yr'].shift(+1, fill_value=0)

In [ ]:
race_result_concat.shape

In [ ]:
# WEIGHT(REL)

race_result_concat['date_match'] = race_result_concat['date'].astype(str) + race_result_concat['match'].astype(str)
race_result_concat['Actual Wt.'] = race_result_concat['Actual Wt.'].astype(int)
race_result_concat['WEIGHT(REL)'] = race_result_concat['Actual Wt.'] - race_result_concat.groupby(['date_match'])['Actual Wt.'].transform('mean')

# WEIGHT(REL)DIST
race_result_concat['WEIGHT(REL)DIST'] = race_result_concat['WEIGHT(REL)'] * race_result_concat['distance']

# FIRSTCALL
'''
Did in above
'''

In [ ]:
# FIRSTCALLDIST
race_result_concat['FIRSTCALLDIST'] = race_result_concat['FIRSTCALL'] * race_result_concat['distance']
race_result_concat['FIRSTCALL120DIST'] = race_result_concat['FIRSTCALL120'] * race_result_concat['distance']

In [ ]:
'''
check any duplicate
'''

race_result_concat.columns

In [ ]:
# DAYSINCE
'''
Do in the last stage
'''
# CAREERSTARTS
'''
= careerstart / div
'''

# FINISH2
'''
Did one in above and one in form_record for checking
'''

# COMPETITORS1
'''
If have time
'''

# COMPETITORS2
'''
If have time
'''

# LASTRACE
'''
Above
'''

# SPECIALDIFF
'''
If have time
'''

In [ ]:
# DISTANCEPREF
# TRACKPREF
# JOCKEYHISTORY
# JOCKETCURRENT
# TRACKWORK
# STRENGTH

In [ ]:
race_result_concat.columns.values

In [ ]:
race_result_concat = race_result_concat[['Plc.', 'Horse No.', 'Horse', 'Jockey', 'Trainer', 'Actual Wt.',
       'Declar. Horse Wt.', 'Draw', 'LBW', 'RunningPosition', 'Finish Time',
       'Win Odds', 'date', 'match', 'cdr', 'prize_money', 'horse_name',
       'horseid', 'date_match', 'ind', 'RunPo_1', 'break_po', 'final_call_po',
       'n_final_call_po', 'n_fp', 'Date', 'days', 'rw', 'rw_t', 'NO', 'FINISH1', 'WEIGHT(REL)', 'WEIGHT(REL)DIST',
       'FIRSTCALL', 'FIRSTCALLDIST', 'LENGTHBEHIND', 'FINISH2', 'LASTRACE', 'class',
       'distance', 'rating', 'FINISH120', 'COMP120', 'POSTADV120', 'LENGTHBEHIND120', 'J_COMP_1yr',
       'T_COMP_1yr', 'FIRSTCALL120', 'FIRSTCALL120DIST'
       ]]

In [ ]:
race_result_concat.shape

In [ ]:
form_record.shape

In [ ]:
# Check before merge

form_record['ind'] = form_record['horseid'] + "_" + form_record['formatted_date'].astype(str)
race_result_concat['ind'] = race_result_concat['horseid'] + "_" + race_result_concat['date'].astype(str)

#print('form_record is missing:', list(set(race_result_concat['ind'].unique()) - set(form_record['ind'].unique()))) # void match
#print("\n")
print('race_result is missing:', len(list(set(form_record['ind'].unique()) - set(race_result_concat['ind'].unique()))))

l = list(set(form_record['ind'].unique()) - set(race_result_concat['ind'].unique()))
form_record.loc[form_record['ind'].isin(l)]['Pla'].value_counts(dropna=False)

In [ ]:
'''
Above tell us all missing in race_result_concat is invalid matches. Thus, we can start merging.
'''
form_record = form_record.merge(race_result_concat[['ind', 'days', 'rw', 'FINISH1', 'WEIGHT(REL)', 'WEIGHT(REL)DIST',
       'FIRSTCALLDIST', 'LENGTHBEHIND', 'FINISH2', 'LASTRACE', 'FINISH120', 'COMP120', 'POSTADV120', 'LENGTHBEHIND120', 'J_COMP_1yr',
       'T_COMP_1yr', 'FIRSTCALL120DIST', # FIRSTCALL120 merged in above
       'distance', 'rating']], on = 'ind', how = 'left')
form_record.columns.values

In [ ]:
form_record.shape

In [ ]:
race_result_concat.to_csv (r'C:\Users\chunl\Desktop\race_result_concat.csv', index = False, header=True)

### Lessmann 2009

In [ ]:
# disavesr
form_record['disavesr_check'] = form_record.groupby(['horseid', 'Dist'])['SPRAT'].apply(lambda x: x.expanding().mean())
form_record['disavesr'] = form_record.groupby(['horseid', 'Dist'])['disavesr_check'].shift(+1, fill_value=0)
form_record['disavesr'] = form_record.groupby(['horseid'])['disavesr'].fillna(method='ffill')

In [ ]:
# go_avesr
form_record['go_avesr_check'] = form_record.groupby(['horseid', 'Ground'])['SPRAT'].apply(lambda x: x.expanding().mean())
form_record['go_avesr'] = form_record.groupby(['horseid', 'Ground'])['go_avesr_check'].shift(+1, fill_value=0)
form_record['go_avesr'] = form_record.groupby(['horseid'])['go_avesr'].fillna(method='ffill')

In [ ]:
form_record_Lessmann = form_record[['RaceIndex', 'Pla', 'Date', 'RC/Track/Course', 'Dist', 'Ground', 'RaceClass', 'horseid', 'formatted_date', 
                                    'FT', 'FinishTime', 'recordtime', 'SPRAT', 'LSPEDRAT', 'AVESPRAT', 'AVESPRAT_ind', 'NEWDIST', 'disavesr_check', 'disavesr', 'go_avesr_check', 'go_avesr', 
                                    'WV_ind_check', 'WV_ind', 'WV_3_ind', 'debut']]
form_record_Lessmann.to_csv (r'C:\Users\chunl\Desktop\form_record_Lessmann.csv', index = False, header=True)

In [ ]:
form_record_Lessmann.shape

### Recency
#### 'win_rate', 'place_rate', 'show_rate', 'j_win_rate','j_place_rate', 'j_show_rate', 't_win_rate','t_place_rate', 't_show_rate'
#### 'WINHISTORY'

In [ ]:
def rate_recency(frame):
    df = frame[['Horse', 'Jockey', 'Trainer', 'Date', 'win_rate', 'place_rate', 'show_rate', 'j_win_rate','j_place_rate', 'j_show_rate', 't_win_rate', 't_place_rate', 't_show_rate']].reset_index(drop=True)
    df = df.sort_values(by=['Date'])
    
    #df['T_FINISH_1yr'] = df.groupby(['Trainer'])['n_fp'].transform(lambda x: x.ewm(halflife='365 days', times=df['Date']).mean())
    df['win_rate_120'] = df.groupby(['Horse'])['win_rate'].transform(lambda x: x.ewm(halflife='120 days', times=df['Date']).mean())
    df['place_rate_120'] = df.groupby(['Horse'])['place_rate'].transform(lambda x: x.ewm(halflife='120 days', times=df['Date']).mean())
    df['show_rate_120'] = df.groupby(['Horse'])['show_rate'].transform(lambda x: x.ewm(halflife='120 days', times=df['Date']).mean())
    
    df = df.drop(['Jockey', 'Trainer', 'win_rate', 'place_rate', 'show_rate', 'j_win_rate','j_place_rate', 'j_show_rate', 't_win_rate','t_place_rate', 't_show_rate'], axis = 1)
    frame = frame.merge(df, on = ['Horse', 'Date'], how = 'left')
    return frame

In [ ]:
form_record = form_record.groupby('Horse').apply(rate_recency)
form_record.reset_index(inplace=True, drop=True)

# rate shifted already

In [ ]:
form_record[['Horse', 'Date', 'win_rate', 'win_rate_120', 'place_rate', 'place_rate_120', 'show_rate', 'show_rate_120']].head(50)

### Final form_record

In [ ]:
form_record.columns.values

In [ ]:
form_record_final = form_record[['RaceIndex', 'Pla', 'Date', 'RC/Track/Course', 'Dist', 'Ground',
                                 'RaceClass', 'Draw', 'Rating', 'Trainer', 'Jockey', 'LBW',
                                 'WinOdds', 'WTCARRIED', 'RunPo', 'FinishTime', 'Declare_Horse_Wt',
                                 'horseid', 'formatted_date', 'ind',
                                 'win_rate', 'place_rate', 'show_rate', 'j_win_rate','j_place_rate', 'j_show_rate', 't_win_rate','t_place_rate', 't_show_rate',
                                 'avg_break_po', 'place_last', 'WV_ind', 'place_last_3', 'WV_3_ind', 'WTCARRIEDIST', 'WEIGHTDIST', 'AVESPRAT', 'AVESPRAT_ind',
                                 'HNOWIN', 'HNOPLACE', 'HNOSHOW', 'JNOWIN', 'JNOPLACE', 'JNOSHOW', 'TNOWIN', 'TNOPLACE', 'TNOSHOW', 
                                 'DAYSINCE', 'DRAWBYTRACK', 'odd_pre', 'pm_pre',
                                 'nfcp_pre', 'Dist_pre', 'oddpm_pre', 'WTCARRIED_pre', 'prize_money', 'season', 'RACE', 'WINNINGS', 'W/RACE', 'LSPEDRAT', 'SPRAT_ind', 'NEWDIST', 
                                 'FINISH1', 'WEIGHT(REL)', 'WEIGHT(REL)DIST', 'FIRSTCALL', 'FIRSTCALLDIST', 'LENGTHBEHIND', 'careerstart', 'FINISH2', 'LASTRACE',
                                 'disavesr', 'go_avesr', 'debut',
                                 'FINISH120', 'COMP120', 'POSTADV120', 'LENGTHBEHIND120', 'J_COMP_1yr', 'T_COMP_1yr', 'FIRSTCALL120','FIRSTCALL120DIST']]
#'win_rate_120', 'place_rate_120', 'show_rate_120', 'j_win_rate_1yr', 'j_place_rate_1yr', 'j_show_rate_1yr', 't_win_rate_1yr', 't_place_rate_1yr', 't_show_rate_1yr',

In [ ]:
form_record_final.shape

In [ ]:
#form_record_final.loc[form_record_final.ind == 'HK_2011_N195_20121118']

### merge form_record (after FE) into race_result 
### Start to build the final df

In [ ]:
form_record_final['ind'] = form_record_final['horseid'] + "_" + form_record_final['formatted_date'].astype(str)
race_result['ind'] = race_result['horseid'] + "_" + race_result['date'].astype(str)

print('form_record is missing:', list(set(race_result['ind'].unique()) - set(form_record_final['ind'].unique()))) # void match
print("\n")
print('race_result is missing:', len(list(set(form_record_final['ind'].unique()) - set(race_result['ind'].unique()))))

In [ ]:
print('form_record is missing:', len(list(set(race_result['ind'].unique()) - set(form_record_final['ind'].unique()))))

l = list(set(race_result['ind'].unique()) - set(form_record_final['ind'].unique()))
print(race_result.loc[race_result['ind'].isin(l)]['Plc.'].value_counts(dropna=False))

In [ ]:
print(race_result.shape)

race_result = race_result.merge(form_record_final, on = ['ind'], how='left', suffixes=('', '_dup'))
print(race_result.shape)
race_result.head()

In [ ]:
race_result.columns.values

In [ ]:
'''race_result[['class', 'distance', 'rating']] = race_result.cdr.str.split(" - ", expand=True)
race_result['class'] = race_result['class'].str.replace("Class ", "")
race_result['distance'] = race_result['distance'].str.replace("M", "")
race_result['distance'] = race_result['distance'].astype(int)'''

### Merge best time from racecard to race_result ###

In [ ]:
race_result["brand_no"] = race_result['horseid'].str[-4:]

# make sure race_result is completed first
# by comparing it to racecard

race_result['check'] = race_result['date'].astype(str) + "_" + race_result['match'].astype(str) + "_" + race_result['brand_no'].astype(str)
racecard['check'] = racecard['date'].astype(str) + "_" +  racecard['match'].astype(str) + "_" + racecard['Brand No.'].astype(str)

In [ ]:
print('racecard is missing:', list(set(race_result['check'].unique()) - set(racecard['check'].unique())))
print("\n")
# This should be blank
print('race_result is missing:', list(set(racecard['check'].unique()) - set(race_result['check'].unique())))

print('racecard is missing:', len(list(set(race_result['check'].unique()) - set(racecard['check'].unique()))))

l = list(set(race_result['check'].unique()) - set(racecard['check'].unique()))
print(race_result.loc[race_result['check'].isin(l)]['Plc.'].value_counts(dropna=False))

# matches with place is the match in 20190323
race_result.loc[(race_result['check'].isin(l)) & (race_result['Plc.'] != 'WV') & (race_result['Plc.'] != 'WX')][['Date']]

In [ ]:
racecard['Best Time'] = racecard['Best Time'].str.replace("-", '00.00.00')
racecard[['Best Time']] = racecard[['Best Time']].fillna(value='00.00.00')
# Best Time check
#from datetime import datetime
#racecard['Best Time'] =  pd.to_datetime(racecard['Best Time'], format='%M.%S.%f').dt.time

In [ ]:
# Turn Best Time into a value
import numpy as np
racecard[['Best_Time_m', 'Best_Time_s']] = racecard['Best Time'].str.split(".", 1, expand=True)
racecard['Best_Time_m'] = racecard['Best_Time_m'].astype(int)
racecard['Best_Time_s'] = racecard['Best_Time_s'].astype(float)
racecard['besttime'] = racecard['Best_Time_m'] * 60 + racecard['Best_Time_s']
racecard['besttime_miss'] = np.where(racecard['besttime'] == 0, 1, 0)
#racecard[['Best Time', 'Best_Time_m', 'Best_Time_s', 'besttime', 'besttime_miss']]

In [ ]:
'''
We need 'Wt.+/- (vs Declaration)', 'Best Time', 'Age', 'Season Stakes', 'Priority'
'''
racecard.rename(columns={'Wt.+/- (vs Declaration)': "WTCARRIEDADJ"}, inplace=True)
racecard.rename(columns={'Season Stakes': "Season_Stakes"}, inplace=True)

In [ ]:
print(race_result.shape)
race_result = race_result.merge(racecard[['besttime', 'besttime_miss', 'WTCARRIEDADJ', 'Age', 'Season_Stakes', 'check']], on = ['check'], how='left')
print(race_result.shape)

In [ ]:
print(race_result['Plc.'].value_counts(dropna=False))
print(race_result['Pla'].value_counts(dropna=False))

race_result = race_result[race_result['Plc.'] != 'WV']
race_result = race_result[race_result['Plc.'] != 'WV A']
race_result = race_result[race_result['Plc.'] != 'PU']
race_result = race_result[race_result['Plc.'] != 'UR']
race_result = race_result[race_result['Plc.'] != 'WX A']
race_result = race_result[race_result['Plc.'] != 'WX']
race_result = race_result[race_result['Plc.'] != 'FE']
race_result = race_result[race_result['Plc.'] != 'VOID']
race_result = race_result[race_result['Plc.'] != 'DNF']
race_result = race_result[race_result['Plc.'] != 'TNP']
race_result = race_result[race_result['Plc.'] != 'DISQ']
race_result = race_result[race_result['Plc.'] != 'WXNR']
race_result['Plc.'] = race_result['Plc.'].str.replace(' DH', '')
print(race_result['Plc.'].value_counts(dropna=False))
print(race_result['Pla'].value_counts(dropna=False))

In [ ]:
# fillna
print(race_result.shape)

race_result['LBW'].mask(race_result['Plc.'] == '1', '0', inplace=True)

race_result['disavesr'] = np.where(race_result['debut'] == 1, 0, race_result['disavesr'])
race_result['go_avesr'] = np.where(race_result['debut'] == 1, 0, race_result['go_avesr'])
race_result['nfcp_pre'] = np.where(race_result['debut'] == 1, 0, race_result['nfcp_pre'])
race_result['odd_pre'] = np.where(race_result['debut'] == 1, 0, race_result['odd_pre'])
race_result['AVESPRAT'] = np.where(race_result['debut'] == 1, 0, race_result['AVESPRAT'])
race_result['place_last_3'] = np.where(race_result['debut'] == 1, 0, race_result['place_last_3'])
race_result['avg_break_po'] = np.where(race_result['debut'] == 1, 0, race_result['avg_break_po'])

race_result = race_result[race_result['date'] != 20190323]
race_result = race_result[race_result['RaceClass'] != 'R']
race_result = race_result[race_result['RaceClass'] != '4YO']

print(race_result.shape)

In [ ]:
race_result_na = race_result[race_result.isnull().any(axis=1)]
print(race_result_na.shape)

# NaN because of divided by zero
print(race_result_na['RACE'].value_counts())
race_result_na[['Horse', 'W/RACE', 'season', 'RACE', 'Date']]

In [ ]:
race_result['W/RACE'].mask(race_result['RACE'] == 0, 0, inplace=True)
race_result_na = race_result[race_result.isnull().any(axis=1)]
race_result_na

In [ ]:
print(race_result_na.shape)
link = race_result_na['horseid'].to_numpy()
for i in range(0, race_result_na.shape[0]):
    print('https://racing.hkjc.com/racing/information/Chinese/Horse/OtherHorse.aspx?HorseId=' + link[i])

In [ ]:
# AGET
race_result['Age'] = race_result['Age'].astype(int)
race_result['AGET'] = 0.75
race_result['AGET'] = np.where( ( (race_result['Age'] == 4) | (race_result['Age'] == 5) ), 1, race_result['AGET'])
race_result['AGET'] = np.where( ( (race_result['Age'] == 3) | (race_result['Age'] == 6) ), 0.95, race_result['AGET'])
race_result['AGET'] = np.where( (race_result['Age'] == 7), 0.9, race_result['AGET'])
race_result['AGET'] = np.where( (race_result['Age'] == 8), 0.85, race_result['AGET'])
race_result['AGET'] = np.where( ( (race_result['Age'] == 1) | (race_result['Age'] == 9) ), 0.8, race_result['AGET'])

In [ ]:
# AGEBYDIST

race_result['AGEBYDIST'] = race_result['Age'] * race_result['Dist']

In [ ]:
race_result.to_csv (r'C:\Users\chunl\Desktop\export_dataframe.csv', index = False, header=True)

In [ ]:
race_result.columns.values

In [ ]:
race_result.sort_values(by = ['Horse', 'Date'])[['Horse', 'Date', 'WINNINGS', 'W/RACE', 'Season_Stakes']].head(50)

In [ ]:
#race_result.sort_values(by = ['Horse', 'Date'])[['Horse', 'Date', 'W/RACE', 'Season_Stakes']].head(50)

In [ ]:
race_result['pm_pre']#'840,000'

In [ ]:
# one-hot?
#racecard.Priority.value_counts(dropna=False)

In [ ]:
#racecard.loc[racecard.Priority.isna()]

In [ ]:
#race_result.loc[race_result.Priority == '-']

In [ ]:
#race_result_na.to_csv (r'C:\Users\chunl\Desktop\race_result_na.csv', index = False, header=True)

In [ ]:
#race_result_concat.loc[race_result_concat['RunPo_1'] == 'NaN']
#df_null = race_result_concat[race_result_concat['RunPo_1'].isnull()]